# build predictive model of genes -> STP


1) Use ABA VISp,Hipp gene expression and STP data to fit RF for STP. 

2) test predictions - crossvalidation, test on hippocampal data, test on exc. neurons data?

3) feature selection - compare datasets? - compare with Paul analysis

4) stp factors genes - find better factors using interactions networks?

## 1) fit RF ABA ge -> STP : Hippocampus, VISp

a. select gene sets

b. load gene expression data Hipp, Ent 

c. hippocampus STP data 

d. combine STP and gene expression data for Hippocampus

e.load gene expression data VISp

f.  coretex STP data

g. combine STP and gene expression data for VISp cortex

h. fit RF from sklit-learn

f. test predictions :  cross validation - part of genes, part of STP labels, genes from different cortical areas, genes from hippocampus (Harris 2018)

g. compare gene sets


In [ ]:
import pandas as pd
import numpy as np


do_laptop=1   # Laptop or desctop version

# a. select gene sets - take Paul 2017 best pre-synaptic + CAMs
# Paul 2017


In [ ]:
#####
##
##  SELECT GENE SET
##
###
gs = '15656' #'5188' #'1512'#'5188' #'1512'#'219' #'1512' #'219'


In [ ]:
def get_prot_list(df_stp_ds, name):
    pre_main=df_stp_ds.loc[:,['Protein','Gene rat']]
    pre_main_nm = pre_main.loc[pre_main.loc[:,'Protein'].str.contains(':',na=False),'Protein']


    pre_main_nm=pre_main_nm.reset_index()
    i = pre_main_nm.loc[pre_main_nm.loc[:,'Protein']==name,:]
    i1 = i.loc[:,'index']
    i2 = pre_main_nm.loc[i.index[0]+1,'index']

    Cas = df_stp_ds.loc[np.arange(i1.values[0]+1,i2),'Gene rat']
    Cas=Cas.loc[Cas.loc[:].isna()==False]
    Cas = Cas.str.split('\W+',expand=True).iloc[:,0].values.tolist()
    return Cas

In [ ]:
d8 = '~/Documents/references/transcriptomes to STP/'
if do_laptop:
    d8 = '/Users/stepaniu/Documents/jan_2020/'

df_stp_ds = pd.read_excel(d8+'Datasets_STP_hippocampus.xlsx', sheet_name='Sheet8')

Cas = get_prot_list(df_stp_ds, 'Calcium sensors:')
AZ2 = get_prot_list(df_stp_ds, 'active zone:')

print(Cas)
print(AZ2)


d6=''
if do_laptop:
    d6 = '/Users/stepaniu/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/'

fn6 = 'Paul_2017.xlsx'
df_Paul_gs=pd.read_excel(d6 + fn6,header=2)

d7 = '~/Documents/references/transcriptomes to STP/transcriptomes/Linnerson_2018/'
fn7 = 'GO: synaptic vesicle cycle, mouse.xlsx'
if do_laptop:
    d7 = '/Users/stepaniu/Documents/jan_2020/'
    fn7 = 'GO_ synaptic vesicle cycle, mouse.xlsx'
df_go_priming = pd.read_excel(d7+fn7)
SV_cycle=df_go_priming.loc[:,'Gene/product label'].value_counts().index.tolist()




#print(df_Paul_gs.head())

#select presynaptic :

if (gs=='1512')|(gs=='5188')|(gs=='15656'):
    pre = ['Netrin-Unc5-Slit-Robo',
    'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln',
    'neuronal_IgCAMs',
    'Contactins',
    'Semaphorin+Plexin',
    'LRR+Slitrk+Elfn+Lphn',
    'Eph+EphR',
    'Calcium channel regulatory subunits',
    'Calcium channel poreforming subunits',
    'Calcium channel gamma/TARP subunits',
    'Orphan GPCRs (based off HGNC)',
    'Metabotropic neurotransmitter receptors',
    'Neuropeptide receptors',
    'Trimeric G_protein_alpha',
    'Trimeric G_protein_beta',
    'Trimeric G_protein_gamma',
    'Rho-GEF',
    'Dock-GEF',
    'Dock + Rho-GEF',
    'Complexin-Syt',
    'SNAP-complex',
    'Syntaxin',
    'RIM-proteins/Active zone',
    'bHLH',
    'Nuclear receptors with C4 zinc fingers',
    'Homeo domain factors',
    'High-mobility group (HMG) domain factors',
    'C2H2 zinc finger factors',
    'bZIP',
    'ARID domain factors',
    'Runt domain factors',
    'Other C4 zinc finger-type',
    'C2HC zinc finger factors',
    'SMAD/NF-1 DNA-binding domain factors',
    'TATA-binding proteins',
    'Rel homology region (RHR) factors',
    'Tryptophan cluster factors',
    'MADS box factors',
    'Cold-shock domain factors',
    'CXXC zinc finger factors',
    'Leucine-rich repeat flightless-interacting proteins',
    'STAT domain factors',
    'Grainyhead domain factors'
    ]

if gs=='219':    
    pre = ['Calcium channel regulatory subunits', 'Netrin-Unc5-Slit-Robo',
          'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln', 'neuronal_IgCAMs', 'Contactins',
          'Metabotropic neurotransmitter receptors',      'Dock + Rho-GEF', 'Complexin-Syt']

#pre = ['Syntaxin', 'RIM-proteins/Active zone', 'Dock-GEF','SNAP-complex' ]

#pre = ['RIM-proteins/Active zone'] 

#pre = []


Elfn = ['Elfn1', 'Elfn2']

pregs = []
for i in pre:
    in_i = (df_Paul_gs.loc[:,'Gene Set']==i)
    n_in_i = df_Paul_gs.loc[in_i,'Number of genes'].astype(int).values[0]
    pri = df_Paul_gs.loc[in_i,df_Paul_gs.columns[4:4+n_in_i]]
    pregs = pregs + pri.values.tolist()[0]
    
#pregs = pregs +  Elfn  
if (gs=='1512')|(gs=='5188')|(gs=='15656'):
    pregs = pregs + SV_cycle
    
pregs = pregs +Cas + AZ2 
#pregs = pregs +Cas 

## best gs
#pregs = best_pre

if (gs=='1512')|(gs=='5188')|(gs=='15656'):
    post = ['Netrin-Unc5-Slit-Robo',
    'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln',
    'neuronal_IgCAMs',
    'Contactins',
    'Semaphorin+Plexin',
    'LRR+Slitrk+Elfn+Lphn',
    'Eph+EphR',
    'Calcium channel regulatory subunits',
    'Calcium channel poreforming subunits',
    'Calcium channel gamma/TARP subunits',
    'AMPA+TARP',
    'Orphan GPCRs (based off HGNC)',
    'Metabotropic neurotransmitter receptors',
    'Neuropeptide receptors']


#post = ['Netrin-Unc5-Slit-Robo',
#'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln',
#'neuronal_IgCAMs',
#'Contactins',
#'Semaphorin+Plexin',
#'LRR+Slitrk+Elfn+Lphn',
#'Eph+EphR']

if gs=='219':
     post = ['Netrin-Unc5-Slit-Robo',
            'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln', 'neuronal_IgCAMs', 'Contactins']


#post = [ 'Nrxn+Nlgn+Nphx+Dystroglycan+Cbln']

#post=[]


postgs = []
for i in post:
    in_i = (df_Paul_gs.loc[:,'Gene Set']==i)
    n_in_i = df_Paul_gs.loc[in_i,'Number of genes'].astype(int).values[0]
    posti = df_Paul_gs.loc[in_i,df_Paul_gs.columns[4:4+n_in_i]]
    postgs = postgs + posti.values.tolist()[0]
    
postgs = postgs +  Elfn    



In [ ]:
## SET FLAG : do_select_best_genes_from_previous_RF_run

In [ ]:
### fi2 = results of previous search
#fi2.to_excel('best_genes_1691_RF.xlsx')
#fi2.to_excel('best_genes_280_RF_2.xlsx')
#fi2.to_excel('best_genes_280_RF_hip.xlsx')

In [ ]:
do_select_best_genes_from_previous_RF_run=0
if do_select_best_genes_from_previous_RF_run==1:
    # fi2 = results of previous search
    fi2  = pd.read_excel('best_genes_1691_RF.xlsx')
    fi2  = pd.read_excel('best_genes_280_RF.xlsx')
    fi2  = pd.read_excel('best_genes_280_RF.xlsx')
    fi2  = pd.read_excel('best_genes_280_RF_hip.xlsx')

    fi2 = pd.read_excel('best_genes_hipp.xlsx')
    fi3 = pd.read_excel('best_genes_visp.xlsx')
    #
    nbest = 10
    bestn = fi2.iloc[0:nbest,:]
    #best_pre = bestn.index[bestn.loc[:,'compartment']=='pre'].values.tolist()
    #best_post = bestn.index[bestn.loc[:,'compartment']=='post'].values.tolist()
    
    best_pre = bestn.loc[bestn.loc[:,'compartment']=='pre','Unnamed: 0'].values.tolist()
    best_post = bestn.loc[bestn.loc[:,'compartment']=='post','Unnamed: 0'].values.tolist()
    
    print('best presynaptic genes')
    print(best_pre)
    print('best postsynaptic genes')
    print(best_post)
    
    ## best gs
    ## best gs
    pregs = best_pre
    postgs = best_post

print('postsynaptic')
print(postgs)
print(len(postgs))

print('presynaptic')
print(pregs)
print(len(pregs))
print(len(postgs))

pregs =np.unique(pregs).tolist()
postgs =np.unique(postgs).tolist()

import pickle
RF_file = open('RF_290_hipp_gs.pickle', mode='wb')
pickle.dump([pregs, postgs],RF_file)

do_visp_features=0
if do_visp_features==1:
    #pickle_in = open('RF_290_hipp.pickle',"rb")
    RF_file = open('RF_290_visp_gs.pickle', mode='rb')
    
    gs_visp = pickle.load(RF_file)
    pregs =gs_visp[0] 
    postgs=gs_visp[1]
    
    
##from mutual_info import entropy
##e = entropy(X, k=1)
#np.finfo(float).eps
#print(len(pregs))
#print(np.unique(pregs).shape)
#print(len(postgs))
#print(np.unique(postgs).shape)
#len(pregs)+len(postgs)    

In [ ]:
dfgs=pd.DataFrame(pregs+postgs)
if (gs=='1512')|(gs=='5188')|(gs=='15656'):
    dfgs.to_excel('gene_set_all_1512.xlsx')
if gs=='219':
    dfgs.to_excel('gene_set_all_219.xlsx')    

## load scVI latent factors data (cortex+hippocampus)

In [ ]:
# load ABA smart-seq?
import pandas as pd
import numpy as np

do_laptop=1

#d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'
if do_laptop:
    d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
else:
    d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'
    #d4 = '/content/drive/My Drive/Colab Notebooks/scVI/'
    

fn='mouse_VISp_2018-06-14_samples-columns.csv'
df_aba_vis_c=pd.read_csv(d4+fn)
print(df_aba_vis_c.columns)
print(df_aba_vis_c.head())

fn = 'mouse_VISp_2018-06-14_genes-rows.csv'
df_aba_g=pd.read_csv(d4+fn)
print(df_aba_g.head())


In [ ]:
in_glut_l23 = (df_aba_vis_c.loc[:,'class']=='Glutamatergic')&(df_aba_vis_c.loc[:,'brain_subregion'].isin(['L2/3','L4','L5']))
sum(in_glut_l23) 

In [ ]:
print(len(df_aba_vis_c.index))

in_glut_l23 = (df_aba_vis_c.loc[:,'class']=='Glutamatergic') #&(df_aba_vis_c.loc[:,'brain_subregion'].isin(['L2/3','L4','L5']))

in_gaba = df_aba_vis_c.loc[:,'class']=='GABAergic'
print(sum(in_gaba))
df_aba_vis_in_c=df_aba_vis_c.loc[in_gaba,:]
df_aba_vis_l23glu_c=df_aba_vis_c.loc[in_glut_l23,:]

In [ ]:
df_aba_vis_in_c.columns

In [ ]:
df_aba_vis_l23glu_c.columns==df_aba_vis_in_c.columns

In [ ]:
#import os

#if do_laptop:
#    os.chdir('/Users/stepaniu/scVI/tests/notebooks')

#print(os.getcwd())
##os.listdir()

save_path = '' #'data/' #'' #'data/'

#d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'
if do_laptop:
    d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
else:
    d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'

fn='mouse_VISp_2018-06-14_samples-columns.csv'
df_aba_vis_c=pd.read_csv(d4+fn)
print(df_aba_vis_c.columns)
print(df_aba_vis_c.head())

fn = 'mouse_VISp_2018-06-14_genes-rows.csv'
df_aba_g=pd.read_csv(d4+fn)
print(df_aba_g.head())
print(len(df_aba_vis_c.index))

in_glut_l23 = (df_aba_vis_c.loc[:,'class']=='Glutamatergic') #&(df_aba_vis_c.loc[:,'brain_subregion'].isin(['L2/3','L4','L5']))

in_gaba = df_aba_vis_c.loc[:,'class']=='GABAergic'
print(sum(in_gaba))
df_aba_vis_in_c=df_aba_vis_c.loc[in_gaba,:]
df_aba_vis_l23glu_c=df_aba_vis_c.loc[in_glut_l23,:]

###
print('loading gene expression...')

d14 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/'
if do_laptop:
    d14 = '/Users/stepaniu/Documents/jan_2020/'
    from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset
else:
    from scvi.dataset import DownloadableAnnDataset

if do_laptop:    
##df_aba_ad.write('mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad')
    aba_vis_in = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad", new_n_genes = 45768,
                                   save_path = save_path) 
else:
    aba_vis_in = DownloadableAnnDataset(d14+"data/mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad") 

if do_laptop:
    #aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L23_Glutamatergic_hda.h5ad", new_n_genes = 45768,
    #                               save_path = save_path) 
    #aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L2345_Glutamatergic_hda.h5ad", new_n_genes = 45768,
    #                               save_path = save_path) 

    aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L23456_Glutamatergic_hda.h5ad", new_n_genes = 45768,
                                   save_path = '') 
else:
    aba_vis_l23glu = DownloadableAnnDataset(d14+"mouse_VISp_2018-06-14_exon-matrix_L23456_Glutamatergic_hda.h5ad") 

diff_order = sum(aba_vis_in.gene_names[0:].astype(int) - np.arange(len(aba_vis_in.gene_names)))
print(diff_order)

aba_vis_in.gene_names = df_aba_g.loc[:,'gene_symbol'].values

aba_vis_l23glu.gene_names = df_aba_g.loc[:,'gene_symbol'].values



print(len(aba_vis_in.gene_names))
print(len(aba_vis_l23glu.gene_names))
len(list(set(aba_vis_l23glu.gene_names).difference(set(aba_vis_in.gene_names))))
print(len(df_aba_vis_l23glu_c.index))
print(len(df_aba_vis_l23glu_c.index))
print(len(df_aba_vis_in_c.index))

#from scvi.dataset.dataset import GeneExpressionDataset
#if do_laptop:
#    vis_dat = GeneExpressionDataset.concat_datasets(aba_vis_in,aba_vis_l23glu)    
#else:    
#    vis_dat  = GeneExpressionDataset()
#    vis_dat.populate_from_datasets([aba_vis_in,aba_vis_l23glu]) 
   

#print((vis_dat.batch_indices==0).sum())
#(vis_dat.batch_indices==1).sum()

In [ ]:
import os

#if do_laptop:
#    os.chdir('/Users/stepaniu/scVI/tests/notebooks')

print(os.getcwd())
#os.listdir()

In [ ]:
#aba_vis_in.gene_names = df_aba_g.loc[:,'gene_symbol'].values
#aba_vis_l23glu.gene_names = df_aba_g.loc[:,'gene_symbol'].values
aba_vis_in._X.shape

In [ ]:
import os
print(os.getcwd())
#os.listdir()

if do_laptop:
    from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset
else:
    from scvi.dataset import DownloadableAnnDataset

d_aba  ='/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'
if do_laptop:
    d_aba  ='/Users/stepaniu/Documents/jan_2020/ABA_2019/'

df_aba_g = pd.read_csv(d_aba + 'medians.csv')
df_aba_c = pd.read_csv(d_aba + 'sample_annotations.csv')


d7 = '/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'
if do_laptop:
    d7  ='/Users/stepaniu/Documents/jan_2020/ABA_2019/'

if do_laptop:    
    aba_hipp_ent = AnnDataset("ABA_2019_transcriptome.h5ad", new_n_genes = 45768,
                             save_path = d7) 
else:     
    aba_hipp_ent = DownloadableAnnDataset(d7 +"ABA_2019_transcriptome.h5ad") 

obs = pd.read_excel(d7 +'ABA_2019_obs.xlsx') # samples
var = pd.read_excel(d7 + 'ABA_2019_var.xlsx') # gene names
#obs = aba_vis_in._obs # samples
#var = aba_vis_in._var # gene names

#df_aba_g = pd.read_csv(d7 + 'medians.csv')
df_aba_ch = pd.read_csv(d7 + 'sample_annotations.csv')

#in_gaba = df_aba_ch.loc[df_aba_ch.loc[:,'sample_name'].isin(obs.iloc[:,0]),['sample_name', 'class_label']]
in_gaba = df_aba_ch.loc[df_aba_ch.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
in_gaba = in_gaba.reset_index().set_index('sample_name',drop=True)
in_gaba = in_gaba.loc[obs.loc[:,'samples'],:]
in_gaba = in_gaba.reset_index().reset_index().set_index('sample_name',drop=True)
in_neurons = in_gaba.loc[in_gaba.loc[:,'class_label'].isin(['GABAergic','Glutamatergic']),:].reset_index().set_index('level_0',drop=True)

#obs = in_gaba.loc[:,'sample_name'].reset_index()
obs = in_gaba.reset_index().loc[:,'sample_name']
#var=var


#in_gaba = df_aba_c.loc[df_aba_c.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
#in_gaba = in_gaba.reset_index().set_index('sample_name',drop=True)
#in_gaba = in_gaba.loc[obs.loc[:,'samples'],:]
#in_gaba = in_gaba.reset_index().reset_index().set_index('sample_name',drop=True)
#in_gaba = in_gaba.loc[in_gaba.loc[:,'class_label']=='GABAergic',:].reset_index().set_index('level_0',drop=True)

#aba_vis_in.gene_names = var.iloc[:,0].values
aba_hipp_ent.gene_names = var.loc[:,'genes'].values
#vis_dat = aba_vis_in
#aba_vis_in = []

#in_ds = df_aba_c.loc[df_aba_c.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
#in_ds = in_ds.reset_index().set_index('sample_name',drop=True)
#in_ds = in_ds.loc[obs.loc[:,'samples'],:]
#in_ds = in_ds.reset_index().reset_index().set_index('sample_name',drop=True)
##in_ds

#df_aba_vis_in_c = df_aba_c.loc[in_ds.loc[:,'index'],:]

In [ ]:
var

In [ ]:
same_g=df_aba_g.loc[:,'Unnamed: 0']==var.loc[:,'genes']
same_g.sum()

In [ ]:
import matplotlib
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#import seaborn as sns

import numpy as np
import numpy.random as random
import pandas as pd

#do_laptop = False
if do_laptop:
    import umap
    umap.__version__='0.3.7'
    import scanpy as sc
    import louvain
    from umap.umap_ import UMAP
#else:
    #import umap
    #umap.__version__='0.3.7'
    #import scanpy as sc
    #import louvain
    #from umap.umap_ import UMAP
    
    
use_cuda = True
from scvi.dataset.dataset import GeneExpressionDataset
from scvi.inference import UnsupervisedTrainer
from scvi.models import SCANVI, VAE


save_path2=''

In [ ]:
## select genes


In [ ]:
stype = ['pre']*len(pregs) + ['post']*len(postgs)
stype = pd.DataFrame(stype,columns  = ['type'])
dfgs = pd.concat([dfgs, stype],axis=1)
dfgs

# STOP!!!

In [ ]:
import scanpy as sc

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, color_map='Greys')
sc.logging.print_versions()
results_file = 'X_scvi.h5ad'



In [ ]:
import scanpy.api as sca
sca.pp.filter(adt1,min_cells=3)
#sc.pp.filter(adt1,min_genes=10)

#all_data  =[aba_vis_in, aba_vis_l23glu, aba_hipp_ent]




In [ ]:
if gs=='1512':
    dfgs.to_excel('gene_set_all_1512.xlsx')
if gs=='219':
    dfgs.to_excel('gene_set_all_219.xlsx')    

# START!!!

In [ ]:
print(len(pregs+postgs))
print(len(list(set(pregs+postgs))))
gene_set = list(set(pregs+postgs))
# if (gs=='5188')|(gs=='15656'):
#     igenes = pd.Series(aba_vis_in.gene_names).isin(gene_set).values
#     gene_names2 = aba_vis_in.gene_names[igenes==False]
    
#     #gene_set = gene_set + gene_names2[np.arange(0,5188-igenes.sum())].tolist()
#     gene_set = aba_vis_in.gene_names[igenes].tolist() + gene_names2[np.arange(0,5188-igenes.sum())].tolist()
len(gene_set)    

In [ ]:
do_filter_gene_set = False
if do_filter_gene_set:
    igenes = pd.Series(aba_vis_in.gene_names).isin(gene_set).values

    aba_vis_in.gene_names = aba_vis_in.gene_names[igenes]
    aba_vis_in.X = aba_vis_in.X[:,igenes]

    #aba_vis_in.__dict__

In [ ]:
if do_filter_gene_set:
    igenes = pd.Series(aba_vis_l23glu.gene_names).isin(gene_set).values

    aba_vis_l23glu.gene_names = aba_vis_l23glu.gene_names[igenes]
    aba_vis_l23glu.X = aba_vis_l23glu.X[:,igenes]

In [ ]:
if do_filter_gene_set:
    aba_hipp_ent.gene_names  = var.genes.values
    igenes = pd.Series(aba_hipp_ent.gene_names).isin(gene_set).values

    aba_hipp_ent.gene_names = aba_hipp_ent.gene_names[igenes]
    aba_hipp_ent.X = aba_hipp_ent.X[:,igenes]

In [ ]:
from scvi.dataset.dataset import GeneExpressionDataset
if do_laptop==1:
    all_dataset = GeneExpressionDataset.concat_datasets(aba_vis_in,aba_vis_l23glu, aba_hipp_ent)    
else:    
    all_dataset = GeneExpressionDataset()
    all_dataset.populate_from_datasets([aba_vis_in, aba_vis_l23glu, aba_hipp_ent])    
   

print((all_dataset.batch_indices==0).sum())
print((all_dataset.batch_indices==2).sum())
(all_dataset.batch_indices==1).sum()

In [ ]:
import os
os.getcwd()

In [ ]:
import torch

if gs=='219':
    n_epochs_all = 100
    n_epochs=100 if n_epochs_all is None else n_epochs_all
    lr=2e-3
    use_batches=True
    use_cuda=True
    
if (gs=='1512')|(gs=='5188'):
    n_epochs_all = 2000
    n_epochs=2000 if n_epochs_all is None else n_epochs_all
    lr=2e-3
    use_batches=True
    use_cuda=True   
    
if  (gs=='15656'):
    n_epochs_all = 2000
    n_epochs=2000 if n_epochs_all is None else n_epochs_all
    lr=2e-3
    use_batches=True
    use_cuda=True     

In [ ]:
if gs=='219':
    vae = VAE(all_dataset.nb_genes, n_batch=all_dataset.n_batches * use_batches, dispersion='gene-batch', 
        n_latent = 10,
        n_layers=3, n_hidden=256)

     #vae = VAE(all_dataset.nb_genes, n_batch=all_dataset.n_batches * use_batches, dispersion='gene-batch', n_latent = 10,
     #         n_layers=2, n_hidden=256)

    trainer = UnsupervisedTrainer(vae,
                              all_dataset,
                              train_size=0.95,
                              use_cuda=use_cuda,
                              frequency=1)

    dn=100
    file_results = 'vae_219_'
    i_load0=0
    i_load = 10
    
if (gs=='1512'):
    vae = VAE(all_dataset.nb_genes, n_batch=all_dataset.n_batches * use_batches, dispersion='gene-batch', 
        n_latent = 20,
        n_layers=3, n_hidden=256)

     #vae = VAE(all_dataset.nb_genes, n_batch=all_dataset.n_batches * use_batches, dispersion='gene-batch', n_latent = 10,
     #         n_layers=2, n_hidden=256)

    trainer = UnsupervisedTrainer(vae,
                              all_dataset,
                              train_size=0.95,
                              use_cuda=use_cuda,
                              frequency=1)

    dn=200
    if gs=='1512':
        dir_results = '/Users/stepaniu/Documents/jan_2020/scvi_vae_gs_1690/drive-download-20201214T201804Z-001'

        file_results = dir_results +'/'+'vae_1690_'
        i_load0=1
        i_load = 10
if gs=='5188':
    dn=200
    dir_results = '/Users/stepaniu/Documents/jan_2020/scvi_vae_gs_5188/gs_5188_2_drive-download-20201214T163529Z-001'
   
    file_results = dir_results +'/'+'vae_5188_2_'
    i_load0=1
    i_load = 7  
        
    import pickle
    trainer_history = pickle.load(open(file_results+'trainer_history.p','rb'))
    trainer_data = pickle.load(open(file_results+'trainer_data.p','rb'))
    
    #trainer_data.gene_names = df_aba_g.loc[trainer_data.gene_names.astype(int),'gene_symbol'].values
    trainer_data.gene_names = df_aba_g.loc[trainer_data.gene_names.astype(int),'Unnamed: 0'].values
    trainer_data.dense = all_dataset.dense
    trainer_data.x_coord = all_dataset.x_coord
    trainer_data.y_coord = all_dataset.y_coord
    trainer_data.batch_indices = all_dataset.batch_indices
    trainer_data.labels = all_dataset.labels
    #all_dataset = trainer_data
    
    
    #igenes = trainer_data.gene_names.astype(int)
    #gene_nms = df_aba_g.loc[igenes,'Unnamed: 0'].values
    #all_dataset.gene_names = all_dataset.gene_names[igenes]
    #all_dataset.X = all_dataset.X[:,igenes]
    
    
    vae = VAE(trainer_data._X.shape[1], n_batch=trainer_data.n_batches * use_batches, dispersion='gene-batch', 
        n_latent = 20,
        n_layers=3, n_hidden=256)
    #vae = VAE(all_dataset.nb_genes, n_batch=all_dataset.n_batches * use_batches, dispersion='gene-batch', 
    #    n_latent = 20,
    #    n_layers=3, n_hidden=256)
    trainer = UnsupervisedTrainer(vae,
                              trainer_data,
                              train_size=0.95,
                              use_cuda=use_cuda,
                              frequency=1)
#     trainer = UnsupervisedTrainer(vae,
#                               all_dataset,
#                               train_size=0.95,
#                               use_cuda=use_cuda,
#                               frequency=1)

if (gs=='15656'):  
    dn=200
    dir_results = '/Users/stepaniu/Documents/jan_2020/scvi_vae_gs_15656'
    file_results = dir_results +'/'+'vae_15656_' #+'dataset_'
    i_load0=1
    i_load = 10
    
    #trainer = torch.load(file_results+'9'+'.pkl',map_location='cpu')
    import pickle
    trainer_history = pickle.load(open(file_results+'trainer_history.p','rb'))
    trainer_data = pickle.load(open(file_results+'trainer_data.p','rb'))
    
    #trainer_data.gene_names = df_aba_g.loc[trainer_data.gene_names.astype(int),'gene_symbol'].values
    trainer_data.gene_names = df_aba_g.loc[trainer_data.gene_names.astype(int),'Unnamed: 0'].values
        
    trainer_data.dense = all_dataset.dense 
    trainer_data.x_coord = all_dataset.x_coord
    trainer_data.y_coord = all_dataset.y_coord
    trainer_data.batch_indices = all_dataset.batch_indices
    trainer_data.labels = all_dataset.labels
    #all_dataset = trainer_data
    
    vae = VAE(trainer_data._X.shape[1], n_batch=trainer_data.n_batches * use_batches, dispersion='gene-batch', 
        n_latent = 20,
        n_layers=3, n_hidden=256)
    trainer = UnsupervisedTrainer(vae,
                              trainer_data,
                              train_size=0.95,
                              use_cuda=use_cuda,
                              frequency=1)
    
        
n2 = int(n_epochs/dn)

load_trained_model=False
train_model=False
if load_trained_model: #os.path.isfile('%s/vae.pkl' % save_path ):
    #i=n2
    i=0
    #trainer.model.load_state_dict(torch.load('%s/vae'+str(i)+'.pkl' % save_path2)) #vae3.pkl
    
    #trainer.model.load_state_dict(torch.load('vae'+str(i)+'.pkl',map_location='cpu')) # last 18_11_2019 alignment on colab gpu
    ##trainer.model.load_state_dict(torch.load('%s/vae2.pkl' % cwd))
    trainer.model.load_state_dict(torch.load(file_results+str(i)+'.pkl')) # 219 selected genes
    
    trainer.model.eval()

if train_model :
    for i in range(0,n2):
        trainer.train(n_epochs=dn, lr=lr)
        #torch.save(trainer.model.state_dict(), 'save_path2'+'vae'+str(i)+'.pkl')
        torch.save(trainer.model.state_dict(), file_results+str(i)+'.pkl')

In [ ]:
trainer_data._X.shape

In [ ]:
all_dataset._X.shape

In [ ]:
len(trainer_history['elbo_train_set'])

In [ ]:
trainer.history["elbo_train_set"]

In [ ]:
import os
print(os.getcwd())
fis =os.listdir(dir_results)
fis = pd.DataFrame(fis)
fis

In [ ]:
(fis==(file_results+str(1)+'.pkl').split('/')[-1]).any()[0]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

elbo_train_set = []
elbo_test_set = []
for i in range(i_load0,i_load):
    is_pkl =(fis==(file_results+str(i)+'.pkl').split('/')[-1]).any()[0]
    if is_pkl:
        trainer.model.load_state_dict(torch.load(file_results+str(i)+'.pkl', map_location='cpu')) # 219 selected genes
    
        trainer.model.eval()
        elbo_train_set = elbo_train_set + trainer.history["elbo_train_set"]
        elbo_test_set = elbo_test_set + trainer.history["elbo_test_set"] 
        print(i,' : is data')
        
    print(i)

    


if (gs=='5188')|(gs=='15656'):
    
    #x = np.linspace(0, 100, (len(trainer_history['elbo_train_set'])))
    plt.plot(trainer_history['elbo_train_set'])
    plt.plot(trainer_history['elbo_test_set'])
    plt.ylim(20000, 30000)
else:
    x = np.linspace(0, 100, (len(elbo_train_set)))
    plt.plot(x, elbo_train_set)
    plt.plot(x, elbo_test_set)
#plt.ylim(500, 700)


In [ ]:
#full = trainer.create_posterior(trainer.model, all_dataset, indices=np.arange(len(all_dataset)))
full = trainer.create_posterior(trainer.model, trainer_data, indices=np.arange(len(trainer_data)))
latent, batch_indices, labels = full.sequential().get_latent()
batch_indices = batch_indices.ravel()

In [ ]:
latent.shape

In [ ]:
full.gene_dataset.gene_names

In [ ]:
# imputation gene expression
px_rates = full.imputation()


In [ ]:
full.gene_dataset.__dict__

In [ ]:
np.sum(np.sum(px_rates, axis=0)==0)

In [ ]:
import magic
import pandas as pd
import matplotlib.pyplot as plt
#magic_operator = magic.MAGIC()
#X = pd.read_csv("test_data.csv")
#X_magic = magic_operator.fit_transform(X, genes=['VIM', 'CDH1', 'ZEB1'])
#plt.scatter(X_magic['VIM'], X_magic['CDH1'], c=X_magic['ZEB1'], s=1, cmap='inferno')
#plt.show()
#magic.plot.animate_magic(X, gene_x='VIM', gene_y='CDH1', gene_color='ZEB1', operator=magic_operator)


In [ ]:
import matplotlib.pyplot as plt

g1 = 'Elfn1'
g2 = 'Cacna2d1'
g3  = 'Cplx1'

genes = full.gene_dataset.gene_names
ii=np.nonzero(genes==g1)[0]
ii2=np.nonzero(genes==g2)[0]
print(genes[ii])

X_scvi = pd.DataFrame(px_rates, columns = genes)

print(ii2)
Xii2=full.gene_dataset._X[:,ii2]

# impute with magic
magic_operator = magic.MAGIC()
#X = pd.read_csv("test_data.csv")
X = pd.DataFrame(full.gene_dataset._X, columns = genes)
X_magic = magic_operator.fit_transform(X, genes=[g1, g2, g3])

In [ ]:
X_magic

In [ ]:
(ax, fig) = plt.subplots(figsize=(15, 5))

plt.gca().legend('')

Xii=full.gene_dataset._X[:,ii]
nc = Xii.shape[0]
#plt.scatter(np.arange(nc),Xii.ravel(),marker='o',c=[1,0,0],s=10)
#plt.scatter(np.arange(nc),px_rates[:, ii].ravel(),marker='x',c=[0,1,0],s=20,alpha=0.3)
plt.title('scVI imputation vs data : '+g1)
plt.scatter(px_rates[:, ii].ravel(), Xii.ravel(), marker='o', c=[1,0,0], s=10)

(ax1, fig1) = plt.subplots(figsize=(15, 5))
plt.title('data   : '+g1)
plt.scatter(np.arange(nc),Xii.ravel(),marker='o',c=[1,0,0],s=10)

(ax1p5, fig1p5) = plt.subplots(figsize=(15, 5))
plt.title('scVI imputation  : '+g1)
plt.scatter(np.arange(nc),px_rates[:, ii].ravel(),marker='x',c=[0,1,0],s=20,alpha=0.3)


(ax4, fig4) = plt.subplots(figsize=(15, 5))
plt.title('data   : '+g2)
plt.scatter(np.arange(nc),Xii2.ravel(),marker='o',c=[1,0,0],s=10)

(ax5, fig5) = plt.subplots(figsize=(15, 5))
plt.title('scVI imputation  : '+g2)
plt.scatter(np.arange(nc),px_rates[:, ii2].ravel(),marker='x',c=[0,1,0],s=20,alpha=0.3)



h1d = np.histogram(Xii, 100)
h1i = np.histogram(px_rates[:, ii], 100)
h1im = np.histogram(X_magic[g1], 100)

h2d = np.histogram(Xii2, 100)
h2i = np.histogram(px_rates[:, ii2], 100)
h2im = np.histogram(X_magic[g2], 100)

(ax2, fig2) = plt.subplots(figsize=(15, 5))
plt.title('histogram : initial data vs scVI and MAGIC imputation  : '+g1)
plt.scatter(h1d[1][:-1],np.log(h1d[0]),marker='o',c=[1,0,0],s=10)
plt.scatter(h1i[1][:-1],np.log(h1i[0]),marker='x',c=[0,1,0],s=10)
plt.scatter(h1im[1][:-1],np.log(h1im[0]),marker='d',c=[0,0,1],s=10)

(ax2, fig2) = plt.subplots(figsize=(15, 5))
plt.title('histogram : initial data vs scVI and MAGIC imputation  : '+g2)
plt.scatter(h2d[1][:-1],np.log(h2d[0]),marker='o',c=[1,0,0],s=10)
plt.scatter(h2i[1][:-1],np.log(h2i[0]),marker='x',c=[0,1,0],s=10)
plt.scatter(h2im[1][:-1],np.log(h2im[0]),marker='d',c=[0,0,1],s=10)

(ax6, fig6) = plt.subplots(figsize=(15, 5))
plt.title('magic imputation')
plt.scatter(X_magic[g1], X_magic[g2], c=X_magic[g3], s=1, cmap='inferno')
plt.show()
magic.plot.animate_magic(X, gene_x=g1, gene_y=g2, gene_color=g3, operator=magic_operator)

In [ ]:
full.__dict__

In [ ]:
df_aba_vis_in_c
df_aba_vis_l23glu_c

In [ ]:
df_aba_vis_in_c.columns

In [ ]:
df_aba_ch

In [ ]:
#aba_vis_in, aba_vis_l23glu, aba_hipp_ent
aba_hipp_ent.obs
aba_vis_in.obs
all_samples = [aba_vis_in.obs.index.values,
               aba_vis_l23glu.obs.loc[:,'sample_name'].values,
               aba_hipp_ent.obs.loc[:,'samples'].values]
all_samples = np.concatenate(all_samples)
all_samples = pd.DataFrame(all_samples,columns=['sample_name'])
print(all_samples.shape)
all_samples

In [ ]:
scVI_latent = pd.DataFrame(latent)
scVI_latent = pd.concat([all_samples, scVI_latent, pd.DataFrame(batch_indices,columns=['batch_indices'])],axis=1)
scVI_latent

In [ ]:
# imputed scVI genes 
X_scvi = pd.DataFrame(px_rates, columns = genes)
do_add_magic=1
if do_add_magic==1:
    X_scvi.loc[:,X_magic.columns] = X_magic

In [ ]:
scVI_all = pd.concat([scVI_latent, X_scvi], axis=1)
scVI_all

In [ ]:
#scVI_all
X_real = pd.DataFrame(full.gene_dataset._X, columns = full.gene_dataset.gene_names)
X_real = pd.concat([all_samples, X_real],axis=1)
X_real

In [ ]:
g1 = 'Elfn1'
g2 = 'Bsn'
g3  = 'Syt1'



(ax, fig) = plt.subplots(figsize=(15, 5))

plt.gca().legend('')

Xii=X_real.loc[:] #full.gene_dataset._X[:,ii]
nc = Xii.shape[0]
#plt.scatter(np.arange(nc),Xii.ravel(),marker='o',c=[1,0,0],s=10)
#plt.scatter(np.arange(nc),px_rates[:, ii].ravel(),marker='x',c=[0,1,0],s=20,alpha=0.3)
plt.title('scVI imputation vs data : '+g1)
plt.scatter(px_rates[:, ii].ravel(), Xii.ravel(), marker='o', c=[1,0,0], s=10)

(ax1, fig1) = plt.subplots(figsize=(15, 5))
plt.title('data   : '+g1)
plt.scatter(np.arange(nc),Xii.ravel(),marker='o',c=[1,0,0],s=10)

(ax1p5, fig1p5) = plt.subplots(figsize=(15, 5))
plt.title('scVI imputation  : '+g1)
plt.scatter(np.arange(nc),px_rates[:, ii].ravel(),marker='x',c=[0,1,0],s=20,alpha=0.3)


(ax4, fig4) = plt.subplots(figsize=(15, 5))
plt.title('data   : '+g2)
plt.scatter(np.arange(nc),Xii2.ravel(),marker='o',c=[1,0,0],s=10)

(ax5, fig5) = plt.subplots(figsize=(15, 5))
plt.title('scVI imputation  : '+g2)
plt.scatter(np.arange(nc),px_rates[:, ii2].ravel(),marker='x',c=[0,1,0],s=20,alpha=0.3)


In [ ]:
scVI_all.head()

In [ ]:
#gs_name = '5188' #'1512' #'219'
gs_name = gs
scVI_all.to_hdf('scVI_latent_factors_and_imputation_aba2019_gs'+gs_name+'.hdf',key='data') # this is 290 genes dataset

# STP modeling functions: 

In [ ]:
# PV->AAC ???
Dt=50
tF=10
tD=410
p0=0.23
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0


In [ ]:
# CCK->CCK ???
Dt=50
tF=1542
tD=115
p0=0.11
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
# PV->AAC ???
Dt=50
tF=1.6
tD=930
p0=0.26
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
# Ivy->PC ???
Dt=50
tF=62
tD=144
p0=0.32
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
# CCK_DTI->PC ???
Dt=50
tF=14
tD=185
p0=0.15
p=p0
n = (1-p)
n=1-(1-n)*np.exp(-Dt/tD)
print(n)
p=p+(1-p)*p0
p=p0 +(p -p0)*np.exp((-Dt/tF))
print(p)
n*p/p0

In [ ]:
def  STP_sim(ge_data, T, init_state=None ):
    # transform labels from TM to An:A1
    #f = 20 # Hz
    #N = 3
    #T = np.arange(N)*1000/f

    N    = len(T)
    nc   = len(ge_data.index)

    x_lower = np.array([1,       0.01,    1,      0.1,     1])
    x_upper = np.array([10000,   1,      10000,    10,     1])
    stp_ns =          ['tF',    'p0',    'tD',   'dp/p0', 'A']
    for jj in range(len(stp_ns)):
        nsj = stp_ns[jj]
        ge_data.loc[:,nsj] = np.maximum(ge_data.loc[:,nsj].values,x_lower[jj])
        ge_data.loc[:,nsj] = np.minimum(ge_data.loc[:,nsj].values,x_upper[jj])
        if nsj=='dp/p0':
            p0   = ge_data.loc[:,'p0'].values
            dpp0 = ge_data.loc[:,'dp/p0'].values
            dp = p0*dpp0
            dp = np.minimum(dp,1)
            ge_data.loc[:,nsj] = dp/(p0 +np.finfo(float).eps)

    dpp0 = ge_data.loc[:,'dp/p0'].values
    p0   = ge_data.loc[:,'p0'].values
    tF   = ge_data.loc[:,'tF'].values
    tD   = ge_data.loc[:,'tD'].values
    A    = 1 + 0*ge_data.loc[:,'A'].values # simplify A

    As = np.zeros((nc,N))
    n = np.zeros((nc,))
    p = np.zeros((nc,))
    ns2 = np.zeros((nc,N))
    ps2 = np.zeros((nc,N))

    i=0
    
    if init_state is None :
        n[:] = 1
        p[:] = p0
    else:
        n = init_state[0]
        p = init_state[1]


    As[:,i] = A*n*p
    
    n = n*(1-p)
    p = p + dp*(1-p)
    
    ns2[:,i]=n
    ps2[:,i]=p

    for i in range(1,N):
        Dt=T[i]-T[i-1]
        #n = 1 - (1 - (n -p*n))*np.exp((-Dt/tD).astype(float))
        #p=p0 -(p0-(p + dpp0*p0*(1-p)))*np.exp((-Dt/tF).astype(float))
        #As[:,i]=A*n*p
        #ns2[:,i]=n
        #ps2[:,i]=p

        
        n = 1 - (1 - n )*np.exp((-Dt/tD ).astype(float))
        p=p0 +(p -p0)*np.exp((-Dt/tF).astype(float))
            

        As[:,i]=A*n*p
       
        n = n*(1-p)
        p = p + dp*(1-p)
        
        ns2[:,i]=n
        ps2[:,i]=p

    

    #aa = [As, ns2, ps2]
    
    return As, ns2, ps2, dpp0, p0, tF, tD, A

In [ ]:
def  STP_sim2(x, T, init_state=None, model_type='tm5' ):

    N    = len(T)
    tF      = x[0] #.astype(float)
    p00     = x[1]
    tD      = x[2] #.astype(float)
    dp      = x[3]
    A       = 1 #x[4] # simplify A
    
    #breakpoint()
    mod_fdr2=False
    if model_type=='tm5_fdr2':  # should be :check freq. dependent recovery
        tDmin     = x[5]
        dd        = x[6]
        t_FDR     = x[7]
        mod_fdr2=True
        tDmax  = tD
        itDmin = 1/tDmin
        itDmax = 1/tDmax
        #breakpoint()
        
    mod_smr=False
    if model_type=='tm5_smr':  # should be :check freq. dependent recovery
        t_SMR   = x[8]
        dp0     = x[9]
        mod_smr=True
        #p00  = p00

    As = np.zeros((N,))
    state = np.zeros((N,4))

   
    if init_state is None :
        n = 1
        p0=p00
        p = p0
        d = 0
    else:
        n = init_state[0]
        p = init_state[1]
        d = init_state[2]
        p0= init_state[3]

    
    for i in range(0,N):
        if i==0:
            Dt = T[i]
        else:
            Dt = T[i]-T[i-1]
        
        if mod_fdr2:
            d0=d
            d = d*np.exp(-Dt/t_FDR) 
            n = 1 - (1 - n )*np.exp(-Dt*itDmax -(itDmin -itDmax)*t_FDR*(d0-d))
        else:
            n = 1 - (1 - n )*np.exp(-Dt/tD )
            
        if mod_smr:
            p01=p0
            p0=p00 + (p0 -p00)*np.exp(-Dt/t_SMR)
            p=p0 +(p -p01)*np.exp(-Dt/tF)
        else:
            p=p0 +(p -p0)*np.exp(-Dt/tF)
            

        As[i]=A*n*p
       
        n = n*(1-p)
        p = p + dp*(1-p)
        if mod_fdr2:
            d  = d + dd*(1-d) 
        if mod_smr:
            p0  = p0 - dp0*p0    
 
        state[i] = [n,p,d,p0]

    #return As, ns2, ps2, dpp0, p0, tF, tD, A
    return As, state

In [ ]:
def STP_sim_complex(ge_data,l_pre_post2,stp_aba_names):
    # transform labels from TM to An:A1
    #fs = [20, 50, 10] # Hz
    fs = [20, 50, 10]
    N = 5

    #Trec = [250, 500, 1000]
    Trec =[250, 1000]
    DT0 = 25000
    if l_pre_post2<ge_data.shape[0]:
        xs  =ge_data.iloc[l_pre_post2:,:].loc[:,stp_aba_names].values
        xs  =np.delete(xs, [5,6],axis=1)
        As2=np.zeros((xs.shape[0],0))
    
    
    As=np.zeros((l_pre_post2,0))
    if 1:

        for f in fs:
            T = np.arange(N)*1000/f

            if l_pre_post2>0:
                Asf, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],T)
                As = np.concatenate([As,Asf],axis=1)

                for ri in range(len(Trec)):
                    Asr, nsr, psr, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],[Trec[ri]],init_state=[ns[:,-1],ps[:,-1]])
                    As = np.concatenate([As,Asr],axis=1)

            if l_pre_post2<ge_data.shape[0]:
                As2f = np.zeros((xs.shape[0],N+len(Trec)))
                for i2 in range(xs.shape[0]):
                    #as2, sts2 = STP_sim2(xs[i2,:],np.arange(8)*50,model_type = 'tm5') # preconditioning series???
                    #as2, sts2 = STP_sim2(xs[i2,:], T+DT0, init_state=sts2[-1], model_type='tm5' ) 

                    as2, sts2 = STP_sim2(xs[i2,:],T,model_type = 'tm5_smr')

                    As2f[i2,0:N] = as2
                    for ri in range(len(Trec)):
                        as2r, sts2r = STP_sim2(xs[i2,:], [Trec[ri]], init_state=sts2[-1], model_type='tm5_smr' ) 
                        As2f[i2,N+ri] = as2r

                As2 = np.concatenate([As2,As2f],axis=1)

        if l_pre_post2<ge_data.shape[0]:
            As2 = As2/As2[:,0].reshape((-1,1))
        else:
            As2  = np.zeros((0,As.shape[1]))
        
        
        if l_pre_post2>0:
            As = As/As[:,0].reshape((-1,1))
            As = np.concatenate([As,As2],axis=0)
        else:
            As =As2
    else:
        As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)


    #import matplotlib.pyplot as plt    
    #plt.plot(As2[400:410,:].transpose(),'o-')
    
    return As

## 
##
## Hippocampus gene expression
##
## 

In [ ]:
import os
print(os.getcwd())
#os.listdir()

from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset

d_aba  ='/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'

df_aba_g = pd.read_csv(d_aba + 'medians.csv')
df_aba_c = pd.read_csv(d_aba + 'sample_annotations.csv')


d7 = '/home/stepaniu/Documents/references/transcriptomes to STP/transcriptomes/ABA_2019/'

aba_vis_in = AnnDataset("ABA_2019_transcriptome.h5ad", new_n_genes = 45768,
                               save_path = d7) 

obs = pd.read_excel(d7 +'ABA_2019_obs.xlsx') # samples
var = pd.read_excel(d7 + 'ABA_2019_var.xlsx') # gene names
#obs = aba_vis_in._obs # samples
#var = aba_vis_in._var # gene names

in_gaba = df_aba_c.loc[df_aba_c.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
in_gaba = in_gaba.reset_index().set_index('sample_name',drop=True)
in_gaba = in_gaba.loc[obs.loc[:,'samples'],:]
in_gaba = in_gaba.reset_index().reset_index().set_index('sample_name',drop=True)
in_gaba = in_gaba.loc[in_gaba.loc[:,'class_label']=='GABAergic',:].reset_index().set_index('level_0',drop=True)

#aba_vis_in.gene_names = var.iloc[:,0].values
aba_vis_in.gene_names = var.loc[:,'genes'].values
vis_dat = aba_vis_in
aba_vis_in = []

in_ds = df_aba_c.loc[df_aba_c.loc[:,'sample_name'].isin(obs.loc[:,'samples']),['sample_name', 'class_label']]
in_ds = in_ds.reset_index().set_index('sample_name',drop=True)
in_ds = in_ds.loc[obs.loc[:,'samples'],:]
in_ds = in_ds.reset_index().reset_index().set_index('sample_name',drop=True)
#in_ds

df_aba_vis_in_c = df_aba_c.loc[in_ds.loc[:,'index'],:]

##
##
##  HIPPOCAMPUS STP
##
##

In [ ]:
#
#
#  HIPPOCAMPUS STP
#
#
import re
import time
t1 = time.time()


# names of gene expression based synapses types

glu_l23=[] #list(set(df_aba_vis_l23glu_c.loc[:,'cluster']))

dsg = [['PC']+glu_l23,
       ['Pvalb','Pvalb Tpbg','Pvalb Reln Itm2a','Pvalb Reln Tac1','Pvalb Sema3e Kank4'], 
       ['Sst','Sst Calb2 Pdlim5','Sst Calb2 Necab1', 'Sst Hpse Cbln4','Sst Hpse Sema3c','Sst Nr2f2 Necab1'],
       ['Vip','Vip Lmo1 Myl1', 'Vip Rspo1 Itga4','Vip Ptprt Pkp2','Vip Rspo4 Rxfp1 Chat'],
       ['Lamp5','Lamp5 Plch2 Dock5'],
       ['MC','L23MC','Sst Calb2 Pdlim5'],
        'L5MC']
      

#set(df_stp.loc[:,'synapse_type_2'])

# names of electrophysiology based synapses types
dse = [['PC', 'L23P', 'L23PC', 'L5P'],
       ['Pvalb', 'PV', 'L23BC', 'L5BC'], 
       ['Sst', 'L23MC', 'L5MC', 'Sst', 'MC'], 
       ['Vip', 'VIP', 'BTC'],
       ['NGC', 'L23NGC'],
       ['L23MC' ],
       ['L5MC']]


# load stp data table
do_laptop=0
if do_laptop:
    d5 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/additional cortical STP data/'
else:
    d5 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/additional cortical STP data/'
#fn5='STP_cortex_interneurons_Cre_labeled.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF2.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF3.xlsx'

do_3pars_TM=0 # 3 or 4 parametric TM model?
if do_3pars_TM:
    fn5='STP_cortex_interneurons_Cre_labeled_UDF5_3pars.xlsx'
else:
    #fn5='STP_cortex_interneurons_Cre_labeled_UDF5_4pars.xlsx'
    fn5='STP_hippocampus_all_UDF3.xlsx'

df_stp=pd.read_excel(d5+fn5)
#df_stp.loc[df_stp.loc[:,'stp_comment'].isna() ,'stp_comment']=''

############   
do_averagig_stp =0
stp_TM_names=['tF', 'p0','tD','dp/p0','A']
stp_TM_start = np.nonzero(df_stp.columns=='references')[0][0]+1 # 187
ncol = len(df_stp.columns)
df_tm_name = pd.DataFrame(np.zeros(ncol).reshape((1,ncol)), columns =df_stp.columns )
df_stp = pd.concat([df_tm_name, df_stp],axis=0).reset_index(drop=True)

for i,nm in enumerate(stp_TM_names):
    ii = stp_TM_start + np.arange(0,ncol-i-stp_TM_start,len(stp_TM_names))+i
    df_stp.iloc[0,ii] = nm


#print(df_stp.head())
#############
##Remove pure PPR data (Jaing_2015_PC)
#df_stp = df_stp.loc[df_stp.loc[:,'name'].str.contains('Jaing_2015_PC')!=True,:].reset_index(drop=True) 

############


#stp_TM_names_2=['tF', 'p0','tD','dp','A','A1','A2','tDmin','dd','t_FDR','t_SMR','dp0'] # tm5+smr
##################
#stp_2_default = np.ones(7)
#for i in range(1,df_stp.shape[0]):
#    stp_par = df_stp.iloc[i,stp_TM_start:].values.reshape((len(stp_TM_names),-1))
    

##################

df_stp

## Hippocampus: allign gene expression and stp data (for selected gene sets)

In [ ]:
# rules for cell types to aba clusters mapping:
#1 dorsal : 'subregion_label'== 'anterior' , ventral: 'subregion_label'== posterior' ,  or by genes : see Cembrowski 2016
#2 TA = ECIII (PCP4+) ->CA1 
#3 pp = ECII (Reelin+) ->CA3, DG, CA2
#4 TA = ECII (Wlf1+, Calb2+) ->CCK ppa in CA1 mostly
#5 no cre lines annotations - use only old cck datatypes->clusters or corresponding cortical cre lines?
#6 Ivy 1, 2 ?? - 2 clusters (Lhx6 1,2) and may be Lhx6 3 (no reelin), but also could be some Lamp5_5? - less reelin, more Nos1 
#  use only only PPF Ivy cells? or try 2 subtypes = 2 clusters?? - dep=lhx6_4
#7 NGF types 1,2? Price 2005 - correlate with age - select only facilitating (type 2)!
#8 HIPP - Sst, pp-associated ~OLM??
#9 HICAP - CCk - comissural assoc - ~ CCK Schffer comm : Sncg6 (Sncg/Ndnf HPF1)
#10 Kohus: CCK DTI vs BC? - use Cbln4, Lgi2 ratio? (Favuzzi 2019) - Sncg2?, (Sncg6) - DTI vs Sncg/Ndnf HPF2,3,6, Serpinf1, Sncg7, Vip_14
#11 CA1 PC: use subregions: anterior, mid-anterior  for dorsal



# batches and gene sets
pre_post = df_stp.loc[:,'synapse_type_2'].str.split(pat='->',expand=True)

vbi = vis_dat.batch_indices.reshape((vis_dat.batch_indices.shape[0],))
vbi=vbi>0

df_gs  = pd.DataFrame( vis_dat.gene_names ) 
in_pregs = df_gs.loc[:,0].isin(pregs).values
in_postgs = df_gs.loc[:,0].isin(postgs).values

pregs2 = df_gs.loc[in_pregs,0].values
postgs2 = df_gs.loc[in_postgs,0].values

do_hippocampus = 1
if do_hippocampus ==0:

    do_2=1;
    # select stp data for appropriate frequency and ages 
    stp_type = {'name':['Walker_2016_PV->MC L23',
                   'Walker_2016_VIP->MC L23',
                   'Yuste_2016_taus_PV->Sst L23 recalc',
                   'Yuste_2016_taus_PV->VIP L23 recalc',
                   'Yuste_2016_taus_Sst->VIP L23 recalc',
                   'Yuste_2016_taus_VIP->Sst L23 recalc',
                   'Yuste_2016_taus_VIP->VIP L23 recalc'],
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }

    if do_2==1:
        stp_type = {'name':['Walker_2016_PV->MC L23',
                    'Walker_2016_VIP->MC L23',
                    'Yuste_2016_taus_PV->Sst L23 recalc',
                    'Yuste_2016_taus_VIP->Sst L23 recalc'],
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }




    in_stp_data = df_stp.loc[:,'stp_freq'].isin([20])
    in_stp_data= in_stp_data|(df_stp.loc[:,'stp_freq'].isin([40,50])&df_stp.loc[:,'name'].isin(stp_type['name']))
    in_stp_data = in_stp_data&df_stp.loc[:,'stp_comment'].isin(pd.Series(stp_type['stp_comment']).str.strip())

else:
    
    in_stp_data = pd.Series(np.ones(df_stp.shape[0])==1)
    # filter stp dataset 
    in_stp_data.loc[df_stp.loc[:,'name'].isin(['Price_2005EC->NGF type1',
                                               'Qin_2017EC->Ivy cell type1',
                                               'Qin_2017CA3->Ivy cell type1'])]=False
    #rem = df_stp.loc[:,'name'].str.contains('HIPP').fillna(value=True)
    #in_stp_data.loc[rem]=False
    
    # 17–20 Sprague-Dawley rats?? - too high depression? - 0.39
    rem = df_stp.loc[:,'name'].str.contains('Fuentealba_2008CA1->Ivy cell').fillna(value=True)
    in_stp_data.loc[rem]=False    
    
# show selected stp dataset
ddd=df_stp.loc[in_stp_data,['name','synapse_type_2', 'stp_freq','area']]
print(ddd)
print(in_stp_data.sum())

synij = np.repeat(False,in_stp_data.shape[0])

cols2 = ['name','perc']
cl2post =pd.DataFrame([],columns= cols2)
cl2pre =pd.DataFrame([],columns= cols2)


# strip cluster names
#list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
df_aba_vis_in_c.loc[:,'cluster_label']=df_aba_vis_in_c.loc[:,'cluster_label'].str.strip()
#df_aba_vis_l23glu_c.loc[:,'cluster']=df_aba_vis_l23glu_c.loc[:,'cluster'].str.strip()

do_simplfy_clusters = 1 # do not take into account minor subclasses
Sst_NonMC ='Sst, Sst Calb2 Pdlim5-, Sst Chrna2 Glra3-, Sst Chrna2 Ptgdr-, Sst Myh8 Etv1-, Sst Tac2 Myh4-, Sst Nr2f2 Necab1-, Sst Chodl-,  Sst Calb2 Necab1-, Sst Tac1 Htr1d-, Sst Myh8 Fibin-';

pregs3 = ['pre__'+s for s in pregs2.tolist()]
postgs3 = ['post_'+s for s in postgs2.tolist()]
ge_columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] +pregs3 +postgs3

stp_columns = stp_TM_names #list(set(df_stp.loc[0,:]).difference(set([0])))

# output datasets
stp_data = pd.DataFrame([], columns =['presynaptic', 'postsynaptic', 'stp_data'] )
ge_data = pd.DataFrame([], columns =ge_columns+stp_columns+['index_ds'])


for i in range(1,len(pre_post.index)): #range(1,len(pre_post.index)): # [1]: 
    
    # syne3 - all available pre- and post- synaptic cells annotations : cre-lines, layers, morphologies
    l = str.strip(df_stp.loc[i,'area'])
    synel = re.split(';',l)
    if len(synel)==1:
        l1=l
    else:
        l  = str.strip(synel[1])
        l1  = str.strip(synel[0])
            
    
    if type(df_stp.loc[i,'synapse_type'])!=float:
        #syne1 = re.split('\;',df_stp.loc[i,'synapse_type']) no cre-line annotations for hippocampal data
        syne1 = ''
    else:
        syne1 =''    
    #syne2 = df_stp.loc[i,'synapse_type2'].str.split(pat='->',expand=True)
    syne2 = re.split('->',df_stp.loc[i,'synapse_type_2'])
    if len(syne2)==1:
        pat ='→'
        syne2 = re.split(pat,df_stp.loc[i,'synapse_type_2'])

    syne3 ={'layer_pre':l1,  'cre_line_pre':'none', 'cell_type2_pre':str.strip(syne2[0]),'cluster_pre':[],
            'layer_post':l, 'cre_line_post':'none','cell_type2_post':str.strip(syne2[1]),'cluster_post':[],  } 

    # add cre-lines when available
    if len(syne1)>1:
        syne3['cre_line_pre']  =str.strip(syne1[0])
        syne3['cre_line_post'] =str.strip(syne1[1])
    
    # PV BC
    if (syne3['cell_type2_post']=='PV')|(syne3['cell_type2_post']=='PVBC'):  
        syne3['cluster_post']=[['Pvalb_3, Pvalb_10, Pvalb_4, Pvalb_6',100]] # modify!!! 
    if (syne3['cell_type2_pre']=='PV')|(syne3['cell_type2_pre']=='PVBC'):  
        syne3['cluster_pre']=[['Pvalb_3, Pvalb_10, Pvalb_4, Pvalb_6',100]] # modify!!!   
                     
  
    # HIPP -> HIPP Bartos
    if syne3['cell_type2_post']=='HIPP':  
        if syne3['layer_post'] == 'DG':
            syne3['cluster_post']=[['Sst_15, Sst_24, Sst_13, Sst_6',100]] # modify!!!        
    if syne3['cell_type2_pre']=='HIPP':  
        if syne3['layer_pre'] == 'DG':
            syne3['cluster_pre']=[['Sst_15, Sst_24, Sst_13, Sst_6',100]] # modify!!! 
            
            
    # HICAP -> HICAP Bartos
    if syne3['cell_type2_post']=='HICAP':  
        if syne3['layer_post'] == 'DG':
            syne3['cluster_post']=[['Sncg6, Sncg/Ndnf HPF_1',100]] # modify!!!        
    if syne3['cell_type2_pre']=='HICAP':  
        if syne3['layer_pre'] == 'DG':
            syne3['cluster_pre']=[['Sncg6, Sncg/Ndnf HPF_1',100]] # modify!!!      
            
            
    # O-LM
    if syne3['cell_type2_post']=='O-LM':  
        if syne3['layer_post'] == 'CA1':
            syne3['cluster_post']=[['Sst_15, Sst_24, Sst_13, Sst_6',100]] # modify!!!     
            
    # CCK BC
    if syne3['cell_type2_post']=='CCKBC':  
        syne3['cluster_post']=[['Sncg/Ndnf HPF_2, Sncg/Ndnf HPF_3, Sncg/Ndnf HPF_6, Serpinf1_1, Sncg_7, Vip_14',100]] # modify!!!    
    if syne3['cell_type2_pre']=='CCKBC':  
        syne3['cluster_pre']=[['Sncg/Ndnf HPF_2, Sncg/Ndnf HPF_3, Sncg/Ndnf HPF_6, Serpinf1_1, Sncg_7, Vip_14',100]] # modify!!!    

    
    # CCK DTI
    if syne3['cell_type2_post']=='CCK_DTI':  
        syne3['cluster_post']=[['Sncg/Ndnf HPF_1, Sncg6, Sncg2',100]] # modify!!!    
    if syne3['cell_type2_pre']=='CCK_DTI':  
        syne3['cluster_pre']=[['Sncg/Ndnf HPF_1, Sncg6, Sncg2',100]] # modify!!!    

        
        
    # O-Bi
    if syne3['cell_type2_post']=='O-Bi':  
        if syne3['layer_post'] == 'CA1':
            syne3['cluster_post']=[['Sst_27, Pvalb_1, Sst_25',100]] # modify!!!   
     
    # AAC
    if syne3['cell_type2_pre']=='AAC':  
        if syne3['layer_pre'] == 'CA3':
            syne3['cluster_pre']=[['Pvalb_12, Pvalb_11',100]] # modify!!!   
    if syne3['cell_type2_post']=='AAC':  
        if syne3['layer_post'] == 'CA3':
            syne3['cluster_post']=[['Pvalb_12, Pvalb_11',100]] # modify!!!         

    # NGF    ? type2 vs type1? - remove type 1?  
    if syne3['cell_type2_pre']=='NGF':  
        syne3['cluster_pre']=[['Lamp5_2, Lamp5_3, Lamp5_4, Lamp5_5, Lamp5 Lhx6_2, Lamp5 Lhx6_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='NGF')|(syne3['cell_type2_post']=='NGF type1')|(syne3['cell_type2_post']=='NGF type2'):  
        syne3['cluster_post']=[['Lamp5_2, Lamp5_3, Lamp5_4, Lamp5_5, Lamp5 Lhx6_2, Lamp5 Lhx6_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
     
    # Ivy cells   ? type2 vs type1? : CGE vs MGE? - no CGE is seen? or Lhs1 vs Lhs4? 
    if syne3['cell_type2_pre']=='Ivy cell':  
        syne3['cluster_pre']=[['Lamp5 Lhx6_1, Lamp5 Lhx6_4',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='Ivy cell')|(syne3['cell_type2_post']=='Ivy cell type1')|(syne3['cell_type2_post']=='Ivy cell type2'):  
        syne3['cluster_post']=[['Lamp5 Lhx6_1, Lamp5 Lhx6_4',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
       
    #IS3
    if syne3['cell_type2_pre']=='IS3':  
        syne3['cluster_pre']=[['Vip_6, Vip_12, Vip_15, Vip_17',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    
    ###### EXCitatory
    # CA1 PC        
    if (syne3['cell_type2_pre']=='CA1'): 
        syne3['cell_type2_pre']=='PC'
        if syne3['layer_pre'] == 'CA1':
            syne3['cluster_pre']=[['CA1sp_7, CA1sp_8',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='CA1'):  
        syne3['cell_type2_post']=='PC'
        if syne3['layer_post'][0:3] == 'CA1':
            syne3['cluster_post']=[['CA1sp_7, CA1sp_8',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
 
            
    # CA3 PC        
    if (syne3['cell_type2_pre']=='CA3'): 
        syne3['cell_type2_pre']=='PC'
        if syne3['layer_pre'] == 'CA3':
            syne3['cluster_pre']=[['CA3sp_6, CA3sp_5, CA3sp_7, CA3sp_1',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='CA3'):  
        syne3['cell_type2_post']=='PC'
        if syne3['layer_post'] == 'CA3':
            syne3['cluster_post']=[['CA3sp_6, CA3sp_5, CA3sp_7, CA3sp_1',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
    
    # CA3 PC ventral
    if (syne3['cell_type2_pre']=='CA3')|(syne3['cell_type2_pre']=='PC'): 
        syne3['cell_type2_pre']=='PC'
        if (syne3['layer_pre'] == 'CA3 ventral')&(syne3['layer_post'] == 'CA1 ventral'):
            syne3['cluster_pre']=[['CA3sp_2, CA3sp_4',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   

    
    
 
    # ECII SS        
    if (syne3['cell_type2_pre']=='EC'): 
        syne3['cell_type2_pre']=='PC'
        if (syne3['layer_pre'] == 'EC')&((syne3['layer_post'] == 'DG')|(syne3['layer_post'] == 'CA3')):
            syne3['cluster_pre']=[['L2/3 IT Ndst4 Endou_2',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       


    # ECIII PC        
    if (syne3['cell_type2_pre']=='EC'):
        syne3['cell_type2_pre']=='PC'
        if (syne3['layer_pre'] == 'EC')&((syne3['layer_post'][0:3] == 'CA1')):
            syne3['cluster_pre']=[['L2/3 IT Plch1_1, L2/3 IT Plch1_2',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       

            
    # CA1 PC ventral       
    if (syne3['cell_type2_pre']=='CA1 ventral'):  
        syne3['cell_type2_pre']=='PC'
        if syne3['layer_pre'] == 'CA1 ventral':
            syne3['cluster_pre']=[['CA1sp_1, CA1sp_2, CA1sp_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='CA1 ventral'):  
        syne3['cell_type2_post']=='PC'
        if syne3['layer_post'] == 'CA1 ventral':
            syne3['cluster_post']=[['CA1sp_1, CA1sp_2, CA1sp_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
    
    

    # DG PC      
    if (syne3['cell_type2_pre']=='DG'):
        syne3['cell_type2_pre']=='PC'
        #if syne3['layer_pre'] == 'DG':
        syne3['cluster_pre']=[['DG_1, DG_2, DG_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions       
    if (syne3['cell_type2_post']=='DG'): 
        syne3['cell_type2_post']=='PC'
        #if syne3['layer_post'] == 'DG':
        syne3['cluster_post']=[['DG_1, DG_2, DG_3',100]] # modify!!! - only ~35 cells? - selected by anterior and mid-anterior subregions   
   
    
    
    ###########  Select data:
    ##synij = (pre_post.loc[:,0].isin(pree))&(pre_post.loc[:,1].isin(poste))
    

    synij[:] = False
    synij[i] = in_stp_data[i]
    
    #print(syne3)
    
    if sum(synij)>0:
        print('\n\n')
        print('i = ',str(i))
        print(syne3)
        #if sum(in_stp_data&synij)>0:
        #    in_stp_data = in_stp_data&synij

        #add ge data
        #'layer_pre':l,  'cre_line_pre':'', 'cell_type2_pre':syne2[0].str.strip(),'cluster_pre':[]
        
        # PC or GABAergic dataset?
        batch = [ syne3['cell_type2_pre']=='PC',syne3['cell_type2_post']=='PC']
        
        # add ge data depending on available stp cell types labels:
        #  select presynaptic cells
        if 0: #batch[0]:
            #in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']]) #modify!!!
            in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion']==syne3['layer_pre'] #modify!!!
        else:
            if len(syne3['cluster_pre'])==0:
                # if len(syne3['cre_line_pre'])>0:
                in_preg = df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_pre'])
                ##in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                #in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_pre'])
            else:                  
                ##in_preg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                in_preg = df_aba_vis_in_c.loc[:,'region_label'].str.contains(syne3['layer_pre'])
                in_preg[:] = True
                
                cl = syne3['cluster_pre']
                #cols2 = ['name','perc']
                #cl2pre =pd.DataFrame([],columns= cols2)
                in_preg0 = in_preg.copy()
                in_preg[:] = False 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = str.strip(clns[icls])
                        clns5 = re.split('\_',clns4)
                        if len(clns5)<2:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass_label'].str.contains(clns6),'cluster_label']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2pre = cl2pre.append(cl22)
                    
                    in_preg = in_preg|(in_preg0&df_aba_vis_in_c.loc[:,'cluster_label'].isin(cls3))
                    
                if (len(syne3['cre_line_pre'])>0)&(syne3['cre_line_pre']!='none'):
                    #in_preg = in_preg&df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_pre'])
                    in_line=df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_pre'])
                    print('is pre cre line')
                    if in_line.sum()>0:
                        in_preg = in_preg&in_line                        
        
        #  select postsynaptic cells
        if 0: #batch[1]:
            #in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_post']]) #modify!!!
            in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion']==syne3['layer_post'] #modify!!!
        else:    
            if len(syne3['cluster_post'])==0:
                # if len(syne3['cre_line_post'])>0:
                in_postg = df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_post'])
                #in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                #in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion_label'].str.contains(syne3['layer_post'])
            else:    
                ## under construction ...
                ##in_postg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                in_postg = df_aba_vis_in_c.loc[:,'region_label'].str.contains(syne3['layer_post'])
                in_postg[:] = True
                
                cl = syne3['cluster_post']
                #cols2 = ['name','perc']
                #cl2post =pd.DataFrame([],columns= cols2)
                in_postg0 = in_postg.copy()
                in_postg[:] = False                 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = str.strip(clns[icls])
                        clns5 = re.split('\_',clns4)
                        if len(clns5)<2:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass_label'].str.contains(clns6),'cluster_label']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2post = cl2post.append(cl22)
                    
                    #in_postg = in_postg&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3)
                    in_postg = in_postg|(in_postg0&df_aba_vis_in_c.loc[:,'cluster_label'].isin(cls3))
                    
                if (len(syne3['cre_line_post'])>0)&(syne3['cre_line_post']!='none'):
                    in_line=df_aba_vis_in_c.loc[:,'full_genotype_label'].str.contains(syne3['cre_line_post'])
                    print('is post cre line')
                    if in_line.sum()>0:
                        in_postg = in_postg&in_line
            
           
            
        # make equal size pre- and post- datasets
        in_preg = in_preg&(in_preg.isna()==False)
        in_postg = in_postg&(in_postg.isna()==False)
        npre = in_preg.sum()
        npost = in_postg.sum()
        nsample = max([npost, npre] )
        #nsample = min([npost, npre] ) # number of cells
        
        if (nsample<60)&(npre>0)&(npost>0):
            nsample=60
            
        if (nsample>180)&(npre>0)&(npost>0):  # restrict nsample to avoid overrepresentation?
            nsample=180    
            
        print('npre = '+ str(npre)+' , npost = ' + str(npost) + ' , nsample = '+str(nsample))    
        
        
        ## select cell indices for gene expression sampling
        #do_bootstrap=1
        #if  do_bootstrap==0:
        #    idx2 =   np.mod(np.arange(nsample),npre) 
        #    in_preg = np.nonzero(in_preg)[0][idx2]
        #    idx2 =   np.mod(np.arange(nsample),npost) 
        #    in_postg = np.nonzero(in_postg)[0][idx2]
        #else :
        #    #random.uniform(low=0.0, high=1.0, size=None)
        #    idx2 =  np.floor( np.random.uniform(0,npre, nsample) ).astype('int')
        #    in_preg = np.nonzero(in_preg)[0][idx2]
        #    idx2 =  np.floor( np.random.uniform(0,npost, nsample) ).astype('int')
        #    in_postg = np.nonzero(in_postg)[0][idx2]
            
            
        # select cell indices for gene expression sampling
        do_bootstrap=1
        if  do_bootstrap==0:
            
            if (nsample<60)&(npre>0)&(npost>0):
                nsample=60
            
            if (nsample>180)&(npre>0)&(npost>0):  # restrict nsample to avoid overrepresentation?
                nsample=180 
            
            idx2 =   np.mod(np.arange(nsample),npre) 
            in_preg = np.nonzero(in_preg)[0][idx2]
            idx2 =   np.mod(np.arange(nsample),npost) 
            in_postg = np.nonzero(in_postg)[0][idx2]
        else :
            if (npre>0)&(npost>0):
                nsample=200
            #random.uniform(low=0.0, high=1.0, size=None)
            idx2pre =  np.floor( np.random.uniform(0,npre, (nsample,1)) ).astype('int')
            in_preg_bs = np.nonzero(in_preg)[0][idx2pre]
            idx2post =  np.floor( np.random.uniform(0,npost, (nsample,1)) ).astype('int')
            in_postg_bs = np.nonzero(in_postg)[0][idx2post]
            in_preg_ = in_preg_bs[:,0]
            in_postg_ = in_postg_bs[:,0]    
            


        
        
        
        if batch[0]:
            #in_preg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_preg,:].loc[:,'cluster']))
            in_preg_cl = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster_label'].value_counts()
        else:
            #in_preg_cl = list(set(df_aba_vis_in_c.iloc[in_preg,:].loc[:,'cluster']))
            in_preg_cl = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster_label'].value_counts()
        
        if batch[1]:
            #in_postg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_postg,:].loc[:,'cluster']))
            in_postg_cl = df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster_label'].value_counts()
        else:        
            #in_postg_cl = list(set(df_aba_vis_in_c.iloc[in_postg,:].loc[:,'cluster']))
            in_postg_cl = df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster_label'].value_counts()
        in_preg_cl = in_preg_cl/in_preg_cl.sum()*100    
        in_postg_cl = in_postg_cl/in_postg_cl.sum()*100    
        print('presynaptic clusters')
        print(in_preg_cl)
        print('postsynaptic clusters')
        print(in_postg_cl)

        #if npost>npre:
        #    idx = np.mod(np.arange(npost),npre)
        #    #prege = prege[idx,:]
        #    #postge = postge
        #    prege = prege
        #    postge = postge[0:npre,:]            
        #elif npost<npre :
        #    idx = np.mod(np.arange(npre),npost)
        #    #prege = prege
        #    #postge = postge[idx,:]
        #    prege = prege[0:npost,:]
        #    postge = postge         
        
        # take ge data    
        #prege = vis_dat.X[:, in_pregs]  # presynaptic gene set expression
        ##prege = prege[(vbi==batch[0]), :]
        #prege = prege[in_preg,:]

        #postge = vis_dat.X[:,  in_postgs] # postsynaptic gene set expression
        ##postge = postge[(vbi==batch[1]),  :]
        #postge = postge[in_postg,:]
        
        
        
        prege = vis_dat.X[:, in_pregs]  # presynaptic gene set expression
        prege = prege[(vbi==batch[0]), :]
        
        postge = vis_dat.X[:,  in_postgs] # postsynaptic gene set expression
        postge = postge[(vbi==batch[1]),  :]
        
        do_median=1
        if  (do_bootstrap==0)|(do_median==0):
            prege = prege[in_preg,:]
            postge = postge[in_postg,:]
        else: 
            nsample = 100
            Nbs = nsample
            ##random.uniform(low=0.0, high=1.0, size=None)
            #nmax=5
            nmax = npre #np.min([1000,npre])
            idx2pre =  np.floor( np.random.uniform(0,npre, (nmax,Nbs)) ).astype('int')
            in_preg_ = np.nonzero(in_preg)[0][np.arange(npre)]
            nmax = npost #np.min([1000,npost])
            idx2post =  np.floor( np.random.uniform(0,npost, (nmax,Nbs)) ).astype('int')
            in_postg_ = np.nonzero(in_postg)[0][np.arange(npost)]

                        # 
            # ??? transform ge to median bootstraped log ???
            prege_ = prege[in_preg_,:]
            postge_ = postge[in_postg_,:]

            y = np.ma.masked_where(prege_ == 0, prege_)
            prege_=np.ma.log2(y).filled(0)
            y = np.ma.masked_where(postge_ == 0, postge_)
            postge_=np.ma.log2(y).filled(0)



            npregs=prege_.shape[1]
            ge_bs = np.zeros((Nbs,npregs+postge_.shape[1]))
            for ibs in range(Nbs):
                #ans=np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, x)
                #ans[np.isnan(ans)]=0.
                xbs = prege_[idx2pre[:,ibs],:]
                pregbs  = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                pregbs[np.isnan(pregbs)]=0.
                xbs = postge_[idx2post[:,ibs],:]
                postgbs = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                postgbs[np.isnan(postgbs)]=0.

                ge_bs[ibs,0:npregs] =pregbs
                ge_bs[ibs,npregs:]  =postgbs


            #df_hr2 = np.log2(df_hr2d)
            #df_hr2 = pd.concat([df_hr2,df_hr2s],axis=1) # log2 normalization
            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hr2 = np.log2(df_hr2)

            #df_hr2 = pd.concat([df_hr.iloc[1:,0],df_hr2],axis=1)

            #df_hr2.dtypes
            #df_hr2.head()

            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hrmg=df_hr2.groupby(['Class']) # logarithm_2 of UMI counts
            #df_hrmg=df_hrmg.median(numeric_only=True) # median of log2(UMI)

            #df_hrmg0 = df_hrmg.iloc[:,0:-1]

            #df_hrmsg = pd.concat([df_hrmg.iloc[:,-1]]*df_hrmg0.shape[1],axis=1)
            #df_hrmsg.columns = df_hrmg.iloc[:,0:-1].columns

            #df_hrmg = df_hrmg.iloc[:,0:-1] - df_hrmsg +6*np.log2(10)  # transform from log2( UMI counts) to log2(CPM)

            # ??? transform ge to median bootstraped log ???


        
        

              

        ## add data to the final dataset    
        #gnij = np.repeat(np.array([syne3['cell_type2_pre'], syne3['cell_type2_post'],
        #                           syne3['layer_pre'], syne3['layer_post'],
        #                           syne3['cre_line_pre'], syne3['cre_line_post']]).reshape((1,6)),nsample,axis=0 )
        #gnij = pd.DataFrame(gnij, columns = ['cell_type2_pre', 'cell_type2_post',
        #                                     'layer_pre','layer_post',
        #                                     'cre_line_pre', 'cre_line_post'] )
        #prege  = pd.DataFrame(prege,columns  = pregs3 )
        #postge = pd.DataFrame(postge,columns = postgs3 )


        #ge_dataij = pd.concat([gnij, prege, postge ], axis = 1)
        
        
        # add data to the final dataset    
        gnij = np.repeat(np.array([syne3['cell_type2_pre'], syne3['cell_type2_post'],
                                   syne3['layer_pre'], syne3['layer_post'],
                                   syne3['cre_line_pre'], syne3['cre_line_post']]).reshape((1,6)),nsample,axis=0 )
        gnij = pd.DataFrame(gnij, columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] )
        
        if  (do_bootstrap==0)|(do_median==0):
            prege  = pd.DataFrame(prege,columns  = pregs3 )
            postge = pd.DataFrame(postge,columns = postgs3 )


            ge_dataij = pd.concat([gnij, prege, postge ], axis = 1)
            
        else:
            ge_bs = pd.DataFrame(ge_bs,columns = pregs3 + postgs3 )


            ge_dataij = pd.concat([gnij, ge_bs ], axis = 1)

       
        
        
        # add stp data
        # stp_dataij =  df_stp.loc[[0,i], stp_type['stp_data_type']!=0]
        stp_dataij =  df_stp.loc[[0,i], df_stp.loc[0, :]!=0]
        

        i2 = np.trunc(np.arange(len(stp_dataij.columns))/len(stp_columns)).astype(int)
        stp_dataij  = stp_dataij.T.set_index(i2).pivot(columns=0,values=i).loc[:,stp_columns] #ss.set_index('par',append=True).unstack('par')
        #stp_dataij =  stp_dataij.T.pivot(index=0, values=synij).T
        
        idx = np.mod(np.arange(nsample),len(stp_dataij.index))
        stp_dataij=stp_dataij.iloc[idx,:].reset_index(drop=True)  
        
        i_data_set = pd.DataFrame(i+np.zeros(len(stp_dataij.index)), index = stp_dataij.index, columns = ['index_ds'])
        stp_dataij=pd.concat([stp_dataij, i_data_set],axis=1)
        
        #if do_averagig_stp:
        #    #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
        #    stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values

        
        #stp_dataij = pd.DataFrame({'presynaptic':pree[0], 'postsynaptic': poste[0], 
        #                           'stp_data':stp_dataij.iloc[0,:].values})
        #stp_data  = stp_data.append(stp_dataij) 
        
        # names of ge and stp columns

        
        ge_dataij  = pd.concat([ge_dataij,stp_dataij] , axis=1)
        
        # add to all ge and stp dataframe
        ge_data  = pd.concat([ge_data, ge_dataij] , axis=0)
        
#stp_data =  stp_data.reset_index(drop=True)  



#stp_data.loc[stp_data.loc[:,'postsynaptic']=='NGC','postsynaptic']='Lamp5'
#stp_data2 = ge_data.iloc[:,0:3].copy(deep=True)                                 # modify!
#stp_data2.columns=['presynaptic', 'postsynaptic', 'ppr']
#print(stp_data2.head())
#for i in stp_data.index: #stp_data.index:
#    pre=stp_data.loc[i,'presynaptic']
#    post=stp_data.loc[i,'postsynaptic']
#    stp_data2.loc[stp_data2.loc[:,'presynaptic'].isin([pre])&stp_data2.loc[:,'postsynaptic'].isin([post]),'ppr']=stp_data.loc[i,'stp_data'] 
    
ge_data = ge_data.reset_index()    




# transform labels from TM to An:A1
#f = 20 # Hz
#N = 5
#T = np.arange(N)*1000/f


#As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)
As = STP_sim_complex(ge_data,ge_data.shape[0],[])
N  = As.shape[1]

#As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T
As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T

#As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T

# add new stp parameters to ge_data dataset
stp_columns2 = ['A'+str(s)+'/A1' for s in range(2,N+1)]
stp_data2 = pd.DataFrame(As,columns = stp_columns2, index = ge_data.index)

ge_data = pd.concat([ge_data, stp_data2 ],axis=1)

if do_averagig_stp:
     #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
     #stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values
    all_i_ds =  ge_data.loc[:,'index_ds']                             
    for i_ds in list(set(all_i_ds)):
              ids2 =  ge_data.loc[:,'index_ds'].isin([i_ds])  
              ge_data.loc[ids2,stp_columns2] = ge_data.loc[ids2,stp_columns2].median(axis=0).values                   
                                   
                                   
## filter TM parameters
do_filter_pars = 0

if do_filter_pars!=0:
    print((ge_data.loc[:,stp_columns ]<0).sum().sum())
    print(len(ge_data.index))

    nannot=6
    annot_columns = ge_columns[0:nannot]
    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f,ax = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax)
    plt.gca().legend('')
    #plt.gca().title('An:A1 before filtering data')
    ax.title.set_text('An:A1 before filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    print('mean of stp before filtering:')
    print(ge_data2)

    #i_filter_out = (ge_data.loc[:,'dp/p0']*ge_data.loc[:,'p0']+)|(ge_data.loc[:,'tD'])
    #i_filter_out = i_filter_out&((ge_data.loc[:,'tF'])|(ge_data.loc[:,'tF']))
    i_filter_out = np.sum(ps2>1,axis=1)|np.sum(ps2<0,axis=1)
    i_filter_out = i_filter_out|(np.sum(ns2>1,axis=1)|np.sum(ns2<0,axis=1))
    ge_data = ge_data.loc[i_filter_out==False ,:]

    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f1,ax1 = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax1)
    plt.gca().legend('')
    #plt.gca().title('An:A1 after filtering data')
    ax1.title.set_text('An:A1 after filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()

    print('mean of stp after filtering:')
    print(ge_data2)
    
#print('mean of stp after filtering:')
#print(ge_data2)


print(len(ge_data.index))
#ge_data.loc[:,stp_columns ] =ge_data.loc[:,stp_columns ].abs()  


#
## normalize stp parameters - for TM parameters
do_normalize_stp_pars = 0;
if do_normalize_stp_pars!=0:
    ge_data.loc[:,stp_columns  ] =np.log(ge_data.loc[:,stp_columns  ].values.astype(float))

#  assign training and labels 
stp_columns1 = stp_columns 
stp_columns = stp_columns2 # select training labels names

nannot = 7
nstp =len(stp_columns)
annot_columns = ge_columns[0:6]+['index_ds']

## averaging of stp for each synapse type 


#X = ge_data.loc[:,ge_data.columns[0:]].values
X = ge_data.loc[:,annot_columns + ge_columns[6:]].values
y = ge_data.loc[:,annot_columns + stp_columns].values 
#y = stp_data2.loc[:,:].values

## normalize stp parameters - for TM parameters
#y[:,nannot: ] =np.log(y[:,nannot: ].astype(float))


    

# print elapsed time
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))

print(y.shape)
X.shape

In [ ]:
ge_data_h = ge_data
X_h = X
y_h = y
annot_columns_h = annot_columns
ge_columns_h = ge_columns
stp_columns_h = stp_columns

In [ ]:
ge_data_h.to_excel('ge_data_h.xlsx')

##
##
##  CORTEX gene expression
##
##

In [ ]:
save_path = 'data/'

#d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'
if do_laptop:
    d4 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/'
else:
    d4='~/Documents/references/transcriptomes to STP/transcriptomes/ABA/mouse_VISp_gene_expression_matrices_2018-06-14/'

fn='mouse_VISp_2018-06-14_samples-columns.csv'
df_aba_vis_c=pd.read_csv(d4+fn)
print(df_aba_vis_c.columns)
print(df_aba_vis_c.head())

fn = 'mouse_VISp_2018-06-14_genes-rows.csv'
df_aba_g=pd.read_csv(d4+fn)
print(df_aba_g.head())
print(len(df_aba_vis_c.index))

in_glut_l23 = (df_aba_vis_c.loc[:,'class']=='Glutamatergic') #&(df_aba_vis_c.loc[:,'brain_subregion'].isin(['L2/3','L4','L5']))

in_gaba = df_aba_vis_c.loc[:,'class']=='GABAergic'
print(sum(in_gaba))
df_aba_vis_in_c=df_aba_vis_c.loc[in_gaba,:]
df_aba_vis_l23glu_c=df_aba_vis_c.loc[in_glut_l23,:]

###
print('loading gene expression...')

from scvi.dataset import LoomDataset, CsvDataset, Dataset10X, AnnDataset

#df_aba_ad.write('mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad')
aba_vis_in = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_GABAergic.h5ad", new_n_genes = 45768,
                               save_path = save_path) 

#aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L23_Glutamatergic_hda.h5ad", new_n_genes = 45768,
#                               save_path = save_path) 
#aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L2345_Glutamatergic_hda.h5ad", new_n_genes = 45768,
#                               save_path = save_path) 

aba_vis_l23glu = AnnDataset("mouse_VISp_2018-06-14_exon-matrix_L23456_Glutamatergic_hda.h5ad", new_n_genes = 45768,
                               save_path = '') 

diff_order = sum(aba_vis_in.gene_names[0:].astype(int) - np.arange(len(aba_vis_in.gene_names)))
print(diff_order)

aba_vis_in.gene_names = df_aba_g.loc[:,'gene_symbol'].values

aba_vis_l23glu.gene_names = df_aba_g.loc[:,'gene_symbol'].values



print(len(aba_vis_in.gene_names))
print(len(aba_vis_l23glu.gene_names))
len(list(set(aba_vis_l23glu.gene_names).difference(set(aba_vis_in.gene_names))))
print(len(df_aba_vis_l23glu_c.index))
print(len(df_aba_vis_l23glu_c.index))
print(len(df_aba_vis_in_c.index))

from scvi.dataset.dataset import GeneExpressionDataset

vis_dat = GeneExpressionDataset.concat_datasets(aba_vis_in,aba_vis_l23glu)    
    

print((vis_dat.batch_indices==0).sum())
(vis_dat.batch_indices==1).sum()

##
##
##  CORTEX STP
##
##

In [ ]:


if do_laptop:
    d5 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/additional cortical STP data/'
else:
    d5 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/additional cortical STP data/'
#fn5='STP_cortex_interneurons_Cre_labeled.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF2.xlsx'
fn5='STP_cortex_interneurons_Cre_labeled_UDF3.xlsx'

df_stp=pd.read_excel(d5+fn5)
df_stp.loc[df_stp.loc[:,'stp_comment'].isna() ,'stp_comment']=''

############   
do_averagig_stp =0
stp_TM_names=['tF', 'p0','tD','dp/p0','A']
stp_TM_start = 29
ncol = len(df_stp.columns)
df_tm_name = pd.DataFrame(np.zeros(ncol).reshape((1,ncol)), columns =df_stp.columns )
df_stp = pd.concat([df_tm_name, df_stp],axis=0).reset_index(drop=True)

for i,nm in enumerate(stp_TM_names):
    ii = stp_TM_start + np.arange(0,ncol-i-stp_TM_start,len(stp_TM_names))+i
    df_stp.iloc[0,ii] = nm

#df_stp.loc[df_stp.loc[:,'name'].str.contains('Jaing_2015_PC')!=True,:].reset_index(drop=True)

################# ADD ABA STP data
do_add_aba_synphys=True

#QTX.to_hdf('fit_aba_2019_A1_8_fited_pulses_syntypes_results','data')
d_aba = '/home/stepaniu/Documents/references/transcriptomes to STP/'
#STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_A1_8_fited_pulses_TM_8_syntypes_results') # tm5 + FDR

#STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_A1_8_fited_pulses_syntypes_results') # tm5
#QTX2 = STP_aba.iloc[0:43,0:14]
#QTX3 = STP_aba.iloc[43:,14:]
#QTX3.index = QTX2.index
#STP_aba = pd.concat([QTX2, QTX3],axis=1)

STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_mean_fited_pulses_TM_SMP0_100bs_syntypes_results') # tm5 + SMR
#STP_aba = pd.read_hdf(d_aba + 'fit_aba_2019_mean_fited_pulses_TM_SMP0_50bs_syntypes_results') 

STP_aba

In [ ]:
%matplotlib inline
# transform labels from TM to An:A1
fs = [20, 50, 10] # Hz
N = 8



#Trec = [125, 250, 500, 1000, 2000]
Trec = [ 250, 500, 1000]
DT0 = 25000
#xs  =ge_data.iloc[l_pre_post2:,:].loc[:,stp_aba_names].values

idx = 'vip L2/3; sst L2/3'
xs  = STP_aba.loc[idx,'differential_evolution best x']

xs  =np.delete(xs, [5,6],axis=1)
As=np.zeros((l_pre_post2,0))
As2=np.zeros((xs.shape[0],0))
if do_add_aba_synphys:
    
    for f in fs:
        T = np.arange(N)*1000/f
        
        Asf, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],T)
        As = np.concatenate([As,Asf],axis=1)
        
        for ri in range(len(Trec)):
            Asr, nsr, psr, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],[Trec[ri]],init_state=[ns[:,-1],ps[:,-1]])
            As = np.concatenate([As,Asr],axis=1)
        

        As2f = np.zeros((xs.shape[0],N+len(Trec)))
        for i2 in range(xs.shape[0]):
            #as2, sts2 = STP_sim2(xs[i2,:],np.arange(8)*50,model_type = 'tm5') # preconditioning series???
            #as2, sts2 = STP_sim2(xs[i2,:], T+DT0, init_state=sts2[-1], model_type='tm5' ) 
            
            as2, sts2 = STP_sim2(xs[i2,:],T,model_type = 'tm5_smr')
            
            As2f[i2,0:N] = as2
            for ri in range(len(Trec)):
                as2r, sts2r = STP_sim2(xs[i2,:], [Trec[ri]], init_state=sts2[-1], model_type='tm5_smr' ) 
                As2f[i2,N+ri] = as2r
                
        As2 = np.concatenate([As2,As2f],axis=1)
            
    
    As2 = As2/As2[:,0].reshape((-1,1))
    As = As/As[:,0].reshape((-1,1))
    As = np.concatenate([As,As2],axis=0)
else:
    As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)
    

import matplotlib.pyplot as plt    
plt.plot(As2[:,:].transpose(),'o-')

## CORTEX : allign gene expression and stp data (for selected gene sets)

In [ ]:
# allign gene expression and stp data (for selected gene sets)
import re
import time
t1 = time.time()



# names of gene expression based synapses types

glu_l23=list(set(df_aba_vis_l23glu_c.loc[:,'cluster']))

dsg = [['PC']+glu_l23,
       ['Pvalb','Pvalb Tpbg','Pvalb Reln Itm2a','Pvalb Reln Tac1','Pvalb Sema3e Kank4'], 
       ['Sst','Sst Calb2 Pdlim5','Sst Calb2 Necab1', 'Sst Hpse Cbln4','Sst Hpse Sema3c','Sst Nr2f2 Necab1'],
       ['Vip','Vip Lmo1 Myl1', 'Vip Rspo1 Itga4','Vip Ptprt Pkp2','Vip Rspo4 Rxfp1 Chat'],
       ['Lamp5','Lamp5 Plch2 Dock5'],
       ['MC','L23MC','Sst Calb2 Pdlim5'],
        'L5MC']
      

set(df_stp.loc[:,'synapse_type_2'])

# names of electrophysiology based synapses types
dse = [['PC', 'L23P', 'L23PC', 'L5P'],
       ['Pvalb', 'PV', 'L23BC', 'L5BC'], 
       ['Sst', 'L23MC', 'L5MC', 'Sst', 'MC'], 
       ['Vip', 'VIP', 'BTC'],
       ['NGC', 'L23NGC'],
       ['L23MC' ],
       ['L5MC']]



# load stp data table
if do_laptop:
    d5 = '~/Documents/references/lists_of_proteins_2/Analysis_of_proteins/genetic_types/transcriptomic_interneurons/additional cortical STP data/'
else:
    d5 = '/home/stepaniu/Documents/references/transcriptomes to STP/scVI_data/additional cortical STP data/'
#fn5='STP_cortex_interneurons_Cre_labeled.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF2.xlsx'
#fn5='STP_cortex_interneurons_Cre_labeled_UDF3.xlsx'

do_3pars_TM=0 # 3 or 4 parametric TM model?
if do_3pars_TM:
    fn5='STP_cortex_interneurons_Cre_labeled_UDF5_3pars.xlsx'
else:
    fn5='STP_cortex_interneurons_Cre_labeled_UDF5_4pars.xlsx'

df_stp=pd.read_excel(d5+fn5)
df_stp.loc[df_stp.loc[:,'stp_comment'].isna() ,'stp_comment']=''

# drop TC->
TC_names = ['Miao_2016_fig1_TC->PC','Miao_2016_fig1_TC->PV']
df_stp = df_stp.drop(index = df_stp.index[df_stp.loc[:,'name'].isin(TC_names)] )

############   
do_averagig_stp =0
stp_TM_names=['tF', 'p0','tD','dp/p0','A']
stp_TM_start = 29
ncol = len(df_stp.columns)
df_tm_name = pd.DataFrame(np.zeros(ncol).reshape((1,ncol)), columns =df_stp.columns )
df_stp = pd.concat([df_tm_name, df_stp],axis=0).reset_index(drop=True)

for i,nm in enumerate(stp_TM_names):
    ii = stp_TM_start + np.arange(0,ncol-i-stp_TM_start,len(stp_TM_names))+i
    df_stp.iloc[0,ii] = nm



print(df_stp.head())
#############
##Remove pure PPR data (Jaing_2015_PC)
#df_stp = df_stp.loc[df_stp.loc[:,'name'].str.contains('Jaing_2015_PC')!=True,:].reset_index(drop=True) 

############

# batches and gene sets
pre_post = df_stp.loc[:,'synapse_type_2'].str.split(pat='->',expand=True)

if do_add_aba_synphys:
    pre_post_aba = STP_aba.loc[:,'synapse_type_2'].str.split(pat=';',expand=True)

vbi = vis_dat.batch_indices.reshape((vis_dat.batch_indices.shape[0],))
vbi=vbi>0

df_gs  = pd.DataFrame( vis_dat.gene_names ) 
in_pregs = df_gs.loc[:,0].isin(pregs).values
in_postgs = df_gs.loc[:,0].isin(postgs).values

pregs2 = df_gs.loc[in_pregs,0].values
postgs2 = df_gs.loc[in_postgs,0].values


do_2=1;
# select stp data for appropriate frequency and ages 
stp_type = {'name':['Walker_2016_PV->MC L23',
                    'Walker_2016_VIP->MC L23',
                    'Yuste_2016_taus_PV->Sst L23 recalc',
                    'Yuste_2016_taus_PV->VIP L23 recalc',
                    'Yuste_2016_taus_Sst->VIP L23 recalc',
                    'Yuste_2016_taus_VIP->Sst L23 recalc',
                    'Yuste_2016_taus_VIP->VIP L23 recalc'],
            'name_remove': [], 
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }

if do_add_aba_synphys:
    STP_aba_type = {'name':[],
            'stp_freq': [], 'stp_comment':  [],
            'stp_data_type': 'differential_evolution best x' }  # 'random_starts best x'
    #stp_aba_names = ['tF', 'p0','tD','dp','A','A1','A2'] # tm5
    stp_aba_names = ['tF', 'p0','tD','dp','A','A1','A2','tDmin','dd','t_FDR','t_SMR','dp0'] # tm5+smr
    
if do_2==1:
    #stp_type = {'name':['Walker_2016_PV->MC L23',
    #               'Walker_2016_VIP->MC L23',
    #                'Yuste_2016_taus_PV->Sst L23 recalc',
    #                'Yuste_2016_taus_VIP->Sst L23 recalc'],
    #        'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
    #        'stp_data_type': df_stp.loc[0, :]!=0 }
    
    # without Yuste and
    stp_type = {'name':['Walker_2016_PV->MC L23',
                    'Walker_2016_VIP->MC L23'],
             'name_remove': ['Jaing','Yuste'],  
            'stp_freq': [20, 40, 50], 'stp_comment':  ['', ' P20-21', ' P20-22', ' P20-23', ' P25-31'],
            'stp_data_type': df_stp.loc[0, :]!=0 }




in_stp_data = df_stp.loc[:,'stp_freq'].isin([20])
in_stp_data= in_stp_data|(df_stp.loc[:,'stp_freq'].isin([40,50])&df_stp.loc[:,'name'].isin(stp_type['name']))
in_stp_data = in_stp_data&df_stp.loc[:,'stp_comment'].isin(pd.Series(stp_type['stp_comment']).str.strip())
if len(stp_type['name_remove'])>0:
        remove = df_stp.loc[:,'name'].str.contains(stp_type['name_remove'][0])
        for rmn in stp_type['name_remove']:
             remove= remove|df_stp.loc[:,'name'].str.contains(rmn)
        in_stp_data = in_stp_data&( remove!=True)
        
#show selected stp dataset
ddd=df_stp.loc[in_stp_data,['name','synapse_type_2', 'stp_freq','area']]
print(ddd)
print(in_stp_data.sum())

synij = np.repeat(False,in_stp_data.shape[0])
if do_add_aba_synphys:
    synij = np.repeat(False,in_stp_data.shape[0]+STP_aba.shape[0])

cols2 = ['name','perc']
cl2post =pd.DataFrame([],columns= cols2)
cl2pre =pd.DataFrame([],columns= cols2)


# strip cluster names
#list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
df_aba_vis_in_c.loc[:,'cluster']=df_aba_vis_in_c.loc[:,'cluster'].str.strip()
df_aba_vis_l23glu_c.loc[:,'cluster']=df_aba_vis_l23glu_c.loc[:,'cluster'].str.strip()

do_simplfy_clusters = 1 # do not take into account minor subclasses
Sst_NonMC ='Sst, Sst Calb2 Pdlim5-, Sst Chrna2 Glra3-, Sst Chrna2 Ptgdr-, Sst Myh8 Etv1-, Sst Tac2 Myh4-, Sst Nr2f2 Necab1-, Sst Chodl-,  Sst Calb2 Necab1-, Sst Tac1 Htr1d-, Sst Myh8 Fibin-';

pregs3 = ['pre__'+s for s in pregs2.tolist()]
postgs3 = ['post_'+s for s in postgs2.tolist()]
ge_columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] +pregs3 +postgs3

stp_columns = stp_TM_names #list(set(df_stp.loc[0,:]).difference(set([0])))

# output datasets
stp_data = pd.DataFrame([], columns =['presynaptic', 'postsynaptic', 'stp_data'] )
ge_data = pd.DataFrame([], columns =ge_columns+stp_columns+['index_ds'])

if do_add_aba_synphys:
    l_pre_post=len(pre_post.index)+len(pre_post_aba.index)
else:
    l_pre_post=len(pre_post.index)
l_pre_post1 = len(pre_post.index)

for i in range(1,l_pre_post): #range(1,len(pre_post.index)):# range(len(pre_post.index)): # [1]: #range(len(pre_post.index)):
    if i==l_pre_post1:
        l_pre_post2 = ge_data.shape[0]
        
    # syne3 - all available pre- and post- synaptic cells annotations : cre-lines, layers, morphologies
    if i<l_pre_post1:
        l = str.strip(df_stp.loc[i,'area'])
        if type(df_stp.loc[i,'synapse_type'])!=float:
            syne1 = re.split('\;',df_stp.loc[i,'synapse_type'])
        else:
            syne1 =''    
        #syne2 = df_stp.loc[i,'synapse_type2'].str.split(pat='->',expand=True)
        syne2 = re.split('->',df_stp.loc[i,'synapse_type_2'])
        if len(syne2)==1:
            pat ='→'
            syne2 = re.split(pat,df_stp.loc[i,'synapse_type_2'])

        syne3 ={'layer_pre':l,  'cre_line_pre':'none', 'cell_type2_pre':str.strip(syne2[0]),'cluster_pre':[],
                'layer_post':l, 'cre_line_post':'none','cell_type2_post':str.strip(syne2[1]),'cluster_post':[],  } 

        # add cre-lines when available
        if len(syne1)>1:
            syne3['cre_line_pre']  =str.strip(syne1[0])
            syne3['cre_line_post'] =str.strip(syne1[1])

        # introduce study-specific modifications to syne3 (using morphology+layer->clusters map)
        if  df_stp.loc[i,'name'][0:5]=='Jaing':

            # modify layer_post and reduce cell_type_post to mtype name
            if syne3['cell_type2_post'][0:3]=='L23':
                syne3['layer_post'] = 'L2/3'
                syne3['cell_type2_post']=syne3['cell_type2_post'][3:]
            elif syne3['cell_type2_post'][0:2]=='L5':
                     syne3['layer_post'] = 'L5'
                     syne3['cell_type2_post']=syne3['cell_type2_post'][2:]  

            #  reduce cell_type_pre to mtype name
            if syne3['cell_type2_pre'][0:3]=='L23':
                syne3['layer_pre'] = 'L2/3'
                syne3['cell_type2_pre']=syne3['cell_type2_pre'][3:]
            elif syne3['cell_type2_pre'][0:2]=='L5':
                     syne3['layer_pre'] = 'L5'
                     syne3['cell_type2_pre']=syne3['cell_type2_pre'][2:]    

            # for each mtype specify expected ABI gene custers and their expected % (based on Jiang 2015 cre-lines to morhology table)
            if syne3['cell_type2_post']=='MC':
                if syne3['layer_post'] == 'L2/3':
                    syne3['cluster_post']=[['Sst Calb2 Pdlim5',100]]
                if syne3['layer_post'] == 'L5':
                    syne3['cre_line_post']='Chrna2'  
                    syne3['cluster_post']=[['Sst',100]]

            if syne3['cell_type2_post']=='BC':  
                if syne3['layer_post'] == 'L2/3':
                    if do_simplfy_clusters==0:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',75],[Sst_NonMC,25]] # modify!!!    
                    else:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',100]] # modify!!!  

                if syne3['layer_post'] == 'L5':
                    if do_simplfy_clusters==0:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',75],[Sst_NonMC,25]] # modify!!!    
                    else:
                        syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-',100]] # modify!!!    
            if syne3['cell_type2_post']=='BTC':
                if do_simplfy_clusters==0:
                    syne3['cluster_post']=[['Vip Lmo1 Myl1, Vip Ptprt Pkp2, Vip Rspo4 Rxfp1 Chat',90], # modify!!! 
                                         [Sst_NonMC,10]]
                else:
                    syne3['cluster_post']=[['Vip Lmo1 Myl1, Vip Ptprt Pkp2, Vip Rspo4 Rxfp1 Chat',100]]
            if syne3['cell_type2_post']=='HEC':  # modify!!! 
                if do_simplfy_clusters==0:
                    syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-,Pvalb Tpbg-,Pvalb Reln Itm2a-,Pvalb Reln Tac1-',75],
                                           [Sst_NonMC,25]]
                else:
                    syne3['cluster_post']=[['Pvalb, Pvalb Vipr2-,Pvalb Tpbg-,Pvalb Reln Itm2a-,Pvalb Reln Tac1-',100]]            
            if syne3['cell_type2_post']=='NGC':            
                  syne3['cluster_post']=[['Lamp5 Ntn1 Npy2r, Lamp5 Plch2 Dock5',100]] 

        if  df_stp.loc[i,'name'][0:5]=='Walke':    
            if syne3['cell_type2_post']=='MC':
                 syne3['cluster_post']=[['Sst Calb2 Pdlim5',100]]            

        #if  df_stp.loc[i,'name'][0:5]=='Miao_':     

        #if  df_stp.loc[i,'name'][0:5]=='Yuste':     
    
    # ABA synphys dataset
    if i>=l_pre_post1:
        #l = str.strip(STP_aba.loc[i1,'area'])

            
        i1 = STP_aba.index[i - l_pre_post1]
          
        syne0 = re.split('\,',STP_aba.loc[i1,'synapse_type_2'])
        syne1 = re.split('\;',syne0[0])
        
        syne0 = re.split('\,',STP_aba.loc[i1,'comments'])
        syne1 = re.split('\;',syne0[0])
        syne12 = re.split('\'',syne1[0])[1]
        syne13 = re.split('\'',syne1[1])[0]

        syne_pre = re.split('\s+',str.strip(syne12))
        syne_post = re.split('\s+',str.strip(syne13))
        
   
        l1 = str.strip(syne_pre[1])
        l2 = str.strip(syne_post[1])
        
        syne3 ={'layer_pre':[l1],  'cre_line_pre':'none', 'cell_type2_pre':'PC','cluster_pre':[],
                'layer_post':[l2], 'cre_line_post':'none','cell_type2_post':'PC','cluster_post':[]  } 

        # add cre-lines when available
        in_lines = pd.DataFrame(['vip','pvalb','sst'])
        if str.strip(syne_pre[0])!='unknown':
            syne3['cre_line_pre']   =str.strip(syne_pre[0])
            if in_lines.isin([syne3['cre_line_pre']]).any()[0]:
                syne3['cell_type2_pre'] = syne3['cre_line_pre'] 
        else:
            syne3['cell_type2_pre']   = 'PC'
            
        if str.strip(syne_post[0])!='unknown':
            syne3['cre_line_post']  =str.strip(syne_post[0])  
            if in_lines.isin([syne3['cre_line_post']]).any()[0]:
                syne3['cell_type2_post'] = syne3['cre_line_post'] 
        else:
            syne3['cell_type2_post']   = 'PC'    
            
            
        if syne3['cre_line_pre']=='tlx3':
            syne3['layer_pre']=['L4-L6']
                    
        if syne3['cre_line_post']=='tlx3':
            syne3['layer_post']=['L4-L6']
        
                    
        if syne3['cre_line_pre']=='sim1':
            syne3['layer_pre']=['L4-L5','L4-L6']
        
                    
        if syne3['cre_line_post']=='sim1':
            syne3['layer_post']=['L4-L5','L4-L6']
            
        if syne3['cre_line_post']=='vip':
            if syne3['layer_post'][0]=='L2/3':
                syne3['layer_post']=['L1-L2/3']
        if syne3['cre_line_pre']=='vip':
            if syne3['layer_pre'][0]=='L2/3':
                syne3['layer_pre']=['L1-L2/3']
                
        if syne3['cre_line_post']=='sst':
            if syne3['layer_post'][0]=='L2/3':
                syne3['layer_post']=['L1-L2/3']
        if syne3['cre_line_pre']=='sst':
            if syne3['layer_pre'][0]=='L2/3':
                syne3['layer_pre']=['L1-L2/3']        
 
    
    # # find sets of gene clusters corresponding to each pre- and post- neuron types pair from stp-table
    # for ii in range(len(dse)): 
    #    if sum(np.array(dse[ii])==pre_post.loc[i,0])>0:
    #        i1=ii
    #    if sum(np.array(dse[ii])==pre_post.loc[i,1])>0:
    #        j1=ii    
    #
    ## select names of stp-types
    #pree = dse[i1]
    #poste = dse[j1]
    
    ## select names of genes-types (ABA cluster)
    #preg = dsg[i1]
    #postg = dsg[j1]

    ##synij = (pre_post.loc[:,0].isin(pree))&(pre_post.loc[:,1].isin(poste))
    

    synij[:] = False
    
    if i<l_pre_post1:
        synij[i] = in_stp_data[i]
    else:
        synij[i] = True
    
    #print(syne3)
    
    if sum(synij)>0:
        print('\n\n')
        print('i = ',str(i))
        print(syne3)
        #if sum(in_stp_data&synij)>0:
        #    in_stp_data = in_stp_data&synij

        #add ge data
        #'layer_pre':l,  'cre_line_pre':'', 'cell_type2_pre':syne2[0].str.strip(),'cluster_pre':[]
        
        # PC or GABAergic dataset?
        batch = [ syne3['cell_type2_pre']=='PC',syne3['cell_type2_post']=='PC']
        
        # add ge data depending on available stp cell types labels:
        #  select presynaptic cells
        if batch[0]:

            if i>=l_pre_post1:
                in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin(syne3['layer_pre']) #modify!!!
                #in_preg  = in_preg&(df_aba_vis_l23glu_c.loc[:,'driver_lines']==syne3['cre_line_pre']) #modify!!!
                if syne3['cre_line_pre']!='none':
                    in_preg  = in_preg&(df_aba_vis_l23glu_c.loc[:,'driver_lines'].str.lower().str.contains(syne3['cre_line_pre'])) #modify!!!
                #set(df_aba_vis_l23glu_c.loc[:,'driver_lines'].str.lower().str.contains('nr5a1'))
            else:
                #in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']]) #modify!!!
                in_preg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']]) #modify!!!
        else:
            if len(syne3['cluster_pre'])==0:
                # if len(syne3['cre_line_pre'])>0:
                in_preg = df_aba_vis_in_c.loc[:,'driver_lines'].str.lower().str.contains(str.lower(syne3['cre_line_pre']))
                #in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                if i>=l_pre_post1:
                    in_preg = in_preg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin(syne3['layer_pre'])
                else:
                    in_preg = in_preg&(df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_pre']))
                    
                #if i==72:
                #    breakpoint()
                    
                    
            else:                  
                #in_preg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_pre']])
                in_preg = df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_pre'])
                cl = syne3['cluster_pre']
                #cols2 = ['name','perc']
                #cl2pre =pd.DataFrame([],columns= cols2)
                in_preg0 = in_preg.copy()
                in_preg[:] = False 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = clns[icls]
                        clns5 = re.split('\s+',clns4)
                        if len(clns5)<3:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2pre = cl2pre.append(cl22)
                    
                    in_preg = in_preg|(in_preg0&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3))
                    
                if len(syne3['cre_line_pre'])>0:
                    #in_preg = in_preg&df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_pre'])
                    in_line=df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_pre'])
                    if in_line.sum()>0:
                        in_preg = in_preg&in_line                        
        
        #  select postsynaptic cells
        if batch[1]:

            if i>=l_pre_post1:
                in_postg  = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin(syne3['layer_post']) #modify!!!
                if syne3['cre_line_post']!='none':
                    in_postg  = in_postg&(df_aba_vis_l23glu_c.loc[:,'driver_lines'].str.lower().str.contains(syne3['cre_line_post'])) #modify!!!
            else:
                #in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion'].isin([syne3['layer_post']]) #modify!!!
                in_postg = df_aba_vis_l23glu_c.loc[:,'brain_subregion']==syne3['layer_post'] #modify!!!
        else:    
            if len(syne3['cluster_post'])==0:
                # if len(syne3['cre_line_post'])>0:
                in_postg = df_aba_vis_in_c.loc[:,'driver_lines'].str.lower().str.contains(str.lower(syne3['cre_line_post']))
                #in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                if i>=l_pre_post1:
                    in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].isin(syne3['layer_post'])
                else:
                    in_postg = in_postg&df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_post'])
            else:    
                ## under construction ...
                #in_postg = df_aba_vis_in_c.loc[:,'brain_subregion'].isin([syne3['layer_post']])
                in_postg = df_aba_vis_in_c.loc[:,'brain_subregion'].str.contains(syne3['layer_post'])
                cl = syne3['cluster_post']
                #cols2 = ['name','perc']
                #cl2post =pd.DataFrame([],columns= cols2)
                in_postg0 = in_postg.copy()
                in_postg[:] = False                 
                for icl in range(len(cl)):
                    clp = cl[icl][1]
                    clns = re.split(',',cl[icl][0])
                    cls3 = []
                    cls3m=[]
                    for icls in range(len(clns)):
                        clns4 = clns[icls]
                        clns5 = re.split('\s+',clns4)
                        if len(clns5)<3:
                            clns6=clns5[0]
                            cls3=cls3 + list(set(df_aba_vis_in_c.loc[df_aba_vis_in_c.loc[:,'subclass'].str.contains(clns6),'cluster']))
                        else:
                            clns4=str.strip(clns4)
                            clns7=re.split('\-',clns4)
                            if len(clns7)>1:
                                clns4=clns7[0]
                                cls3m=cls3m+[str.strip(clns4)]
                            else:
                                cls3=cls3+[str.strip(clns4)]
                    cls3 = list(set(cls3).difference(set(cls3m)))
                    cl22=pd.DataFrame([cls3,[clp]],index=cols2).T
                    cl2post = cl2post.append(cl22)
                    
                    #in_postg = in_postg&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3)
                    in_postg = in_postg|(in_postg0&df_aba_vis_in_c.loc[:,'cluster'].isin(cls3))
                    
                if len(syne3['cre_line_post'])>0:
                    in_line=df_aba_vis_in_c.loc[:,'driver_lines'].str.contains(syne3['cre_line_post'])
                    if in_line.sum()>0:
                        in_postg = in_postg&in_line
            
            
        ## make equal size pre- and post- datasets
        #in_preg = in_preg&(in_preg.isna()==False)
        #in_postg = in_postg&(in_postg.isna()==False)
        #npre = in_preg.sum()
        #npost = in_postg.sum()
        ##nsample = max([npost, npre] )
        #nsample = min([npost, npre] ) # number of cells
        #print('npre = '+ str(npre)+' , npost = ',str(npost))
        #if (nsample<150)&(npre>0)&(npost>0):
        #    nsample=150
        
        #idx2 =  np.mod(np.arange(nsample),npre)
        #in_preg = np.nonzero(in_preg)[0][idx2]
        #idx2 =  np.mod(np.arange(nsample),npost)
        #in_postg = np.nonzero(in_postg)[0][idx2]
        
        
        # make equal size pre- and post- datasets
        in_preg = in_preg&(in_preg.isna()==False)
        in_postg = in_postg&(in_postg.isna()==False)
        npre = in_preg.sum()
        npost = in_postg.sum()
        nsample = max([npost, npre] )
        #nsample = min([npost, npre] ) # number of cells
        

        
   
            
        print('npre = '+ str(npre)+' , npost = ' + str(npost) + ' , nsample = '+str(nsample))    
        
        
        # select cell indices for gene expression sampling
        do_bootstrap=1
        if  do_bootstrap==0:
            
            if (nsample<60)&(npre>0)&(npost>0):
                nsample=60
            
            if (nsample>180)&(npre>0)&(npost>0):  # restrict nsample to avoid overrepresentation?
                nsample=180 
            
            idx2 =   np.mod(np.arange(nsample),npre) 
            in_preg = np.nonzero(in_preg)[0][idx2]
            idx2 =   np.mod(np.arange(nsample),npost) 
            in_postg = np.nonzero(in_postg)[0][idx2]
        else :
            if (npre>0)&(npost>0):
                nsample=200
            #random.uniform(low=0.0, high=1.0, size=None)
            idx2pre =  np.floor( np.random.uniform(0,npre, (nsample,1)) ).astype('int')
            in_preg_bs = np.nonzero(in_preg)[0][idx2pre]
            idx2post =  np.floor( np.random.uniform(0,npost, (nsample,1)) ).astype('int')
            in_postg_bs = np.nonzero(in_postg)[0][idx2post]
            in_preg_ = in_preg_bs[:,0]
            in_postg_ = in_postg_bs[:,0]
        
        

        if batch[0]:
            #in_preg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_preg,:].loc[:,'cluster']))
            in_preg_cl = df_aba_vis_l23glu_c.iloc[in_preg_,:].loc[:,'cluster'].value_counts()
        else:
            #in_preg_cl = list(set(df_aba_vis_in_c.iloc[in_preg,:].loc[:,'cluster']))
            in_preg_cl = df_aba_vis_in_c.iloc[in_preg_,:].loc[:,'cluster'].value_counts()
        
        if batch[1]:
            #in_postg_cl = list(set(df_aba_vis_l23glu_c.iloc[in_postg,:].loc[:,'cluster']))
            in_postg_cl = df_aba_vis_l23glu_c.iloc[in_postg_,:].loc[:,'cluster'].value_counts()
        else:        
            #in_postg_cl = list(set(df_aba_vis_in_c.iloc[in_postg,:].loc[:,'cluster']))
            in_postg_cl = df_aba_vis_in_c.iloc[in_postg_,:].loc[:,'cluster'].value_counts()
        in_preg_cl = in_preg_cl/in_preg_cl.sum()*100    
        in_postg_cl = in_postg_cl/in_postg_cl.sum()*100    
        print('presynaptic clusters')
        print(in_preg_cl)
        print('postsynaptic clusters')
        print(in_postg_cl)
        
        

        #if npost>npre:
        #    idx = np.mod(np.arange(npost),npre)
        #    #prege = prege[idx,:]
        #    #postge = postge
        #    prege = prege
        #    postge = postge[0:npre,:]            
        #elif npost<npre :
        #    idx = np.mod(np.arange(npre),npost)
        #    #prege = prege
        #    #postge = postge[idx,:]
        #    prege = prege[0:npost,:]
        #    postge = postge         
        
        # take ge data  
        
        prege = vis_dat.X[:, in_pregs]  # presynaptic gene set expression
        prege = prege[(vbi==batch[0]), :]
        
        postge = vis_dat.X[:,  in_postgs] # postsynaptic gene set expression
        postge = postge[(vbi==batch[1]),  :]
        
        do_median=1
        if  (do_bootstrap==0)|(do_median==0):
            prege = prege[in_preg,:]
            postge = postge[in_postg,:]
        else: 
            nsample = 100
            Nbs = nsample
            ##random.uniform(low=0.0, high=1.0, size=None)
            #nmax=5
            nmax = npre #np.min([1000,npre])
            idx2pre =  np.floor( np.random.uniform(0,npre, (nmax,Nbs)) ).astype('int')
            in_preg_ = np.nonzero(in_preg)[0][np.arange(npre)]
            nmax = npost #np.min([1000,npost])
            idx2post =  np.floor( np.random.uniform(0,npost, (nmax,Nbs)) ).astype('int')
            in_postg_ = np.nonzero(in_postg)[0][np.arange(npost)]

                        # 
            # ??? transform ge to median bootstraped log ???
            prege_ = prege[in_preg_,:]
            postge_ = postge[in_postg_,:]

            y = np.ma.masked_where(prege_ == 0, prege_)
            prege_=np.ma.log2(y).filled(0)
            y = np.ma.masked_where(postge_ == 0, postge_)
            postge_=np.ma.log2(y).filled(0)



            npregs=prege_.shape[1]
            ge_bs = np.zeros((Nbs,npregs+postge_.shape[1]))
            for ibs in range(Nbs):
                #ans=np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, x)
                #ans[np.isnan(ans)]=0.
                xbs = prege_[idx2pre[:,ibs],:]
                pregbs  = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                pregbs[np.isnan(pregbs)]=0.
                xbs = postge_[idx2post[:,ibs],:]
                postgbs = np.apply_along_axis(lambda v: np.median(v[v!=0]), 0, xbs)
                postgbs[np.isnan(postgbs)]=0.

                ge_bs[ibs,0:npregs] =pregbs
                ge_bs[ibs,npregs:]  =postgbs


            #df_hr2 = np.log2(df_hr2d)
            #df_hr2 = pd.concat([df_hr2,df_hr2s],axis=1) # log2 normalization
            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hr2 = np.log2(df_hr2)

            #df_hr2 = pd.concat([df_hr.iloc[1:,0],df_hr2],axis=1)

            #df_hr2.dtypes
            #df_hr2.head()

            #df_hr2 = df_hr2.replace({0:np.nan})
            #df_hrmg=df_hr2.groupby(['Class']) # logarithm_2 of UMI counts
            #df_hrmg=df_hrmg.median(numeric_only=True) # median of log2(UMI)

            #df_hrmg0 = df_hrmg.iloc[:,0:-1]

            #df_hrmsg = pd.concat([df_hrmg.iloc[:,-1]]*df_hrmg0.shape[1],axis=1)
            #df_hrmsg.columns = df_hrmg.iloc[:,0:-1].columns

            #df_hrmg = df_hrmg.iloc[:,0:-1] - df_hrmsg +6*np.log2(10)  # transform from log2( UMI counts) to log2(CPM)

            # ??? transform ge to median bootstraped log ???


              

        # add data to the final dataset    
        gnij = np.repeat(np.array([syne3['cell_type2_pre'], syne3['cell_type2_post'],
                                   syne3['layer_pre'], syne3['layer_post'],
                                   syne3['cre_line_pre'], syne3['cre_line_post']]).reshape((1,6)),nsample,axis=0 )
        gnij = pd.DataFrame(gnij, columns = ['cell_type2_pre', 'cell_type2_post',
                                             'layer_pre','layer_post',
                                             'cre_line_pre', 'cre_line_post'] )
        
        if  (do_bootstrap==0)|(do_median==0):
            prege  = pd.DataFrame(prege,columns  = pregs3 )
            postge = pd.DataFrame(postge,columns = postgs3 )


            ge_dataij = pd.concat([gnij, prege, postge ], axis = 1)
            
        else:
            ge_bs = pd.DataFrame(ge_bs,columns = pregs3 + postgs3 )


            ge_dataij = pd.concat([gnij, ge_bs ], axis = 1)
       

        
        
        
        # add stp data
        if i<l_pre_post1:
            stp_dataij =  df_stp.loc[[0,i], stp_type['stp_data_type']].T
            stp_columns = stp_TM_names
        else:
            #stp_dataij  = STP_aba.loc[i1, STP_aba_type['stp_data_type']]
            #stp_columns = np.array(stp_aba_names)
            #nbs = int(len(stp_dataij)/len(stp_columns))
            #stp_dataij  = pd.DataFrame(stp_dataij, index = np.repeat(stp_columns,nbs)).T
            ##i2 = np.trunc(np.arange(len(stp_dataij.columns))/len(stp_columns)).astype(int)
            #stp_dataij = stp_dataij.T.reset_index(drop=False)
            #stp_dataij.columns =[0,i]
            
            stp_dataij  = STP_aba.loc[i1, STP_aba_type['stp_data_type']]
            stp_columns = np.array(stp_aba_names)
            #nbs = int(len(stp_dataij)/len(stp_columns))
            nbs = int(stp_dataij.shape[0]) #/len(stp_columns))
            stp_dataij  = pd.DataFrame(stp_dataij.reshape((-1,)), index = np.tile(np.array(stp_aba_names),(nbs,))).T
            #i2 = np.trunc(np.arange(len(stp_dataij.columns))/len(stp_columns)).astype(int)
            stp_dataij = stp_dataij.T.reset_index(drop=False)
            stp_dataij.columns =[0,i]
            
        

        i2 = np.trunc(np.arange(len(stp_dataij.index))/len(stp_columns)).astype(int)
        stp_dataij  = stp_dataij.set_index(i2).pivot(columns=0,values=i).loc[:,stp_columns] #ss.set_index('par',append=True).unstack('par')
        #stp_dataij =  stp_dataij.T.pivot(index=0, values=synij).T
        
        idx = np.mod(np.arange(nsample),len(stp_dataij.index))
        stp_dataij=stp_dataij.iloc[idx,:].reset_index(drop=True)  
        
        i_data_set = pd.DataFrame(i+np.zeros(len(stp_dataij.index)), index = stp_dataij.index, columns = ['index_ds'])
        stp_dataij=pd.concat([stp_dataij, i_data_set],axis=1)
        
        #if do_averagig_stp:
        #    #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
        #    stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values

        
        #stp_dataij = pd.DataFrame({'presynaptic':pree[0], 'postsynaptic': poste[0], 
        #                           'stp_data':stp_dataij.iloc[0,:].values})
        #stp_data  = stp_data.append(stp_dataij) 
        
        # names of ge and stp columns

        
        ge_dataij  = pd.concat([ge_dataij,stp_dataij] , axis=1)
        
        # add to all ge and stp dataframe
        ge_data  = pd.concat([ge_data, ge_dataij] , axis=0)
        

        
#stp_data =  stp_data.reset_index(drop=True)  



#stp_data.loc[stp_data.loc[:,'postsynaptic']=='NGC','postsynaptic']='Lamp5'
#stp_data2 = ge_data.iloc[:,0:3].copy(deep=True)                                 # modify!
#stp_data2.columns=['presynaptic', 'postsynaptic', 'ppr']
#print(stp_data2.head())
#for i in stp_data.index: #stp_data.index:
#    pre=stp_data.loc[i,'presynaptic']
#    post=stp_data.loc[i,'postsynaptic']
#    stp_data2.loc[stp_data2.loc[:,'presynaptic'].isin([pre])&stp_data2.loc[:,'postsynaptic'].isin([post]),'ppr']=stp_data.loc[i,'stp_data'] 
    
ge_data = ge_data.reset_index()    

 
## transform labels from TM to An:A1
As = STP_sim_complex(ge_data,l_pre_post2,stp_aba_names)

N = As.shape[1]

As =  As[:,1:]/As[:,0].reshape((-1,1))

#As =  As[:,1:]/np.tile(As[:,0],(N-1,1)).T

## add new stp parameters to ge_data dataset
#stp_columns2 = ['A'+str(s)+'/A1' for s in range(1,N+1)]
stp_columns2 = ['A'+str(s)+'/A1' for s in range(2,N+1)]
stp_data2 = pd.DataFrame(As,columns = stp_columns2, index = ge_data.index)

ge_data = pd.concat([ge_data, stp_data2 ],axis=1)

if do_averagig_stp:
     #stp_dataij.loc[:,:] = stp_dataij.mean(axis=0).values
     #stp_dataij.loc[:,:] = stp_dataij.median(axis=0).values
    all_i_ds =  ge_data.loc[:,'index_ds']                             
    for i_ds in list(set(all_i_ds)):
              ids2 =  ge_data.loc[:,'index_ds'].isin([i_ds])  
              ge_data.loc[ids2,stp_columns2] = ge_data.loc[ids2,stp_columns2].median(axis=0).values                   
                                   
                                   
## filter TM parameters
do_filter_pars = 0

if do_filter_pars!=0:
    print((ge_data.loc[:,stp_columns ]<0).sum().sum())
    print(len(ge_data.index))

    nannot=6
    annot_columns = ge_columns[0:nannot]
    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f,ax = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax)
    plt.gca().legend('')
    #plt.gca().title('An:A1 before filtering data')
    ax.title.set_text('An:A1 before filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    print('mean of stp before filtering:')
    print(ge_data2)

    #i_filter_out = (ge_data.loc[:,'dp/p0']*ge_data.loc[:,'p0']+)|(ge_data.loc[:,'tD'])
    #i_filter_out = i_filter_out&((ge_data.loc[:,'tF'])|(ge_data.loc[:,'tF']))
    i_filter_out = np.sum(ps2>1,axis=1)|np.sum(ps2<0,axis=1)
    i_filter_out = i_filter_out|(np.sum(ns2>1,axis=1)|np.sum(ns2<0,axis=1))
    ge_data = ge_data.loc[i_filter_out==False ,:]

    ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()
    f1,ax1 = plt.subplots(figsize=(5, 5))
    ge_data2.T.plot(ax=ax1)
    plt.gca().legend('')
    #plt.gca().title('An:A1 after filtering data')
    ax1.title.set_text('An:A1 after filtering data')
    #ge_data2 = ge_data.loc[:,annot_columns + stp_columns2].groupby(annot_columns).mean()

    print('mean of stp after filtering:')
    print(ge_data2)
    
#print('mean of stp after filtering:')
#print(ge_data2)


print(len(ge_data.index))
#ge_data.loc[:,stp_columns ] =ge_data.loc[:,stp_columns ].abs()  


#
## normalize stp parameters - for TM parameters
do_normalize_stp_pars = 0;
if do_normalize_stp_pars!=0:
    ge_data.loc[:,stp_columns  ] =np.log(ge_data.loc[:,stp_columns  ].values.astype(float))

#  assign training and labels 
stp_columns1 = stp_columns 
stp_columns = stp_columns2 # select training labels names

nannot = 7
nstp =len(stp_columns)
annot_columns = ge_columns[0:6]+['index_ds']

## averaging of stp for each synapse type 


#X = ge_data.loc[:,ge_data.columns[0:]].values
X_c = ge_data.loc[:,annot_columns + ge_columns[6:]].values
y_c = ge_data.loc[:,annot_columns + stp_columns].values 
#y = stp_data2.loc[:,:].values

## normalize stp parameters - for TM parameters
#y[:,nannot: ] =np.log(y[:,nannot: ].astype(float))


    

# print elapsed time
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))

print(y_c.shape)
X_c.shape

In [ ]:
%matplotlib inline
# transform labels from TM to An:A1
fs = [20 , 10, 40] # Hz
N = 8

Trec = [250, 1000]
Trec=[]
DT0 = 25000
xs  =ge_data.iloc[l_pre_post2:,:].loc[:,stp_aba_names].values
xs  =np.delete(xs, [5,6],axis=1)
As=np.zeros((l_pre_post2,0))
As2=np.zeros((xs.shape[0],0))
if do_add_aba_synphys:
    
    for f in fs:
        T = np.arange(N)*1000/f
        
        Asf, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],T)
        As = np.concatenate([As,Asf],axis=1)
        
        for ri in range(len(Trec)):
            Asr, nsr, psr, dpp0, p0, tF, tD, A = STP_sim(ge_data.iloc[0:l_pre_post2,:],[Trec[ri]],init_state=[ns[:,-1],ps[:,-1]])
            As = np.concatenate([As,Asr],axis=1)
        

        As2f = np.zeros((xs.shape[0],N+len(Trec)))
        for i2 in range(xs.shape[0]):
            #as2, sts2 = STP_sim2(xs[i2,:],np.arange(8)*50,model_type = 'tm5') # preconditioning series???
            #as2, sts2 = STP_sim2(xs[i2,:], T+DT0, init_state=sts2[-1], model_type='tm5' ) 
            
            as2, sts2 = STP_sim2(xs[i2,:],T,model_type = 'tm5_smr')
            
            As2f[i2,0:N] = as2
            for ri in range(len(Trec)):
                as2r, sts2r = STP_sim2(xs[i2,:], [Trec[ri]], init_state=sts2[-1], model_type='tm5_smr' ) 
                As2f[i2,N+ri] = as2r
                
        As2 = np.concatenate([As2,As2f],axis=1)
            
    
    As2 = As2/As2[:,0].reshape((-1,1))
    As = As/As[:,0].reshape((-1,1))
    As = np.concatenate([As,As2],axis=0)
else:
    As, ns, ps, dpp0, p0, tF, tD, A = STP_sim(ge_data,T)
    

import matplotlib.pyplot as plt    
plt.plot(As2[0:100,:].transpose(),'o-')

##
##
##  combine cortex and hippocampus datasets 
##
##

In [ ]:
# ge_data_h = ge_data
# X_h = X
# y_h = y
# annot_columns_h = annot_columns
# ge_columns_h = ge_columns
# stp_columns_h = stp_colum

X = np.concatenate([X_c, X_h],axis=0)
y = np.concatenate([y_c, y_h],axis=0)

print(X_h.shape)
print(X_c.shape)
X.shape

In [ ]:
import time
t1 = time.time()
subclasses2_hipp = ['e_dg','e_ca','e_e','Pvalb_h','Sst_h','Vip','Lamp5_h','Cck_h']
subclasses3_hipp = [['DG','CA3','CA3 ventral'],
                    ['CA1','CA1 ventral'],
                    ['EC'],
                    ['PV','Pvalb','PVBC','AAC'],
                    ['Sst','HIPP','O-LM','O-Bi'],
                    ['Vip','IS3'],
                    ['Lamp5','Ivy cell','NGF'],
                    ['Cck','CCKBC','HICAP','CCK_DTI']]


ge_data_h_subcl = pd.DataFrame(np.zeros((ge_data_h.shape[0],4)),columns = ['name_long_pre','name_long_post',
                                                                           'subclass_pre','subclass_post'])
ge_data_h_subcl.index = ge_data_h.index
precol = ge_data_h.columns[[1,5]]
postcol = ge_data_h.columns[[2,6]]
for i in ge_data_h.index:
    all_type_1=''
    all_type_2=''
    for j in range(3):
        all_type_1 = all_type_1+ str(ge_data_h.loc[i,ge_data_h.columns[2*j+1]])+' '
    for j in range(3):
        all_type_2 = all_type_2+ str(ge_data_h.loc[i,ge_data_h.columns[2*j+2]])+' '  
    ge_data_h_subcl.loc[i,'name_long_pre'] = all_type_1
    ge_data_h_subcl.loc[i,'name_long_post'] = all_type_2
    
    ge_data_hi = ge_data_h.loc[i,:]
    for j in range(len(subclasses3_hipp)): 
        is_in_pre = ge_data_hi.loc[precol].isin(subclasses3_hipp[j]).sum()>0
        is_in_post = ge_data_hi.loc[postcol].isin(subclasses3_hipp[j]).sum()>0
        if is_in_pre:
            ge_data_h_subcl.loc[i,'subclass_pre'] =subclasses2_hipp[j]
        if is_in_post:
            ge_data_h_subcl.loc[i,'subclass_post'] =subclasses2_hipp[j]    
        
    #all_types = all_types  +[all_type]    

    
    
# all_types =  pd.DataFrame(all_types)   
# subclasses = [['Pvalb', 'eh'],
#               ['Pvalb', 'Pvalb'],
#               ['eh','Cck'],
#               ['ee','Lamp5'],
#               ['eh','eh'],
#               ['ee','Lamp5'],
#               ['Cck','eh'],
#               ['Pvalb','eh'],
#               ['Lamp5', 'Lamp5'],
#               ['eh','Pvalb'],
#               ['eh','eh'],
#               ['eh','eh'],
#               ['Lamp5','eh'],
#               ['Cck','eh'],
#               ['eh','Pvalb'],
#               ['Lamp5','eh'],
#               ['Pvalb','Pvalb'],
#               ['ee','Lamp5'],
#               ['ee','Lamp5'],
#               ['eh','eh'],
#               ['eh','Cck'],
#               ['ee','Cck'],
#               ['ee','eh'],
#               ['eh','Sst'],
#               ['Vip','Sst'],
#               ['ee','eh'],
#               ['ee','Pvalb'],
#               ['eh','Lamp5'],
#               ['eh','Lamp5'],
#               ['Cck','Cck'],
#               ['Pvalb','eh'],
#               ['eh', 'Sst']]

# #subclasses_hipp1 = pd.DataFrame(list(set(all_types)) )
# subclasses_hipp1 = pd.DataFrame(pd.unique(all_types.loc[:,0].values))
# subclasses =  pd.DataFrame(subclasses,columns=['pre', 'post'] )
# subclasses_hipp = pd.concat([subclasses_hipp1, subclasses], axis=1)


# ge_data_h_subcl = pd.DataFrame(np.tile(all_types,(1,4)),columns=['name_long_pre','name_long_post',
#                                                                  'subclass_pre','subclass_post'])
# for i in range(subclasses_hipp.shape[0]):

#     ge_data_h_subcl.loc[ge_data_h_subcl.loc[:,'name_long'].isin([subclasses_hipp.loc[i,0]]),'subclass_pre'] =subclasses_hipp.loc[i,'pre']
#     ge_data_h_subcl.loc[ge_data_h_subcl.loc[:,'name_long'].isin([subclasses_hipp.loc[i,0]]),'subclass_post'] =subclasses_hipp.loc[i,'post']
t2=time.time()
print('elapsed time : '+str(t2-t1))
ge_data_h_subcl    

In [ ]:
all_types = []
all_types2 = []
columns = ['cell_type2_pre','cell_type2_post','layer_pre','layer_post','cre_line_pre','cre_line_post']
for i in ge_data.index:
    all_type=''
    all_type2=''
    for j in range(3):
        sj = str(ge_data.loc[i,columns[2*j]])
        if sj=='MC':
            sj='Sst'    
        all_type = all_type  +' '+sj
        
    for j in range(3):
        sj = str(ge_data.loc[i,columns[2*j+1]])
        if sj=='MC':
            sj='Sst'    
        all_type2 = all_type2 +' '+sj    
    all_types = all_types  +[str.lower(all_type)]  
    all_types2 = all_types2  +[str.lower(all_type2)]   

all_types =  pd.DataFrame(all_types, columns=['pre'])
all_types2 =  pd.DataFrame(all_types2, columns=['post'])
all_types = pd.concat([all_types,all_types2], axis=1)

subclasses = pd.Series(['Sst','Pvalb','Vip','ev_l23','ev_l4','ev_l56','ev_l56'])
subclasses2 = pd.Series(['sst','pvalb','vip','l2',   'l4',   'l5',    'l6'])
subclasses2_visp =subclasses2   

subclasses_visp_1 = pd.DataFrame(pd.unique(all_types.loc[:,'pre'].values))
subclasses_visp_2 = pd.DataFrame(pd.unique(all_types.loc[:,'post'].values))
# subclasses =  pd.DataFrame(subclasses,columns=['pre', 'post'] )
# subclasses_visp = pd.concat([subclasses_visp1, subclasses], axis=1)

ge_data_c_subcl = pd.DataFrame(np.tile(all_types,(1,2)))
ge_data_c_subcl.columns=['name_long_pre','name_long_post','subclass_pre','subclass_post']
for i in range(subclasses_visp_1.shape[0]):
    type_name = str.lower(subclasses_visp_1.loc[i,0])
#     is_in=[]
#     for si in subclasses2:
#         is_in = is_in +[ss.contains(si)]
#     is_in=np.nonzero(np.array(is_in))[0][0] 
#     subcl_name = subclasses[is_in]
    
    for ii,si in enumerate(subclasses2):
        if si in type_name:
            subcl_name = subclasses[ii]
            break
    ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long_pre']==type_name,'subclass_pre'] =subcl_name
    
for i in range(subclasses_visp_2.shape[0]):
    type_name = str.lower(subclasses_visp_2.loc[i,0])
    for ii,si in enumerate(subclasses2):
        if si in type_name:
            subcl_name = subclasses[ii]
            break
    ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long_post']==type_name,'subclass_post'] =subcl_name    

#     ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long'].isin([subclasses_visp.loc[i,0]]),'subclass_pre'] =subclasses_visp.loc[i,'pre']
#     ge_data_c_subcl.loc[ge_data_c_subcl.loc[:,'name_long'].isin([subclasses_visp.loc[i,0]]),'subclass_post'] =subclasses_visp.loc[i,'post']

#ge_data_c_subcl    

In [ ]:
ge_data_subcl = pd.concat([ge_data_c_subcl, ge_data_h_subcl],sort=False)
subclasses2 =pd.concat([subclasses, pd.Series(subclasses2_hipp)])
ge_data_subcl



# TRAIN MODEL

##
##
##  train RF 
##
##

In [ ]:
subclasses2

In [ ]:
classes = ['all','ex', 'inh', 'ex_ctx', 'ex_hipp','ex_ec','mge','cge','pvalb','vip','sst','cck','lamp5','ex_he',
          'ex_l23','ex_l6','ex_l4','ca','dg','pvalb_h','pvalb_c','sst_h','sst_c']
y_features = pd.DataFrame([['A2/A1_20Hz', 'A5/A1_20Hz', 'A2/A1_50Hz','A5/A1_50Hz','A250/A1','A1000/A1'],[0,4,8,11,5,6]],index=['names','index_y'])
print("y features : \n", y_features,"\n\n")
subclasses2 = pd.unique(subclasses2 )
print("subclasses : \n",subclasses2,"\n\n") 

cl_in_subcl = np.array([[0,1,2,3,4,5,6,7,8,9,10,11,12],[3,4,5,6,7,8],
                        [0,1,2,9,10,11,12],[3,4,5],[6,7],[8],[1,9,0,10],
                        [2,11,12],[1,9],[2],[0,10],[12],[11],[6,7,8],[3],[4],[5],[7],[6],[9],[1],[10],[0]])
classes = pd.DataFrame([classes, cl_in_subcl],index=['classes','subclasses'])
classes = classes.T.set_index('classes').T
classes

In [ ]:
def make_data(X,y,data_subclasses,subclasses):
    #X_train = np.array([]).reshape((0,X.shape[1]))
    #y_train = np.array([]).reshape((0,y.shape[1]))

    in_subcl = (data_subclasses.loc[:,'subclass_post']!=False)
    for i0 in range(len(subclasses[0])):
        in_subcl = in_subcl|(data_subclasses.loc[:,'subclass_pre']==subclasses[0][i0])
        
    for i0 in range(len(subclasses[1])):        
        #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_post']==subclasses2[sbi[i0]])
        in_subcl = in_subcl|(data_subclasses.loc[:,'subclass_post']==subclasses[1][i0])
        
    in_subcl = np.nonzero(in_subcl.values)[0]
    #X_train = np.concatenate([X_train,X[in_subcl]])   #np.delete(X, in_subcl,axis=0)
    #y_train = np.concatenate([y_train,y[in_subcl]])   #np.delete(y, in_subcl,axis=0)
    X_train = np.copy(X[in_subcl])   #np.delete(X, in_subcl,axis=0)
    y_train = np.copy(y[in_subcl])   #np.delete(y, in_subcl,axis=0)
    return X_train, y_train    

In [ ]:
modf = pd.read_excel('synapses_types_tree.xlsx')
modf

In [ ]:
stp_columns3 = []
stp_columns0 = ['A1',
                'A2',
                'A3',
                'A4',
                'A5']
stpn_fr=[s+'_20Hz' for s in stp_columns0[1:]]
stp_columns3 = stp_columns3 +stpn_fr+['A250_20Hz', 'A1000_20Hz']

stpn_fr=[s+'_50Hz' for s in stp_columns0[0:]]
stp_columns3 = stp_columns3 +stpn_fr+['A250_50Hz', 'A1000_50Hz']

stpn_fr=[s+'_10Hz' for s in stp_columns0[0:]]
stp_columns3 = stp_columns3 +stpn_fr+['A250_10Hz', 'A1000_10Hz']
stp_columns3

X_train = pd.DataFrame(np.copy(X),columns = annot_columns+ge_columns[6:])   #np.delete(X, in_subcl,axis=0)
y_train = pd.DataFrame(np.copy(y),columns = annot_columns+stp_columns3)     #np.delete(X, in_subcl,axis=0)

X_train = pd.concat([X_train,y_train.loc[:,stp_columns3]],axis=1)

#X_train 

#classes 	all 	ex 	inh 	ex_ctx 	ex_hipp 	ex_ec 	mge 	cge 	pvalb 	vip 	sst 	cck 	lamp5

modf = pd.read_excel('synapses_types_tree.xlsx')
modf2 = pd.read_excel('synapses_types_tree.xlsx',sheet_name='Sheet2')
mod =[ {'name': 'all',    'classes':['all'],                                   'classes':'pre'},
       {'name': 'ex_inh', 'classes':['ex','inh'],                              'side':'pre'},
       {'name': 'ex_inh_post', 'classes':['ex','inh'],                         'side':'post'},
       {'name': 'ex2_inh', 'classes':['ex_ctx','ex_ec','ex_hipp','inh'],       'side':'pre'},
       {'name': 'ex_inh2', 'classes':['ex','mge','cge'],                       'side':'pre'},
       {'name': 'ex_inh', 'classes':['ex','inh'],                              'side':'pre'},
       {'name': 'ex2_inh', 'classes':['ex_ctx','ex_ec','ex_hipp','inh'],       'side':'pre'},
       {'name': 'ex_inh3', 'classes':['ex','pvalb','sst','cge'],               'side':'pre'},
       {'name': 'ex_inh4', 'classes':['ex','pvalb','sst','vip','cck','lamp5'], 'side':'pre'},
     ]

subcl_pre=ge_data_subcl.loc[:,'subclass_pre'].copy()
subcl_pre.index=X_train.index

subcl_post=ge_data_subcl.loc[:,'subclass_post'].copy()
subcl_post.index=X_train.index

# classes_columns = []
# for md in mod:
#     in_subcl = subcl_pre.copy()
#     for icl, cl in enumerate(md['classes']):
#         #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
#         subcl = subclasses2[classes.loc['subclasses',cl]]
#         in_subcl.loc[subcl_pre.isin(subcl)] = icl #icl
#     in_subcl.name =  md['name']  
#     classes_columns = classes_columns + [in_subcl.name]
#     X_train = pd.concat([X_train,in_subcl],axis=1)    
    
classes_columns = []
for i in modf.index:
    md = modf.loc[i,:]
    in_subcl = subcl_pre.copy()
    mdcl = np.char.strip(np.array(str.split(md['classes'],',')))
    for icl, cl in enumerate(mdcl):
        #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
        subcl = subclasses2[classes.loc['subclasses',cl]]
        in_subcl.loc[subcl_pre.isin(subcl)] = icl #icl
    in_subcl.name =  md['name']+'__pre'  
    classes_columns = classes_columns + [in_subcl.name]
    X_train = pd.concat([X_train,in_subcl],axis=1)  
    
    in_subcl = subcl_post.copy()
    mdcl = np.char.strip(np.array(str.split(md['classes_post'],',')))
    for icl, cl in enumerate(mdcl):
        #in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_pre'].isin(classes.loc['subclasses']))
        subcl = subclasses2[classes.loc['subclasses',cl]]
        in_subcl.loc[subcl_post.isin(subcl)] = icl #icl
    in_subcl.name =  md['name']+'__post'  
    classes_columns = classes_columns + [in_subcl.name]
    X_train = pd.concat([X_train,in_subcl],axis=1)
        
         
#for i in
#mod1_ = mod1_ +  [ sm.OLS( y_train, X_train) ]
#res = mod.fit()
#print(res.summary())
ge_columns_train      = ge_columns[6:]
annot_columns_train   = annot_columns
stp_columns_train     = stp_columns3
classes_columns_train = classes_columns
print(X_train.columns)
X_train

In [ ]:
classes

In [ ]:
columns  =[
[ 'ge_columns_train',
  'annot_columns_train',
  'stp_columns_train',
  'classes_columns_train',],
[ ge_columns_train,
  annot_columns_train,
  stp_columns_train,
  classes_columns_train,
]]
columns = pd.DataFrame(columns)
columns=columns.T.set_index([0]).T
columns

In [ ]:
columns.to_hdf('stp_to_ge_all_columns.hdf',key='data')
X_train.to_hdf('stp_to_ge_all_data.hdf',key='data')

# START HERE

In [ ]:
import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data.hdf')

In [ ]:
modf

In [ ]:
import sklearn as sk

In [ ]:
## Fit cell types hierarhy tree with linear models in leaves

In [ ]:
def fit_classes_tree(X2,y2,X2_cl,cli,model_type='lasso',nmin = 20, alpha=1, l1_ratio=0.5):
    model = []
    support =[]
    ncl1 = np.max(X2_cl[:,cli[0]])+1 
    ncl2 = np.max(X2_cl[:,cli[1]])+1 
    for i1 in range(ncl1):
        model1 = []
        support1 = []
        for i2 in range(ncl2):

            is_in_cl = (X2_cl[:,cli[0]]==i1)&(X2_cl[:,cli[1]]==i2)
            #           support2 = [np.sum(is_in_cl),
            #           np.nonzero((X2_cl[:,cli[0]]==i1))[0],
            #           np.nonzero((X2_cl[:,cli[1]]==i2))[0]]
            support2 = np.sum(is_in_cl)
            
            support1 = support1 + [support2]
            regr = 0
            if support2>nmin:
                y_train = y2[is_in_cl,:]
                X_train = X2[is_in_cl,:]
                if model_type=='ridge':
                    regr = sk.linear_model.Ridge(alpha=0.50, fit_intercept=True, normalize=False,
                                                 copy_X=True, max_iter=None, tol=0.001, solver='auto',
                                                 random_state=None)
                
                elif model_type=='lasso':
                    regr = sk.linear_model.Lasso(alpha=alpha, fit_intercept=True, normalize=False,
                                                 precompute=False, copy_X=True,
                                                 max_iter=1000, tol=0.0001, warm_start=False, positive=False, 
                                                 random_state=None, selection='cyclic')
                elif model_type=='elastic_net':   
                    regr = sk.linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True,
                                    normalize=False,
                                    max_iter=1000, copy_X=True, tol=0.0001, warm_start=False,
                                    random_state=None, selection='cyclic')
                else:
                    print(" model types supported: ridge, lasso, elastic_net")
                
                regr.fit(X_train, y_train)
                #y_pred = regr.predict(X_test[:,nannot:]) 


            model1 = model1 + [regr]
            
        support = support + [np.array(support1)]
        model = model + [np.array(model1)]    
    return np.array(model), np.array(support)        

In [ ]:
def predict_classes_tree(model,X2,X2_cl,cli,nout=1,nmin = 20):
    
    ncl1 = np.max(X2_cl[:,cli[0]])+1 
    ncl2 = np.max(X2_cl[:,cli[1]])+1 
    y_pred = np.zeros((X2.shape[0],nout))
    for i1 in range(ncl1):
        #print(i1," f1")
        if len(model)>i1:
            #print(i1," f2")
            model1 = model[i1]
            for i2 in range(ncl2):

                is_in_cl = (X2_cl[:,cli[0]]==i1)&(X2_cl[:,cli[1]]==i2)

                support2 = np.sum(is_in_cl)
                
                #print(i1,i2," f3 ",support2)
                #regr = 0
                if support2>nmin:
                    #print(i1,i2," f3 ",support2)
                    if len(model1)>i2:
                        #print(i1,i2," f4")
                        regr = model1[i2]
                        #regr = sk.linear_model.MultiTaskElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=True,
                        #                    normalize=False,
                        #                    max_iter=1000, copy_X=True, tol=0.0001, warm_start=False,
                        #                    random_state=None, selection='cyclic')
                        #regr.fit(X_train, y_train)
                        if regr!=0:
                            X_test = X2[is_in_cl,:]
                            y_pred[is_in_cl] = regr.predict(X_test).reshape((-1,1)) 
    
    return y_pred       

In [ ]:
## Draw some interesting plots

In [ ]:
# Load important genes from iRF search

%matplotlib inline
import matplotlib.pyplot as plt
imp  = np.load('importance.npy')
imp = pd.DataFrame(imp)
imp = pd.concat([imp,pd.DataFrame(ge_columns[6:])],axis=1)
imp.columns = ['importance', 'genes']
imp = imp.set_index('genes')

imp0  = np.load('importance0.npy')
imp0 = pd.DataFrame(imp0)
imp0 = pd.concat([imp0,pd.DataFrame(ge_columns[6:])],axis=1)
imp0.columns = ['importance0', 'genes']
imp0 = imp0.set_index('genes')


# List of important genes : iRF
imp = pd.concat([imp,imp0],axis=1)

imp50 = imp.sort_values('importance',ascending=False).iloc[0:49,:]
plt.plot(imp.sort_values('importance',ascending=False).loc[:,['importance','importance0']].values)
imp50

In [ ]:
import matplotlib.pyplot as plt
import time
t1=time.time()

Dn = 2
cla_n = classes_columns_train #['ex_inh']
ge_n = imp50.index[0:50].tolist() #ge_columns_train #imp50.index[0:25].tolist()
X2 = X_train0.loc[:,ge_n ]
X2_cl = X_train0.loc[:,cla_n ]
#i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
X2_cl=X2_cl.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
#  'A2_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A2_50Hz',
#  'A2_10Hz',
#  'A5_50Hz',
#  'A5_10Hz',
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
##header = ge_n + cla_n + stp_n 
#header  = ge_n  + stp_n 

iy0=0
mod_index = modf.index[[13]] #[3,9,13,16] [3] # modf.index #[0, 3, 9, 11, 13, 15]

for ig,g in enumerate(ge_n):
    #f, ax =plt.subplots(figsize=(16, 10))
    ##f, ax = plt.figure()
    ##ax = f.add_axes()
    
    #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
    #plt.title(stp_columns[iy]+", model : "+mdn)
    
    #ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
    #yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    #plt.plot(y2.ravel(),'ob')
    #plt.plot(y_pred.ravel(),'xr')
    for i in mod_index:
        md = modf.loc[i,:]


        mdn = md['name']
        #print(mdn)

        cla_n = np.array(classes_columns_train)

        cli = np.nonzero(cla_n==mdn+'__pre')[0] + np.arange(2)

        mdcl2 = np.char.strip(np.array(str.split(md['classes_post'],',')))
        mdcl1 = np.char.strip(np.array(str.split(md['classes'],','))) 
        #print(mdcl1)
        #print(mdcl2)
        
        f, ax =plt.subplots(figsize=(16, 10))
        ##f, ax = plt.figure()
        ##ax = f.add_axes()
    
        #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
        plt.title(g+", model : "+mdn+" classes pre : "+str(mdcl1)+" classes post : "+str(mdcl2))
        
        
        plt.scatter(X2[:,ig],y2.ravel(),marker='o',c=X2_cl[:,cli[0]],s=10)
        plt.scatter(X2[:,ig],y2.ravel(),marker='o',c=X2_cl[:,cli[1]],s=70,alpha=0.2)

In [ ]:
## Fit set of linear models for each level of cell types hierarhy

In [ ]:
import matplotlib.pyplot as plt
import time
t1=time.time()

Dn = 10
cla_n = classes_columns_train #['ex_inh']
ge_n = ge_columns_train #imp50.index[0:25].tolist()
X2 = X_train0.loc[:,ge_n ]
X2_cl = X_train0.loc[:,cla_n ]
#i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
X2_cl=X2_cl.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
#  'A2_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A2_50Hz',
#  'A2_10Hz',
#  'A5_50Hz',
#  'A5_10Hz',
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
##header = ge_n + cla_n + stp_n 
#header  = ge_n  + stp_n 

iy0=0
mod_index = modf.index #[3] # modf.index #[0, 3, 9, 11, 13, 15]
Y_pred = []
for i in mod_index:
    md = modf.loc[i,:]


    mdn = md['name']
    print(mdn)

    cla_n = np.array(classes_columns_train)

    cli = np.nonzero(cla_n==mdn+'__pre')[0] + np.arange(2)

    mdcl2 = np.char.strip(np.array(str.split(md['classes_post'],',')))
    mdcl1 = np.char.strip(np.array(str.split(md['classes'],','))) 
    print(mdcl1)
    print(mdcl2)

    y_pred = np.zeros((0,y2.shape[1]))
    
    ncv = 10
    n_samp_cv = np.rint(X2.shape[0]/ncv)
    samples_all = np.arange(X2.shape[0])
    r2cv = np.zeros(ncv)
    r4cv = np.zeros(ncv)
    for icv in range(ncv): #range(ncv):
        samples_test = (np.arange(n_samp_cv) + icv*n_samp_cv).astype(int)
        samples_train = np.delete(np.copy(samples_all),samples_test)
        X2train, y2train, X2train_cl = X2[samples_train,:], y2[samples_train,:], X2_cl[samples_train,:]
        X2test, y2test, X2test_cl = X2[samples_test,:], y2[samples_test,:], X2_cl[samples_test,:]
        
        model, support = fit_classes_tree(X2train,y2train,X2train_cl,cli,model_type='elastic_net',
                                          nmin = 20, alpha=0.2, l1_ratio=0.05)
        #model, support = fit_classes_tree(X2,y2,X2_cl,cli,model_type='ridge',nmin = 20, alpha=1, l1_ratio=0.5)
        y_pred_i = predict_classes_tree(model,X2test,X2test_cl,cli,nout=y2.shape[1],nmin = 2)
        y_pred = np.concatenate([y_pred, y_pred_i],axis=0)
        
        iy=iy0
        if np.var(y2[:,iy])!=0:
            nonz = y_pred_i[:,iy]!=0
            if np.sum(nonz)>0:
                r2cv[icv]=1 - np.mean((y2test[nonz,iy] - y_pred_i[nonz,iy])**2)/np.var(y2[:,iy])
                r4cv[icv]=1 - np.mean((y2test[nonz,iy] - y_pred_i[nonz,iy])**2)/np.var(y2test[nonz,iy])
    
    Y_pred = Y_pred + [y_pred]
    
    f, ax =plt.subplots(figsize=(16, 10))
    ##f, ax = plt.figure()
    ##ax = f.add_axes()
    
    #plt.title(stp_columns[iy]+", model : "+mdn+", cv : "+str(icv))
    plt.title(stp_columns[iy]+", model : "+mdn)
    
    #ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
    #yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.plot(y2.ravel(),'ob')
    plt.plot(y_pred.ravel(),'xr')
    #plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    iy=iy0
    r2=0
    r4=0
    if np.var(y2[:,iy])!=0:
        r2=1 - np.mean((y2[:,iy] - y_pred[:,iy])**2)/np.var(y2[:,iy])
        r4=1 - np.var(y_pred[:,iy])/np.var(y2[:,iy])
        
    print("R**2 = ",r2,"  Part of Var = ",r4,"\n",
          "  R**2 cv mean = ", np.mean(r2cv),"\n  R**2 cv unnormed mean = ", np.mean(r4cv),
          "\n  R**2 cv = ", r2cv,"\n  R**2 cv unnormed = ", r4cv,"\n\n")
    
    
t2 =time.time()    
print("Elapsed time ",t2-t1)

In [ ]:
plt.plot(y_pred_i,'ro')

In [ ]:
# F-test of regression models with and without subclass splitting 

import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.graphics.api import interaction_plot, abline_plot
from statsmodels.stats.anova import anova_lm

formula = "A2_20Hz ~ C(ex_in)  +  pre__Cplx1"
lm = ols(formula, data=X_train0[0:X_train0.shape[0]:10]).fit()
#print(lm.summary())

In [ ]:
ii=np.nonzero(np.array(ge_columns)=='pre__Cplx1')[0]
ge_columns[ii[0]]

In [ ]:
np.array(ge_columns)

In [ ]:
print(lm.__doc__)

In [ ]:
# Load important genes from iRF search

%matplotlib inline
import matplotlib.pyplot as plt
imp  = np.load('importance.npy')
imp = pd.DataFrame(imp)
imp = pd.concat([imp,pd.DataFrame(ge_columns[6:])],axis=1)
imp.columns = ['importance', 'genes']
imp = imp.set_index('genes')

imp0  = np.load('importance0.npy')
imp0 = pd.DataFrame(imp0)
imp0 = pd.concat([imp0,pd.DataFrame(ge_columns[6:])],axis=1)
imp0.columns = ['importance0', 'genes']
imp0 = imp0.set_index('genes')


# List of important genes : iRF
imp = pd.concat([imp,imp0],axis=1)

imp50 = imp.sort_values('importance',ascending=False).iloc[0:49,:]
plt.plot(imp.sort_values('importance',ascending=False).loc[:,['importance','importance0']].values)
imp50

In [ ]:
imp50.index[0:15].tolist()

In [ ]:
#import pixiedust

In [ ]:
#%%pixie_debugger

import time

import numpy as np
import pandas as pd


d_mtree = "/home/stepaniu/LearningX/advanced_ML/model_tree"

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, d_mtree)

#from models.linear_regr import linear_regr
import models.linear_regr as lir

from models.svm_regr import svm_regr
from models.DT_sklearn_regr import DT_sklearn_regr

from models.modal_clf import modal_clf
from models.DT_sklearn_clf import DT_sklearn_clf

import os, pickle, csv

import src.ModelTree as mtree
from src.utils import load_csv_data, cross_validate


from importlib import reload
mtree = reload(mtree)
lir=reload(lir)



# ====================
# Settings
# ====================
mode = "regr"  # "clf" / "regr"
save_model_tree = True  # save model tree?
save_model_tree_predictions = True  # save model tree predictions/explanations?
cross_validation = True  # cross-validate model tree?

# ====================
# Load data
# ====================
#data_csv_data_filename = os.path.join(d_mtree,"data", "data_clf.csv")
#X, y, header = load_csv_data(data_csv_data_filename, mode=mode, verbose=True)

#X2 = np.copy(X[range(0,X.shape[0],10)])
#y2 = np.copy(y[range(0,X.shape[0],10)])
#nsel = np.arange(1)
#y2=y2[:,0:nannot+nsel.shape[0]]


# add unspliting class
#cl1 = np.zeros((X.shape[0],1))
#cl1[0:200]=1
#cl1[1000:1200]=2


#X2 = np.concatenate([X2,cl1],axis=1)
#i_cl = [X2.shape[1]-1]

# ['all', 'ex_inh']
Dn = 10
cla_n = ['ex_inh']
ge_n = imp50.index[0:25].tolist()
X2 = X_train0.loc[:,ge_n + cla_n]
i_cl = np.nonzero(X2.columns.isin(cla_n))[0]
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
# 'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 

t1=time.time()

# Choose model
model = lir.linear_regr()
model.lambda_classes  = 0.1
model.class_index = i_cl

# Build model tree
model_tree = mtree.ModelTree(model, max_depth=4, min_samples_leaf=10,
                       search_type="greedy", n_search_grid=100)

# ====================
# Train model tree
# ====================
print("Training model tree with '{}'...".format(model.__class__.__name__))
model_tree.fit(X2, y2, verbose=True)

t2=time.time() 

y2_pred = model_tree.predict(X2)

t3=time.time() 

explanations = model_tree.explain(X2, header)
loss = model_tree.loss(X2, y2, y2_pred)
print(" -> loss_train={:.6f}\n".format(loss))
model_tree.export_graphviz(os.path.join("output", "model_tree"), header,
                           export_png=True, export_pdf=False)

print("elapsed time : ",[t2-t1,t3-t2])

# ====================
# Save model tree results
# ====================
if save_model_tree:
    model_tree_filename = os.path.join("output", "model_tree.p")
    print("Saving model tree to '{}'...".format(model_tree_filename))
    pickle.dump(model, open(model_tree_filename, 'wb'))

if save_model_tree_predictions:
    predictions_csv_filename = os.path.join("output", "model_tree_pred.csv")
    print("Saving mode tree predictions to '{}'".format(predictions_csv_filename))
    with open(predictions_csv_filename, "w") as f:
        writer = csv.writer(f)
        field_names = ["x2", "y2", "y2_pred", "explanation"]
        writer.writerow(field_names)
        for (x_i, y_i, y_pred_i, exp_i) in zip(X2, y2, y2_pred, explanations):
            field_values = [x_i, y_i, y_pred_i, exp_i]
            writer.writerow(field_values)

# ====================
# Cross-validate model tree
# ====================
cross_validation = 0
if cross_validation:
    t5 = time.time()
    cross_validate(model_tree, X2, y2, kfold=5, seed=1)
    t6 = time.time()
    print("time for cv : ", t6-t5)

In [ ]:
cross_validation = 0
if cross_validation:
    t5 = time.time()
    cross_validate(model_tree, X2, y2, kfold=5, seed=1)
    t6 = time.time()
    print("time for cv : ", t6-t5)

In [ ]:
np.histogram([1,1,2,2,2,3,3,4,9])

## Crossvalidation 

In [ ]:
import pandas as pd
import numpy as np
columns  = pd.read_hdf('stp_to_ge_all_columns.hdf')
ge_columns_train      = columns.loc[:,'ge_columns_train'].values[0]
annot_columns_train   = columns.loc[:,'annot_columns_train'].values[0]
stp_columns_train     = columns.loc[:,'stp_columns_train'].values[0]
classes_columns_train = columns.loc[:,'classes_columns_train'].values[0]
#print(X_train.columns)
X_train0 = pd.read_hdf('stp_to_ge_all_data.hdf')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import sklearn as sk

import time
t1 = time.time()

do_oob_subclasses=0

#X2 = np.copy(X[range(0,X.shape[0],1)])
#y2 = np.copy(y[range(0,X.shape[0],1)])
#nsel = np.arange(1)
#y2=y2[:,0:nannot+nsel.shape[0]]


from importlib import reload
mtree = reload(mtree)
lir=reload(lir)



# ====================
# Settings
# ====================
mode = "regr"  # "clf" / "regr"
save_model_tree = True  # save model tree?
save_model_tree_predictions = True  # save model tree predictions/explanations?
cross_validation = True  # cross-validate model tree?

# ====================
# Load data
# ====================

# ['all', 'ex_inh']
Dn = 10
cla_n = ['ex_inh'] #,'ex1_inh1']
ge_n = imp50.index[0:5].tolist()
X2 = X_train0.loc[:,annot_columns_train+ge_n + cla_n]
i_cl = np.nonzero(X2.columns.isin(cla_n))[0] - nannot
X2=X2.iloc[0:X_train0.shape[0]:Dn,:].values
stp_n = ['A2_20Hz']
# 'A2_20Hz',
#  'A3_20Hz',
#  'A4_20Hz',
#  'A5_20Hz',
#  'A250_20Hz',
#  'A1000_20Hz',
#  'A1_50Hz',
#  'A2_50Hz',
#  'A3_50Hz',
#  'A4_50Hz',
#  'A5_50Hz',
#  'A250_50Hz',
#  'A1000_50Hz',
#  'A1_10Hz',
#  'A2_10Hz',
#  'A3_10Hz',
#  'A4_10Hz',
#  'A5_10Hz',
#  'A250_10Hz',
#  'A1000_10Hz']
y2 =  X_train0.loc[:,annot_columns_train+stp_n].iloc[0:X_train0.shape[0]:Dn,:].values
#header = ge_n + cla_n + stp_n 
header = ge_n  + stp_n 
annot_columns2 = annot_columns
 

if do_oob_subclasses==0:
    ## select datasets for training and validation
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

    #X, y = make_regression(n_features=4, n_informative=2,
    #                        random_state=0, shuffle=False)

    kf = KFold(n_splits=10, shuffle=False)
    i0=0
    r6=[]
    r7=[]
    for train_index, test_index in kf.split(X2):
        i0=i0+1
        print('\n\n '+ str(i0))
        #print("TRAIN:", train_index,"\n", "TEST:", test_index)
        
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 


        met=4
        
        if met==1:
            regr = RandomForestRegressor(random_state=2026,max_depth=8,min_samples_leaf=1,
                                    n_estimators=2000, oob_score=True, n_jobs=-1, max_samples=500)
                    # RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
            #           max_features='auto', max_leaf_nodes=None,
            #           min_impurity_decrease=0.0, min_impurity_split=None,
            #           min_samples_leaf=1, min_samples_split=2,
            #           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            #           oob_score=False, random_state=0, verbose=0, warm_start=False)
        elif met==2:
            if i0==1:
                regr = sk.linear_model.MultiTaskElasticNetCV(cv=5,n_jobs=-1)
                regr.fit(X[:,nannot:], y[:,nannot:])
                print("alpha ",regr.alpha_)
                print("l1_ratio ",regr.l1_ratio_)
        
            regr = sk.linear_model.MultiTaskElasticNet(alpha=regr.alpha_, l1_ratio=regr.l1_ratio_,
                                                       fit_intercept=True,
                                                       normalize=False,
                                 max_iter=1000, copy_X=True, tol=0.0001,
                                                       warm_start=False, positive=False, 
                                                       random_state=None, selection='cyclic')
        
        elif met==3:
            regr = sk.svm.SVR(kernel='rbf', degree=3, gamma='scale',
                    coef0=0.0, tol=0.001, C=3.0, epsilon=0.1, shrinking=True, 
                              cache_size=200, verbose=False, max_iter=-1)
        
        
        elif met==4:
            # Choose model
            model = lir.linear_regr()
            model.lambda_classes  = 0.1
            model.class_index = i_cl

            # Build model tree
            model_tree = mtree.ModelTree(model, max_depth=4, min_samples_leaf=10,
                                   search_type="adaptive", n_search_grid=20)


        if met==3:
            #regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
            regr.fit(X_train[:,nannot:], y_train[:,nannot:].ravel())  
            y_pred = regr.predict(X_test[:,nannot:]) 
        elif met!=4:
            regr.fit(X_train[:,nannot:], y_train[:,nannot:])
            y_pred = regr.predict(X_test[:,nannot:]) 
        elif met==4:
            # ====================
            # Train model tree
            # ====================
            print("Training model tree with '{}'...".format(model.__class__.__name__))
            model_tree.fit(X_train[:,nannot:], y_train[:,nannot:], verbose=True)
            y_pred = model_tree.predict(X_test[:,nannot:])


        
        if (met==3)|(met==1)|(met==4):
            y_pred = y_pred.reshape((-1,1))
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test[0:len(nsel)] +stpn_pred[0:len(nsel)])
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,2,4])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y_test2[:,i])
                r4=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y2[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+str(i0))
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        
        if met==1:
            print('out of bag R^2: '+str(regr.oob_score_))

    rr = np.array([r6,r7])
    rr = pd.DataFrame(rr)


    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))



In [ ]:
import matplotlib

%matplotlib widget

import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = (20,10)
#plt.plot([1,2,3])

A=plt.imread('output/model_tree.png')
fig, ax = plt.subplots() #(figsize=(5,10))
im = ax.imshow(A)
plt.show()

In [ ]:
# 
sb0 = np.arange(len(classes))
mods  = []
mods2  = []
for i in range(classes.shape[1]):
    sbi = classes.loc['subclasses',i]
    print(sbi)
        
    subclasses3 = subclasses2[sbi]
    X_train, y_train =  make_data(X,y,ge_data_subcl,[subclasses3,[]])   
    regr = sk.linear_model.MultiTaskElasticNet(alpha=0.1, l1_ratio=0.5, fit_intercept=True,
                                                   normalize=False,max_iter=1000, copy_X=True, tol=0.0001, 
                                        warm_start=False, random_state=None, selection='cyclic')
    regr.fit(X_train[:,nannot:], y_train[:,nannot:])
    
    
    X_train2, y_train2 =  make_data(X,y,ge_data_subcl,[[],subclasses3]) 
    regr2 = sk.linear_model.MultiTaskElasticNet(alpha=0.1, l1_ratio=0.5, fit_intercept=True,
                                                   normalize=False,max_iter=1000, copy_X=True, tol=0.0001, 
                                        warm_start=False, random_state=None, selection='cyclic')
    regr2.fit(X_train2[:,nannot:], y_train2[:,nannot:])
    
    mods = mods + [regr]
    mods2 = mods2 + [regr2]
    #X_test = X[in_subcl,:]
    #y_test = y[in_subcl,:]

In [ ]:
print(X.shape)
y.shape

In [ ]:
y_pred.shape
y_train.shape

In [ ]:
rr # elnet a=0.1

In [ ]:
rr # elastic net a=1

In [ ]:
rr # cv 300

In [ ]:
rr # cv 1000

In [ ]:
rr # cv 8000

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import sklearn as sk

import time
t1 = time.time()

do_oob_subclasses=0

X2 = np.copy(X[range(0,X.shape[0],1)])
y2 = np.copy(y[range(0,X.shape[0],1)])
nsel = np.arange(1)
y2=y2[:,0:nannot+nsel.shape[0]]

if do_oob_subclasses==0:
    ## select datasets for training and validation
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 

    #X, y = make_regression(n_features=4, n_informative=2,
    #                        random_state=0, shuffle=False)

    kf = KFold(n_splits=10, shuffle=False)
    i0=0
    r6=[]
    r7=[]
    for train_index, test_index in kf.split(X2):
        i0=i0+1
        print('\n\n '+ str(i0))
        #print("TRAIN:", train_index,"\n", "TEST:", test_index)
        
        X_train, X_test = X2[train_index], X2[test_index]
        y_train, y_test = y2[train_index], y2[test_index]
        
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0) 


        met=1
        
        if met==1:
            regr = RandomForestRegressor(random_state=2026,max_depth=8,min_samples_leaf=1,
                                    n_estimators=2000, oob_score=True, n_jobs=-1, max_samples=500)
                    # RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
            #           max_features='auto', max_leaf_nodes=None,
            #           min_impurity_decrease=0.0, min_impurity_split=None,
            #           min_samples_leaf=1, min_samples_split=2,
            #           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            #           oob_score=False, random_state=0, verbose=0, warm_start=False)
        elif met==2:
            if i0==1:
                regr = sk.linear_model.MultiTaskElasticNetCV(cv=5,n_jobs=-1)
                regr.fit(X[:,nannot:], y[:,nannot:])
                print("alpha ",regr.alpha_)
                print("l1_ratio ",regr.l1_ratio_)
        
            regr = sk.linear_model.MultiTaskElasticNet(alpha=regr.alpha_, l1_ratio=regr.l1_ratio_, fit_intercept=True,
                                                       normalize=False,
                                 max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
        
        elif met==3:
            regr = sk.svm.SVR(kernel='rbf', degree=3, gamma='scale',
                    coef0=0.0, tol=0.001, C=3.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
        
        if met==3:
            #regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
            regr.fit(X_train[:,nannot:], y_train[:,nannot:].ravel())  
        else:
            regr.fit(X_train[:,nannot:], y_train[:,nannot:])


        y_pred = regr.predict(X_test[:,nannot:]) 
        if (met==3)|(met==1):
            y_pred = y_pred.reshape((-1,1))
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test[0:len(nsel)] +stpn_pred[0:len(nsel)])
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,2,4])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+str(i0))
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y_test2[:,i])
                r4=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+str(i0))
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        
        if met==1:
            print('out of bag R^2: '+str(regr.oob_score_))

    rr = np.array([r6,r7])
    rr = pd.DataFrame(rr)
else:
    
    oobs = []
    errors = []
    annot_columns2 = annot_columns
    
    r6=[]
    r7=[]
    for i0 in range(len(subclasses2)):
        in_subcl = ge_data_subcl.loc[:,'subclass_pre']==subclasses2[i0]
        in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_post']==subclasses2[i0])
        in_subcl = np.nonzero(in_subcl.values)[0]

        X_train = np.delete(X, in_subcl,axis=0)
        y_train = np.delete(y, in_subcl,axis=0)
        #X_train = X[:,:]
        #y_train = y[:,:]
        
        X_test = X[in_subcl,:]
        y_test = y[in_subcl,:]
        
        do_this = 1

        X_train, X_test0, y_train, y_test0 = train_test_split(X, y, test_size=0.25, random_state=0) 
        
        print(X_train.shape)
        print(X_test.shape)
        
        if do_this==1:
            #if i0=0:
            regr = RandomForestRegressor(random_state=2026,
                                    n_estimators=500, oob_score=True, n_jobs=-1, max_samples=300)
                
        else:
            regr = RandomForestRegressor(random_state=2026,
                                    n_estimators=500, oob_score=True, n_jobs=-1)
        
        regr.fit(X_train[:,nannot:], y_train[:,nannot:])  
        
        y_pred = regr.predict(X_test[:,nannot:]) 
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,1,9])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+subclasses2[i0])
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y_test2[:,i])
                r4=1 - np.mean((y_test2[:,i] - y_pred[:,i])**2)/np.var(y[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+subclasses2[i0])
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        oobs = oobs + [regr.oob_score_]
    print('out of bag R^2: '+str(oobs))
    
    rr = np.array([subclasses2,r6,r7])
    rr = pd.DataFrame(rr)
    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))



In [ ]:
TO DO : CHECK R^2 for ca and dg, ee cases!!!

In [ ]:
yy

In [ ]:
print(regr.alpha_)
print(regr.__dict__)

In [ ]:
rr # out of 10 cross val

In [ ]:
rr # all data, 300 bs

In [ ]:
rr # all data, 1000 bs

In [ ]:
rr # cross val, 1000 bs

In [ ]:
rr # cross val, 300 bs

In [ ]:
rr # cross val, all bs

In [ ]:
rr  # cross val, 93 classes bs

In [ ]:
rr  # all data, classes bs

In [ ]:
ca1=1.6
ca2=2
k=2.79
# k=1.09

cc1=(k**4 + ca1**4)/ca1**4*ca2**4/(k**4+ca2**4)

k=1.09
cc2=(k**4 + ca1**4)/ca1**4*ca2**4/(k**4+ca2**4)

print(str([cc1,cc2,0.5*(cc1+cc2)]))

## Linear regression model

In [ ]:
# Linear regression

t1 = time.time()

#regr = RandomForestRegressor(max_depth=15, random_state=0,
#                            n_estimators=100)
#regr = RandomForestRegressor(random_state=2026,
#                            n_estimators=500)
import matplotlib.pyplot as plt 
from sklearn import linear_model
regr2 = linear_model.Lasso(alpha=0.01)
regr2.fit(X_train[:,nannot:], y_train[:,nannot:])  
#Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
#   normalize=False, positive=False, precompute=False, random_state=None,
#   selection='cyclic', tol=0.0001, warm_start=False)


#regr.fit(X_train[:,6:], y_train[:,0:].ravel())  
#regr.fit(X_train[:,nannot:], y_train[:,nannot:])  
# RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
#           max_features='auto', max_leaf_nodes=None,
#           min_impurity_decrease=0.0, min_impurity_split=None,
#           min_samples_leaf=1, min_samples_split=2,
#           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#           oob_score=False, random_state=0, verbose=0, warm_start=False)
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))

# some analysis of results
y_predl = regr2.predict(X_test[:,nannot:])  
if len(y_predl.shape)==1:
    y_predl=y_predl.reshape((len(y_predl),1))
    
yyl = np.concatenate((y_test, y_predl),axis=1)
stpn_test=[s+'_test' for s in stp_columns]
stpn_pred=[s+'_pred' for s in stp_columns]
yyl = pd.DataFrame(yyl,columns=annot_columns +stpn_test +stpn_pred)




# make input and output stp labels dataset with errors
stpn_error = [s+'_error' for s in stp_columns]
yel = pd.DataFrame(yyl.loc[:,stpn_pred].values - yyl.loc[:,stpn_test].values, columns=stpn_error)
yyl = pd.concat([yyl, yel.abs()], axis=1)

#yy.set_index(['pre','post']).groupby()
#yy.set_index(['pre','post']).groupby(['pre','post']).mean()

# plot errors
annot_columns2 = annot_columns

fig, ax = plt.subplots(figsize=(16, 10))

#yy1l=yyl.set_index(annot_columns2)
yyl.loc[:,annot_columns2] = yyl.loc[:,annot_columns2].astype(str)
yy1l = yyl.set_index(annot_columns2)
yy1l.loc[:,stpn_error]=yy1l.loc[:,stpn_error].values/(yy1l.loc[:,stpn_test].values+1e-15)*100
yy1l=yy1l.astype(float).groupby(annot_columns2).mean()
yy1l.loc[:,stpn_error].plot(ax=ax)
plt.xticks(np.arange(len(yy1l.index)), yy1l.index, rotation=90)
plt.title('errors of An/A1 predicted by lasso linear regrassion, %')
plt.ylim((0,100))    


## plot test vs train STP data

1. plot all parameters
2. additional cross validation? - better estimate of generalization error?
3. check dependence on rf parameters
4. dependence on % of training data
5. lists of most significant genes
6. check gs assignment - plot some figures
7. project on hipp-ds -> predict stp, compare
8. an:a1 vs TM-par
## 
## new (sep 2019):
9. compare with linear fit
10. iRFcompare small gs predictions
11. compare with factors predicting neuron types, predict n-types using small ds
12. R^2
13. compare with list of STP molecular factors
14. relative errors
15. some graphs for particular factors?
16. iRF




In [ ]:
#annot_columns +stpn_test +stpn_pred


In [ ]:
print(X_train.shape)
print(y_train.shape)
y_test.shape

In [ ]:
# RF case
y_pred = regr.predict(X_test[:,nannot:]) 
if len(y_pred.shape)==1:
    y_pred=y_pred.reshape((len(y_pred),1))





from sklearn import metrics

#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,2:].ravel(), y_pred))  
#print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))  
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,nannot:], y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,nannot:], y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,nannot:], y_pred))) 


#y_pred2 = y_pred.reshape((len(y_pred),nstp))
yy = np.concatenate((y_test, y_pred),axis=1)
stpn_test=[s+'_test' for s in stp_columns]
stpn_pred=[s+'_pred' for s in stp_columns]
yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)




# make input and output stp labels dataset with errors
stpn_error = [s+'_error' for s in stp_columns]
ye = pd.DataFrame(yy.loc[:,stpn_pred].values - yy.loc[:,stpn_test].values, columns=stpn_error)
yy = pd.concat([yy, ye.abs()], axis=1)

#yy.set_index(['pre','post']).groupby()
#yy.set_index(['pre','post']).groupby(['pre','post']).mean()

# plot errors
annot_columns2 = annot_columns

fig, ax = plt.subplots(figsize=(16, 10))

#yy1=yy.set_index(annot_columns2)
yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
yy1 = yy.set_index(annot_columns2)
yy1.loc[:,stpn_error]=yy1.loc[:,stpn_error].values/(yy1.loc[:,stpn_test].values+1e-15)*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
yy1.loc[:,stpn_error].plot(ax=ax)
plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
plt.title('errors of An/A1 predicted by RF, %')
plt.ylim((0,100))

In [ ]:
yy.to_excel('all_tested_rf_280gs_with_PPR.xlsx')
yy.head()

In [ ]:
# Linear model case

#plot predicted and real (test) stp data
#plt.figure()
#yy1l=yyl.set_index(annot_columns2)

#yyl.loc[:,annot_columns2] = yyl.loc[:,annot_columns2].astype(str)
yy1l = yyl.set_index(annot_columns2)
#yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
yy1l=yy1l.astype(float).groupby(annot_columns2).mean()

r3=[]
for i in range(nstp):
    f, ax =plt.subplots(figsize=(16, 10))
    #f, ax = plt.figure()
    #ax = f.add_axes()
    #plt.title(stp_columns[i])
    ax.set_title(stp_columns[i])
    yy1l.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.xticks(np.arange(len(yy1l.index)), yy1l.index, rotation=90)
    if i==0:
        istr=0
    else:
        istr=0 #11 - if ppr included
    
    ytest= yy1l.loc[:,stpn_test[i]]  
    ypred=yy1l.loc[:,stpn_pred[i]]
    
    if ytest.iloc[istr:].var()!=0:
        r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/(ytest.iloc[istr:].var() )
    print(str(i) +' R^2 = '+str(r2))
    r3 = r3 + [r2]

print('mean R^2 = '+str(np.array(r3).mean()))
#print(yy1.mean())
yy1l.loc[:,stpn_test +stpn_pred]

In [ ]:
# RF model case

#plot predicted and real (test) stp data
#plt.figure()

#yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
#yy1 = yy.set_index(annot_columns2)

##yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
r3=[]
for i in range(0,nstp):
    f, ax =plt.subplots(figsize=(16, 10))
    #f, ax = plt.figure()
    #ax = f.add_axes()
    #plt.title(stp_columns[i])
    ax.set_title(stp_columns[i])
    yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    if i==0:
        istr=0
    else:
        istr=0 # 11 - if ppr data excluded
        
    ytest= yy1.loc[:,stpn_test[i]]  
    ypred=yy1.loc[:,stpn_pred[i]]
    #r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
    if ytest.iloc[istr:].var()!=0:
        r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).var()/ytest.iloc[istr:].var()
    print(ytest.iloc[istr:].var())
    print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
    print(str(i) +' R^2 = '+str(r2))
    r3 = r3 + [r2]

print('mean R^2 = '+str(np.array(r3).mean()))

#f, ax =plt.subplots(figsize=(10, 10))
#yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)

#print(yy1.mean())
yy1.loc[:,stpn_test +stpn_pred]

In [ ]:
#yy1.to_excel('results_RF_10gs_without_PPR.xlsx')
yy1.to_excel('results_RF_10gs_with_PPR.xlsx')

## Look at most important genes

In [ ]:
#linear model case

# set of most significant genes
 #fil = regr2.feature_importances_

#plt.plot(fi)
fil = regr2.coef_
fil = np.absolute(fil)
#fil = np.sum(fil,axis=0).transpose()
fil = fil[0,:].transpose()
fil.shape

ifi = np.nonzero(fil)[0]
fi2l = fil[ifi]
ifi2 = np.argsort(fi2l)
l=len(fi2l)
#print(np.concatenate((ifi2.reshape((l,1)),fi2[ifi2].reshape((l,1))),axis=1))

print(len(ge_columns))
print(len(ge_data.columns))
print(fil.shape)
print(ge_columns[0:6])

fi2l = pd.DataFrame(fil,columns=['importance'],index = ge_columns[6:])
fi2l = pd.concat([fi2l, pd.DataFrame(np.repeat('post',len(fi2l.index)),columns = ['compartment'],index=fi2l.index)],axis=1)
#fi2.loc[np.arange(len(fi2.index))<len(pregs),'compartment']='pre'
fi2l.loc[fi2l.index.str.contains('pre'),'compartment']='pre'

fi2l.index =  [id[5:] for id in fi2l.index ]

fi2l=fi2l.sort_values('importance',ascending=False)
best10 = fi2l.iloc[0:10,:]
best20 = fi2l.iloc[0:20,:]
best50 = fi2l.iloc[0:50,:]


## linear case
#fi2l.iloc[0:50,:] # A1:5, without averaging, 500 trees, with Yuste, 384gs

In [ ]:
fi2l.iloc[0:50,:] 

In [ ]:
fi2l.iloc[0:50,:] # A1:5, without averaging, 500 trees,  50 best genes found from 280 gs

In [ ]:
fi2l.iloc[0:50,:] # A1:5, without averaging, 500 trees, with Yuste, 20 best genes found from 280 gs

In [ ]:
# RF model case

# set of most significant genes
fi = regr.feature_importances_
#plt.plot(fi)

ifi = np.nonzero(fi)[0]
fi2 = fi[ifi]
ifi2 = np.argsort(fi2)
l=len(fi2)
#print(np.concatenate((ifi2.reshape((l,1)),fi2[ifi2].reshape((l,1))),axis=1))

print(len(ge_columns))
print(len(ge_data.columns))
print(fi.shape)
print(ge_columns[0:6])

fi2 = pd.DataFrame(fi,columns=['importance'],index = ge_columns[6:])
fi2 = pd.concat([fi2, pd.DataFrame(np.repeat('post',len(fi2.index)),columns = ['compartment'],index=fi2.index)],axis=1)
#fi2.loc[np.arange(len(fi2.index))<len(pregs),'compartment']='pre'
fi2.loc[fi2.index.str.contains('pre'),'compartment']='pre'

fi2.index =  [id[5:] for id in fi2.index ]

fi2=fi2.sort_values('importance',ascending=False)
best10 = fi2.iloc[0:10,:]
best20 = fi2.iloc[0:20,:]
best50 = fi2.iloc[0:50,:]

In [ ]:
fi2.iloc[0:50,:] 

# STOP!!!

In [ ]:
fi2.to_excel('best_genes_hipp.xlsx')

## Compare with VISp trained model

In [ ]:
import pickle
RF_file = open('RF_290_hipp.pickle', mode='wb')
pickle.dump(regr,RF_file)

In [ ]:
import os
os.getcwd()

In [ ]:
RF_file = open('RF_290_visp.pickle', mode='rb')
regr_visp = pickle.load(RF_file)

In [ ]:
# RF case
y_pred = regr_visp.predict(X_test[:,nannot:]) 
if len(y_pred.shape)==1:
    y_pred=y_pred.reshape((len(y_pred),1))





from sklearn import metrics

#print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,2:].ravel(), y_pred))  
#print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))  
#print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,2:].ravel(), y_pred))) 

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test[:,nannot:], y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test[:,nannot:], y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test[:,nannot:], y_pred))) 


#y_pred2 = y_pred.reshape((len(y_pred),nstp))
yy = np.concatenate((y_test, y_pred),axis=1)
stpn_test=[s+'_test' for s in stp_columns]
stpn_pred=[s+'_pred' for s in stp_columns]
yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)




# make input and output stp labels dataset with errors
stpn_error = [s+'_error' for s in stp_columns]
ye = pd.DataFrame(yy.loc[:,stpn_pred].values - yy.loc[:,stpn_test].values, columns=stpn_error)
yy = pd.concat([yy, ye.abs()], axis=1)

#yy.set_index(['pre','post']).groupby()
#yy.set_index(['pre','post']).groupby(['pre','post']).mean()

# plot errors
annot_columns2 = annot_columns

fig, ax = plt.subplots(figsize=(12, 18))

yy1=yy.set_index(annot_columns2)
yy1.loc[:,stpn_error]=yy1.loc[:,stpn_error].values/(yy1.loc[:,stpn_test].values+1e-15)*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
yy1.loc[:,stpn_error].plot(ax=ax)
plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


In [ ]:
# RF model case

#plot predicted and real (test) stp data
#plt.figure()
yy1=yy.set_index(annot_columns2)
#yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
yy1=yy1.astype(float).groupby(annot_columns2).mean()
r3=[]
for i in range(nstp):
    f, ax =plt.subplots(figsize=(10, 10))
    #f, ax = plt.figure()
    #ax = f.add_axes()
    #plt.title(stp_columns[i])
    ax.set_title(stp_columns[i])
    yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
    plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)
    if i==0:
        istr=0
    else:
        istr=0 # 11 - if ppr data excluded
        
    ytest= yy1.loc[:,stpn_test[i]]  
    ypred=yy1.loc[:,stpn_pred[i]]
    #r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
    r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).var()/ytest.iloc[istr:].var()
    print(ytest.iloc[istr:].var())
    print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
    print(str(i) +' R^2 = '+str(r2))
    r3 = r3 + [r2]

print('mean R^2 = '+str(np.array(r3).mean()))

f, ax =plt.subplots(figsize=(10, 10))
yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)

#print(yy1.mean())
yy1.loc[:,stpn_test +stpn_pred]

In [ ]:
fi2.iloc[0:50,:] # A1:5, without averaging, 500 trees,  10 from 280 gs R^2 = 0.86

In [ ]:
#fi2.to_excel('10_selected_from_280_gs.xlsx')

# START!
## iterative RF - search for stable interations of features

## run python version of iRF :

## # PYTHON iRF WORKS ONLY WITH BYNARY CLASSIFICATION DATA!!! - USE R iRF2.0 for multiclasses (or 1D regression)!!!

In [ ]:
#import numpy as np
from irf import irf_utils

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
rng = np.random.RandomState(10)  # deterministic random data
a = np.hstack((rng.normal(size=1000),  rng.normal(loc=5, scale=2, size=1000)))
print(a.shape)
plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

bins =[0.45, 0.75,1.0, 1.5, 5]
y_train[:,nannot]
in_bins =  np.histogram(y_train[:,nannot],10)

In [ ]:
import umap
from sklearn.datasets import load_digits

digits = load_digits()

# embedding = umap.UMAP().fit_transform(digits.data)



embedding = umap.UMAP(n_neighbors=5,
                      min_dist=0.3,
                      metric='correlation').fit_transform(digits.data)

In [ ]:
#import louvain
import pyclustering

In [ ]:
from pyclustering.cluster import cluster_visualizer

In [ ]:
yy=y_train[:,nannot:]
y_u = UMAP(spread=2).fit_transform(yy)

from sklearn.manifold import TSNE

y_u2 = TSNE(n_components=2).fit_transform(yy)

fig = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
plt.scatter(y_u[:, 0], y_u[:, 1],
                edgecolors='none', c="#ff4400", s=10)

plt.axis('off')
fig.set_tight_layout(True)

fig1 = plt.figure(figsize=(10, 10), facecolor='w', edgecolor='k')
plt.scatter(y_u2[:, 0], y_u2[:, 1],
                edgecolors='none', c="#ff4400", s=10)

plt.axis('off')
fig1.set_tight_layout(True)

In [ ]:
colors = ["#991f1f", "#ff9999", "#ff4400", "#ff8800", "#664014", "#665c52",
          "#cca300", "#f1ff33", "#b4cca3", "#0e6600", "#33ff4e", "#00ccbe",
          "#0088ff", "#7aa6cc", "#293966", "#0000ff", "#9352cc", "#cca3c9", "#cc2996"]

fig = plt.figure(figsize=(15, 15), facecolor='w', edgecolor='k')
for i, k in enumerate(list(set(df_aba_vis_in_c_s_vis.loc[:,'cluster_label'] ))):
    i_clu = df_aba_vis_in_c_s_vis.index[df_aba_vis_in_c_s_vis.loc[:,'cluster_label'].isin([k])]
    c =df_aba_vis_in_c_s_vis.loc[i_clu,'cluster_color'].iloc[0]
    
    plt.scatter(latent_u_vis[i_clu, 0], latent_u_vis[i_clu, 1], label=k,
                edgecolors='none', c=c, s=15)
    plt.legend(loc=9, borderaxespad=0, fontsize='small', markerscale=3)

plt.axis('off')
fig.set_tight_layout(True)

## run R iRF in notebook

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
np.save('X.npy',X[:,nannot:].astype(float))
np.save('y.npy',y[:,nannot:].astype(float))

np.save('X_train.npy',X_train[:,nannot:].astype(float))
np.save('X_test.npy',X_test[:,nannot:].astype(float))
np.save('y_train.npy',y_train[:,nannot:].astype(float))
np.save('y_test.npy',y_test[:,nannot:].astype(float))

#np.savetxt('X_train.csv',X_train[:,nannot:].astype(),fmt='%10.5f', delimiter=', ')

In [ ]:
from rpy2.robjects import r, pandas2ri
import rpy2.robjects.numpy2ri

In [ ]:
from rpy2.robjects.packages import importr
irfr = importr('iRF')
bs = importr('base')

In [ ]:
#iRFr=r('iRF')
iRFr = irfr.iRF
X_test_r=X_test[:,nannot:]
X_train_r=X_train[:,nannot:]
y_train_r = y_train[:,nannot:]
y_test_r = y_test[:,nannot:]

import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

X_test_r2 = ro.r.matrix(X_test_r, nrow=X_test_r.shape[0], ncol=X_test_r.shape[1])
X_train_r2 = ro.r.matrix(X_train_r, nrow=X_train_r.shape[0], ncol=X_train_r.shape[1])
y_test_r2 = ro.r.matrix(y_test_r, nrow=y_test_r.shape[0], ncol=y_test_r.shape[1])
y_train_r2 = ro.r.matrix(y_train_r, nrow=y_train_r.shape[0], ncol=y_train_r.shape[1])
ro.r.assign("X_test_r", X_test_r2 )
ro.r.assign("X_train_r", X_train_r2 )

In [ ]:
fit = irfr.iRF(x=X_train_r2, y=y_train_r2, xtest=X_test_r2, ytest=y_test_r2 )

In [ ]:
X_test_r

In [ ]:
# PYTHON iRF WORKS ONLY WITH BYNARY CLASSIFICATION DATA!!! - USE R iRF2.0 for multiclasses (or 1D regression)!!!

from irf import irf_utils
import time
t1 = time.time()

all_rf_weights, all_K_iter_rf_data, \
    all_rf_bootstrap_output, all_rit_bootstrap_output, \
    stability_score = irf_utils.run_iRF(X_train=X_train[:,nannot:],
                                        X_test=X_test[:,nannot:],
                                        y_train=y_train[:,-1],
                                        y_test=y_test[:,-1],
                                        K=5,                          # number of iteration
                                        n_estimators=20,              # number of trees in the forest
                                        B=30,
                                        random_state_classifier=2026, # random seed
                                        propn_n_samples=.2,
                                        bin_class_type=1,
                                        M=20,
                                        max_depth=5,
                                        noisy_split=False,
                                        num_splits=2,
                                        n_estimators_bootstrap=5)


t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))

In [ ]:
fit = iRFr(x=X_train_r, 
         y=y_train_r, 
         xtest=X_test_r, 
         ytest=y_test_r, 
         n.iter=5, 
         n.core=8,
         select.iter = TRUE,
         n.bootstrap=10
        )

## run iRF in R studio

In [ ]:

np.save('X.npy',X[:,nannot:].astype(float))
np.save('y.npy',y[:,nannot:].astype(float))

In [ ]:
### HERE clustering by xmeans_STP.ipynb  should be done

In [ ]:
yc  = np.load('y_clusters.npy')

X_train, X_test, y_train, y_test = train_test_split(X, yc, test_size=0.25, random_state=0)
print(X_train.shape)
print(X_test.shape)
print(set(yc[:,-1]))
print( len(list( set(yc[:,-1]) )))
print(yc.shape)

np.save('Xtrain.npy',X_train[:,nannot:].astype(float))
np.save('yctrain.npy',np.rint(y_train[:,-1]).astype(float))
np.save('Xtest.npy',X_test[:,nannot:].astype(float))
np.save('yctest.npy',np.rint(y_test[:,-1]).astype(float))

In [ ]:
### HERE do iRF.Rmd comutations

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
imp  = np.load('importance.npy')
imp = pd.DataFrame(imp)
imp = pd.concat([imp,pd.DataFrame(ge_columns[6:])],axis=1)
imp.columns = ['importance', 'genes']
imp = imp.set_index('genes')

imp0  = np.load('importance0.npy')
imp0 = pd.DataFrame(imp0)
imp0 = pd.concat([imp0,pd.DataFrame(ge_columns[6:])],axis=1)
imp0.columns = ['importance0', 'genes']
imp0 = imp0.set_index('genes')


# List of important genes : iRF
imp = pd.concat([imp,imp0],axis=1)

imp50 = imp.sort_values('importance',ascending=False).iloc[0:49,:]
plt.plot(imp.sort_values('importance',ascending=False).loc[:,['importance','importance0']].values)
imp50

In [ ]:
#annot_columns + ge_columns[6:]
imp500 = imp.sort_values('importance0',ascending=False).iloc[0:50,:]
imp500

In [ ]:
ge_columns[6:]

In [ ]:
### # train subclasses with best iRF subsets:

In [ ]:
ge_data_subcl

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

from sklearn.model_selection import train_test_split
import time
t1 = time.time()

ge_columns_x = ge_columns[6:]
ge_columns_x = pd.DataFrame(ge_columns_x)

nsel=25
print(imp50.index[0:nsel-1])
selected_columns_x = ge_columns_x.isin(imp50.index[0:nsel-1])
selected_columns_x =ge_columns_x[selected_columns_x.values].index.values 
print(selected_columns_x)
selected_columns_x2 = np.concatenate([np.arange(nannot),selected_columns_x+ nannot])
X_selected = X[:,selected_columns_x2]

do_oob_subclasses=1

subclasses2 = pd.unique(subclasses2 )
print(subclasses2) 

if do_oob_subclasses==1:

    
    oobs = []
    errors = []
    annot_columns2 = annot_columns
    
    r6=[]
    r7=[]
    for i0 in range(len(subclasses2)):
        in_subcl = ge_data_subcl.loc[:,'subclass_pre']==subclasses2[i0]
        in_subcl = in_subcl|(ge_data_subcl.loc[:,'subclass_post']==subclasses2[i0])
        in_subcl = np.nonzero(in_subcl.values)[0]

        X_train = np.delete(X_selected, in_subcl,axis=0)
        y_train = np.delete(y, in_subcl,axis=0)
        #X_train = X[:,:]
        #y_train = y[:,:]
        
        X_test = X_selected[in_subcl,:]
        y_test = y[in_subcl,:]
        
        

        X_train, X_test0, y_train, y_test0 = train_test_split(X_selected, y, test_size=0.25, random_state=0) 
        
        print(X_train.shape)
        print(X_test.shape)
        
        regr = RandomForestRegressor(random_state=2026,
                                    n_estimators=2500, oob_score=True, n_jobs=-1, max_samples=93)
        regr.fit(X_train[:,nannot:], y_train[:,nannot:])  
        
        y_pred = regr.predict(X_test[:,nannot:]) 
        y_test2 = y_test[:,nannot:]
        
        
        yy = np.concatenate((y_test, y_pred),axis=1)
        stpn_test=[s+'_test' for s in stp_columns]
        stpn_pred=[s+'_pred' for s in stp_columns]
        yy = pd.DataFrame(yy,columns=annot_columns +stpn_test +stpn_pred)
        yy.loc[:,annot_columns2] = yy.loc[:,annot_columns2].astype(str)
        yy1 = yy.set_index(annot_columns2)

        ###yy1=yy1.loc[:,stpn_test+stpn_pred] #/yy1.loc[:,'test']*100
        yy1=yy1.astype(float).groupby(annot_columns2).mean()
        r3=[]
        r5=[]
        for i in range(y_pred.shape[1]):
            if (i==np.array([0,1,9])).sum()>0:
                f, ax =plt.subplots(figsize=(16, 10))
                ##f, ax = plt.figure()
                ##ax = f.add_axes()
                ##plt.title(stp_columns[i])
                ax.set_title(stp_columns[i]+' '+', subclass out of bag: '+subclasses2[i0])
                yy1.loc[:,[stpn_test[i],stpn_pred[i]]].plot(ax=ax)
                plt.xticks(np.arange(len(yy1.index)), yy1.index, rotation=90)


            #ytest= yy1.loc[:,stpn_test[i]]  
            #ypred=yy1.loc[:,stpn_pred[i]]
            
            ##r2=1 - (ytest.iloc[istr:] - ypred.iloc[istr:]).pow(2).mean()/ytest.iloc[istr:].var()
            if np.var(y_test2[:,i])!=0:
                r2=1 - np.var(y_test2[:,i] - y_pred[:,i])/np.var(y_test2[:,i])
                r4=1 - np.var(y_test2[:,i] - y_pred[:,i])/np.var(y[:,nannot+i])
           # print(ytest.iloc[istr:].var())
           # print((ytest.iloc[istr:] - ypred.iloc[istr:]).var())
           # print(str(i) +' R^2 = '+str(r2))
            r3 = r3 + [r2]
            r5 = r5 + [r4]
        print('subclass out of bag: '+subclasses2[i0])
        print('mean R^2 = '+str(np.array(r3).mean()))
        print('all R^2 = '+str(np.array(r3)))
        print('mean R^2 total = '+str(np.array(r5).mean()))
        print('all R^2 total = '+str(np.array(r5)))

        r6 = r6 + [np.array(r3).mean()]
        r7 = r7 + [np.array(r5).mean()]
        oobs = oobs + [regr.oob_score_]
    print('out of bag R^2: '+str(oobs))
    
    rr = np.array([subclasses2,r6,r7])
    rr = pd.DataFrame(rr)
    
t2 = time.time()
#print(regr.feature_importances_)
print('Elapsed time: '+str(t2-t1))
#print(regr.predict([[0, 0, 0, 0]]))





In [ ]:
rr

In [ ]:
rr

## Utils

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

def rfr_model(X, y):
    # Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)
    # Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return scores